In [0]:
%pip install pycountry
%pip install uk-postcodes-parsing

# Active Appeals CCD MVP Payment Pending (Silver Layer)
<table style='float:left;'>
   <tbody>
      <tr>
         <td style='text-align: left;'><b>Name: </b></td>
         <td>SILVER_TO_GOLD__PAYMENT_PENDING_JSON</td>
      </tr>
      <tr>
         <td style='text-align: left;'><b>Description: </b></td>
         <td>Notebook dedicated for the payment pending, not common for any other active appeal states.</td>
      </tr>
      <tr>
         <td style='text-align: left;'><b>First Created: </b></td>
         <td>July-2025</td>
      </tr>
      <tr>
         <th style='text-align: left;'><b>Changelog (JIRA ref/initials/date):</b></th>
         <th>Comments</th>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-667">ARIADM-667</a>/NSA/JUL-2025</td>
         <td>Create Silver Staging tables: stg_main_payment_pending_validation, stg_valid_payment_pending_records, stg_invalid_payment_pending_quarantine_records</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-668">ARIADM-668</a>/NSA/JUL-2025</td>
         <td>appealType 1:1 & defaults mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-670">ARIADM-670</a>/NSA/JUL-2025</td>
         <td>appealType logic mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-672">ARIADM-672</a>/NSA/JUL-2025</td>
         <td>caseData 1:1 & defaults mappings</td>
      </tr>
       <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-669">ARIADM-669</a>/NSA/JUL-2025</td>
         <td>appealType 1:1 - Data quality & constriant checks implementation</td>
      </tr>
       <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-671">ARIADM-671</a>/NSA/JUL-2025</td>
         <td>appealType logic - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-707">ARIADM-707</a>/NSA/JUL-2025</td>
         <td>caseData logic mappings - Hearing Centre</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-674">ARIADM-674</a>/NSA/JUL-2025</td>
         <td>caseData logic mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-673">ARIADM-673</a>/NSA/JUL-2025</td>
         <td>caseData 1:1 - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-675">ARIADM-675</a>/NSA/JUL-2025</td>
         <td>caseData logic mappings - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-707">ARIADM-707</a>/NSA/JUL-2025</td>
         <td>caseData logic mappings - Hearing Centre</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-708">ARIADM-708</a>/NSA/JUL-2025</td>
         <td>caseData logic mappings - Hearing Centre - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-857">ARIADM-857</a>/NSA/JUL-2025</td>
         <td>Update Active Bronze Notebook to Include Column Aliasing Rules in Bronze Tables</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-767">ARIADM-767</a>/AM/JUL-2025</td>
         <td>Include 1:1 mapping of legalRepDetails ticket 767</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-768">ARIADM-768</a>/AM/JUL-2025</td>
         <td>Include validation for legalRepDetails ticket 768</td>
      </tr>
       <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-755">ARIADM-755</a>/NSA/JUL-2025</td>
         <td>appellantDetails 1:1 mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-756">ARIADM-756</a>/NSA/JUL-2025</td>
         <td>appellantDetails 1:1 mappings -  - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-771">ARIADM-771</a>/AM/JUL-2025</td>
         <td>legalRepDetails - contact details inc. cleansing logic</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-772">ARIADM-772</a>/AM/JUL-2025</td>
         <td>legalRepDetails - contact details inc. cleansing logic - Data Quality checks</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-757">ARIADM-757</a>/NSA/JUL-2025</td>
         <td>appellantDetails logic mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-758">ARIADM-758</a>/NSA/JUL-2025</td>
         <td>appellantDetails logic mappings  - Data quality & constriant checks implementation</td>
      </tr>
         <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-769">ARIADM-769</a>/AM/JUL-2025</td>
         <td>legalRepDetails - Address logic mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-770">ARIADM-770</a>/AM/JUL-2025</td>
         <td>legalRepDetails - Address logic mappings  - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-709">ARIADM-709</a>/GB/JUL-2025</td>
         <td>flagsLabels defaults mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-710">ARIADM-710</a>/GB/JUL-2025</td>
         <td>flagsLabels logic mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-712">ARIADM-712</a>/GB/JUL-2025</td>
         <td>flagsLabels logic mappings - case & appellant flags</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-713">ARIADM-713</a>/GB/JUL-2025</td>
         <td>flagsLabels logic mappings  - Data quality & constriant checks implementation</td>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-779">ARIADM-779</a>/AM/JUL-2025</td>
         <td>PartyID logic mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-780">ARIADM-780</a>/AM/JUL-2025</td>
         <td>PartyID logic mappings  - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-759">ARIADM-779</a>/AM/AUG-2025</td>
         <td>appellantDetails IN COUNTRY address mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-760">ARIADM-780</a>/AM/AUG-2025</td>
         <td>appellantDetails IN COUNTRY address mappings  - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-761">ARIADM-779</a>/AM/AUG-2025</td>
         <td>appellantDetails OOC address mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-784">ARIADM-784</a>/AM/AUG-2025</td>
         <td>remissionTypes logic mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-785">ARIADM-785</a>/AM/AUG-2025</td>
         <td>remissionTypes logic mappings  - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-759">ARIADM-779</a>/AM/AUG-2025</td>
         <td>appellantDetails IN COUNTRY address mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-760">ARIADM-780</a>/AM/AUG-2025</td>
         <td>appellantDetails IN COUNTRY address mappings  - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-761">ARIADM-779</a>/AM/AUG-2025</td>
         <td>appellantDetails OOC address mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-784">ARIADM-784</a>/AM/AUG-2025</td>
         <td>remissionTypes logic mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-785">ARIADM-785</a>/AM/AUG-2025</td>
         <td>remissionTypes logic mappings  - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-773">ARIADM-773</a>/GB/AUG-2025</td>
         <td>sponsorDetails - logic mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-774">ARIADM-774</a>/GB/AUG-2025</td>
         <td>sponsorDetails - logic mappings - Data quality & checks constrainst checks</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-775">ARIADM-775</a>/GB/AUG-2025</td>
         <td>sponsorDetails - Address logic</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-776">ARIADM-776</a>/GB/AUG-2025</td>
         <td>sponsorDetails - Address logic - Data quality & constraints checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-777">ARIADM-777</a>/GB/AUG-2025</td>
         <td>sponsorDetails - contact details inc. cleansing logic</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-778">ARIADM-778</a>/GB/AUG-2025</td>
         <td>sponsorDetails - contact details inc. cleansing logic - Data quality & constraint checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-762">ARIADM-762</a>/NSA/AUG-2025</td>
         <td>appellantDetails OOC address mappings -  - Data quality & constriant checks implementation</td>
      </tr>
       <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-779">ARIADM-779</a>/NSA/AUG-2025</td>
         <td>appellantDetails contact details & cleansing - logic & mappings</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-780">ARIADM-780</a>/NSA/AUG-2025</td>
         <td>appellantDetails contact details & cleansing -  - Data quality & constriant checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-788">ARIADM-788</a>/NSA/AUG-2025</td>
         <td>homeOffice - logic mappingsn</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-789">ARIADM-789</a>/NSA/AUG-2025</td>
         <td>homeOffice - logic mappings - Data quality & constraint checks implementation</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-791">ARIADM-791</a>/NSA/AUG-2025</td>
         <td>homeOffice - home office reference cleansing</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-792">ARIADM-792</a>/NSA/AUG-2025</td>
         <td>homeOffice - home office reference cleansing - Data quality & constraint checks implementation</td>
      </tr>
       <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-945">ARIADM-945</a>/NSA/AUG-2025</td>
         <td>paymentPending: appellantDetails - logic mappings [new requirement]</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-946">ARIADM-946</a>/NSA/AUG-2025</td>
         <td>paymentPending: homeOffice - logic mappings [new requirement]</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-948">ARIADM-948</a>/NSA/AUG-2025</td>
         <td>paymentPending: caseData - Hearing Centres [new requirement]</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-955">ARIADM-955</a>/NSA/AUG-2025</td>
         <td>appellantAddress - Mandatory sub field (AddressLine1 missing) FIX</td>
      </tr>
   </tbody>
</table>

### Import packages

In [0]:
import dlt
import json
from datetime import datetime
from pyspark.sql.functions import *
import uk_postcodes_parsing

In [0]:
config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env_name = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()

print(f"env_code: {lz_key}")  # This won't be redacted
print(f"env_name: {env_name}")  # This won't be redacted

KeyVault_name = f"ingest{lz_key}-meta002-{env_name}"
print(f"KeyVault_name: {KeyVault_name}") 

In [0]:
# Service principal credentials
client_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-ID")
client_secret = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-SECRET")
tenant_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-TENANT-ID")

# Storage account names
curated_storage = f"ingest{lz_key}curated{env_name}"
checkpoint_storage = f"ingest{lz_key}xcutting{env_name}"
raw_storage = f"ingest{lz_key}raw{env_name}"
landing_storage = f"ingest{lz_key}landing{env_name}"
external_storage = f"ingest{lz_key}external{env_name}"


# Spark config for curated storage (Delta table)
spark.conf.set(f"fs.azure.account.auth.type.{curated_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{curated_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{curated_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{curated_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{curated_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{checkpoint_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{checkpoint_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{checkpoint_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{checkpoint_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{checkpoint_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{raw_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{raw_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{raw_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{raw_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{raw_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{landing_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{landing_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{landing_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{landing_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{landing_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{external_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{external_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{external_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{external_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{external_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

## Set Variables

In [0]:
AppealState = "paymentPending"

# Setting variables for use in subsequent cells
bronze_path = f"abfss://bronze@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/"
silver_path = f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/"
audit_path = f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/AUDIT/{AppealState}"
gold_outputs = f"ARIADM/ACTIVE/CCD/APPEALS/{AppealState}"



# Print all variables
variables = {
    # "read_hive": read_hive,
    
    "bronze_path": bronze_path,
    "silver_path": silver_path,
    "audit_path": audit_path,
    "gold_outputs": gold_outputs,
    "key_vault": KeyVault_name,
    "AppealState": AppealState

}

display(variables)

## PaymentPending: Silver DLT staging table for gold transformation

## Transformation Functions

In [0]:
from pyspark.sql.functions import col, when, lit, array

# AppealType grouping
def appealType(silver_m1):
    conditions = (col("dv_representation").isin('LR', 'AIP')) & (col("lu_appealType").isNotNull())
    lr_condition = (col("dv_representation") == "LR") & (col("lu_appealType").isNotNull())
    aip_condition = (col("dv_representation") == "AIP") & (col("lu_appealType").isNotNull())

    df = silver_m1.select(
        col("CaseNo"), 
        when(
            conditions,
            col("lu_appealType")
        ).otherwise(None).alias("appealType"),
        when(
            conditions,
            col("lu_hmctsCaseCategory")
        ).otherwise(None).alias("hmctsCaseCategory"),
        when(
            conditions,
            col("CaseNo")
        ).otherwise(None).alias("appealReferenceNumber"),
        when(
            conditions,
            col("lu_appealTypeDescription")
        ).otherwise(None).alias("appealTypeDescription"),
        when(
            lr_condition,
            col("lu_caseManagementCategory")
        ).otherwise(None).alias("caseManagementCategory"),
        when(
            aip_condition,
            lit("Yes")
        ).otherwise(lit(None)).alias("isAppealReferenceNumberAvailable"),
        when(
            conditions,
            lit("")
        ).otherwise(lit(None)).alias("ccdReferenceNumberForDisplay")
    )

    common_inputFields = [lit("dv_representation"), lit("lu_appealType")]
    common_inputValues = [col("audit.dv_representation"), col("audit.lu_appealType")]

    df_audit = silver_m1.alias("audit").join(df.alias("content"), ["CaseNo"], "left").select(
        col("CaseNo"),

        #Audit appealType
        array(struct(*common_inputFields)).alias("appealType_inputFields"),
        array(struct(*common_inputValues)).alias("appealType_inputValues"),
        col("content.appealType"),
        lit("yes").alias("appealType_Transformation"),

        #Audit appealReferenceNumber
        array(struct(*common_inputFields, lit("CaseNo"))).alias("appealReferenceNumber_inputfields"),
        array(struct(*common_inputValues, col("CaseNo"))).alias("appealReferenceNumber_inputvalues"),
        col("content.appealReferenceNumber"),
        lit("no").alias("appealReferenceNumber_Transformation"),

        #Audit hmctsCaseCategory
        array(struct(*common_inputFields, lit("lu_hmctsCaseCategory"))).alias("hmctsCaseCategory_inputfields"),
        array(struct(*common_inputValues, col("audit.lu_hmctsCaseCategory"))).alias("hmctsCaseCategory_inputvalues"),
        col("content.hmctsCaseCategory"),
        lit("yes").alias("hmctsCaseCategory_Transformation"),

        #Audit appealTypeDescription
        array(struct(*common_inputFields, lit("lu_appealTypeDescription"))).alias("appealTypeDescription_inputFields"),
        array(struct(*common_inputValues, col("audit.lu_appealTypeDescription"))).alias("appealTypeDescription_inputValues"),
        col("content.appealTypeDescription"),
        lit("yes").alias("appealTypeDescription_Transformation"),

        #Audit caseManagementCategory
        array(struct(*common_inputFields, lit("lu_caseManagementCategory"))).alias("caseManagementCategory_inputFields"),
        array(struct(*common_inputValues, col("audit.lu_caseManagementCategory"))).alias("caseManagementCategory_inputValues"),
        col("content.caseManagementCategory"),
        lit("yes").alias("caseManagementCategory_Transformation"),


        #Audit isAppealReferenceNumberAvailable
        array(struct(*common_inputFields)).alias("isAppealReferenceNumberAvailable_inputFields"),
        array(struct(*common_inputValues)).alias("isAppealReferenceNumberAvailable_inputValues"),
        col("content.isAppealReferenceNumberAvailable"),
        lit("yes").alias("isAppealReferenceNumberAvailable_Transformation"),

        #Audit isAppealReferenceNumberAvailable
        array(struct(*common_inputFields)).alias("ccdReferenceNumberForDisplay_inputFields"),
        array(struct(*common_inputValues)).alias("ccdReferenceNumberForDisplay_inputValues"),
        col("content.ccdReferenceNumberForDisplay"),
        lit("yes").alias("ccdReferenceNumberForDisplay_Transformation")

    )
    
    return df, df_audit

# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()
# df, df_audit = appealType(silver_m1)
# display(df_audit)
# df_audit.printSchema()

In [0]:
from pyspark.sql.functions import collect_list, struct, when, lit, col, max as spark_max, date_format, row_number, split, coalesce
from pyspark.sql.window import Window

# caseData grouping
def caseData(silver_m1, silver_m2, silver_m3, silver_h, bronze_hearing_centres, bronze_derive_hearing_centres):
    # Define the window specification
    window_spec = Window.partitionBy("CaseNo").orderBy(col("HistoryId").desc())

    # Define the base DataFrame
    silver_h = silver_h.filter(col("HistType") == 6)

    # Define the common filter condition
    common_filter = ~col("Comment").like("%Castle Park Storage%") & ~col("Comment").like("%Field House%") & ~col("Comment").like("%UT (IAC)%")

    # Define the function to get the latest history record based on the filter
    def get_latest_history(df, filter_condition):
        return df.filter(filter_condition).withColumn("row_num", row_number().over(window_spec)).filter(col("row_num") == 1).select(
            col("CaseNo"),
            col("Comment"),
            split(col("Comment"), ',')[0].alias("der_prevFileLocation")
        )

    # Create DataFrames for each location
    derived_history_df = get_latest_history(silver_h, common_filter)

    # Define common columns for hearingCentreDynamicList and caseManagementLocationRefData
    common_columns = {
        "hearingCentreDynamicList": struct(
            struct(
                col("locationCode").alias("code"),
                col("locationLabel").alias("label")
            ).alias("value"),
            array(
                struct(lit("227101").alias("code"), lit("Newport Tribunal Centre - Columbus House").alias("label")),
                struct(lit("231596").alias("code"), lit("Birmingham Civil And Family Justice Centre").alias("label")),
                struct(lit("28837").alias("code"), lit("Harmondsworth Tribunal Hearing Centre").alias("label")),
                struct(lit("366559").alias("code"), lit("Atlantic Quay - Glasgow").alias("label")),
                struct(lit("366796").alias("code"), lit("Newcastle Civil And Family Courts And Tribunals Centre").alias("label")),
                struct(lit("386417").alias("code"), lit("Hatton Cross Tribunal Hearing Centre").alias("label")),
                struct(lit("512401").alias("code"), lit("Manchester Tribunal Hearing Centre - Piccadilly Exchange").alias("label")),
                struct(lit("649000").alias("code"), lit("Yarls Wood Immigration And Asylum Hearing Centre").alias("label")),
                struct(lit("698118").alias("code"), lit("Bradford Tribunal Hearing Centre").alias("label")),
                struct(lit("765324").alias("code"), lit("Taylor House Tribunal Hearing Centre").alias("label"))
            ).alias("list_items")
        ),
        "caseManagementLocationRefData": struct(
            lit("1").alias("region"),
            struct(
                struct(
                    col("locationCode").alias("code"),
                    col("locationLabel").alias("label")
                ).alias("value"),
                array(
                    struct(lit("227101").alias("code"), lit("Newport Tribunal Centre - Columbus House").alias("label")),
                    struct(lit("231596").alias("code"), lit("Birmingham Civil And Family Justice Centre").alias("label")),
                    struct(lit("28837").alias("code"), lit("Harmondsworth Tribunal Hearing Centre").alias("label")),
                    struct(lit("366559").alias("code"), lit("Atlantic Quay - Glasgow").alias("label")),
                    struct(lit("366796").alias("code"), lit("Newcastle Civil And Family Courts And Tribunals Centre").alias("label")),
                    struct(lit("386417").alias("code"), lit("Hatton Cross Tribunal Hearing Centre").alias("label")),
                    struct(lit("512401").alias("code"), lit("Manchester Tribunal Hearing Centre - Piccadilly Exchange").alias("label")),
                    struct(lit("649000").alias("code"), lit("Yarls Wood Immigration And Asylum Hearing Centre").alias("label")),
                    struct(lit("698118").alias("code"), lit("Bradford Tribunal Hearing Centre").alias("label")),
                    struct(lit("765324").alias("code"), lit("Taylor House Tribunal Hearing Centre").alias("label"))
                ).alias("list_items")
            ).alias("baseLocation")
        )
    }

    # Join result_df with bronze_derive_hearing_centres to derive derive_hearing_centres
    bronze_derive_hearing_centres = bronze_derive_hearing_centres.withColumn(
        "hearingCentreDynamicList",
        when(
            col("locationCode").isNotNull() & col("locationLabel").isNotNull(),
            common_columns["hearingCentreDynamicList"]
        ).otherwise(None)
    ).withColumn(
        "caseManagementLocationRefData",
        when(
            col("locationCode").isNotNull() & col("locationLabel").isNotNull(),
            common_columns["caseManagementLocationRefData"]
        ).otherwise(None)
    )

    # Join result_df with bronze_hearing_centres to derive hearingCentre
    bronze_hearing_centres = bronze_hearing_centres.withColumn(
        "hearingCentreDynamicList",
        when(
            col("locationCode").isNotNull() & col("locationLabel").isNotNull(),
            common_columns["hearingCentreDynamicList"]
        ).otherwise(None)
    ).withColumn(
        "caseManagementLocationRefData",
        when(
            col("locationCode").isNotNull() & col("locationLabel").isNotNull(),
            common_columns["caseManagementLocationRefData"]
        ).otherwise(None)
    )

    silver_m2 = silver_m2.filter(col("Relationship").isNull())

    # Define postcode mappings
    postcode_mappings = {
        "bradford": ["BD", "DN", "HD", "HG", "HU", "HX", "LS", "S", "WF", "YO"],
        "manchester": ["BB", "BL", "CH", "CW", "FY", "LL", "ST", "L", "LA", "M", "OL", "PR", "SK", "WA", "WN"],
        "newport": ["BA", "BS", "CF", "DT", "EX", "HR", "LD", "NP", "PL", "SA", "SN", "SP", "TA", "TQ", "TR"],
        "taylorHouse": ["AL", "BN", "BR", "CB", "CM", "CO", "CR", "CT", "DA", "E", "EC", "EN", "IG", "IP", "ME", "N", "NR", "NW", "RH", "RM", "SE", "SG", "SS", "TN", "W", "WC"],
        "newcastle": ["CA", "DH", "DL", "NE", "SR", "TS"],
        "birmingham": ["B", "CV", "DE", "DY", "GL", "HP", "LE", "LN", "LU", "MK", "NG", "NN", "OX", "PE", "RG", "SY", "TF", "WD", "WR", "WS", "WV"],
        "hattonCross": ["BH", "GU", "HA", "KT", "PO", "SL", "SM", "SO", "SW", "TW", "UB"],
        "glasgow": ["AB", "DD", "DG", "EH", "FK", "G", "HS", "IV", "KA", "KW", "KY", "ML", "PA", "PH", "TD", "ZE", "BT"]
    }

    # Function to map postcodes to hearing centres
    def map_postcode_to_hearing_centre(postcode):
        if postcode is None:
            return None
        postcode = postcode.replace(" ", "").upper()
        first2 = postcode[:2]
        first1 = postcode[:1]
        for centre, codes in postcode_mappings.items():
            # Try 2-char match first
            if any(first2 == code for code in codes if len(code) == 2):
                return centre
        for centre, codes in postcode_mappings.items():
            # Then try 1-char match
            if any(first1 == code for code in codes if len(code) == 1):
                return centre
        return None

    # Register the UDF
    map_postcode_to_hearing_centre_udf = udf(map_postcode_to_hearing_centre, StringType())

    ##################################################################################################

    result_with_hearing_centre_df = silver_m1.alias("m1").join(
            bronze_hearing_centres.alias("bhc"),
            col("m1.CentreId") == col("bhc.CentreId"),
            how="left").join(derived_history_df.alias("h"),
                                ((col("m1.CaseNo") == col("h.CaseNo")) &
                                (col("m1.CentreId").isin(77,476,101,55,296,13,79,522,406,517,37))),
                                how="left") \
                                .join(bronze_hearing_centres.alias("bhc2"),
                                (col("h.der_prevFileLocation") == col("bhc2.prevFileLocation")),
                                 how="left").join(silver_m2.alias("m2"), col("m1.CaseNo") == col("m2.CaseNo"), how="left") \
                                .withColumn("map_postcode_to_hearing_centre", 
                                           when((col("h.der_prevFileLocation").isin("Arnhem House","Arnhem House (Exceptions)","Loughborough","North Shields (Kings Court)","Not known at this time") | col("h.der_prevFileLocation").isNull()),
                                           map_postcode_to_hearing_centre_udf(coalesce(col('m1.Rep_Postcode'),col('m1.CaseRep_Postcode'),('m2.Appellant_Postcode')))).otherwise(None)) \
                                .join(bronze_derive_hearing_centres.alias("bhc3"), col("map_postcode_to_hearing_centre") == col("bhc3.hearingCentre"), how="left") \
                                .select(
        col("m1.CaseNo"),
        col('bhc.Conditions').alias("lu_conditions"),
        col('bhc.hearingCentre').alias("dv_dhc_hearingCentre"),
        col("h.der_prevFileLocation").alias("dv_prevFileLocation"),
        col('m1.Rep_Postcode'),
        col('m1.CaseRep_Postcode'),
        col("m2.Appellant_Postcode"),
        col("m1.CentreId"),
        col("bhc.prevFileLocation").alias("dv_dhc_prevFileLocation"),
        col("bhc2.hearingCentre").alias("dv_dhc2_hearingCentre"),

        col("bhc2.staffLocation").alias("dv_dhc2_staffLocation"),
        col("bhc3.staffLocation").alias("dv_dhc3_staffLocation"),
        col("bhc2.caseManagementLocation").alias("dv_dhc2_caseManagementLocation"),
        col("bhc3.caseManagementLocation").alias("dv_dhc3_caseManagementLocation"),
        col("bhc2.hearingCentreDynamicList").alias("dv_dhc2_hearingCentreDynamicList"),
        col("bhc3.hearingCentreDynamicList").alias("dv_dhc3_hearingCentreDynamicList"),
        col("bhc2.caseManagementLocationRefData").alias("dv_dhc2_caseManagementLocationRefData"),
        col("bhc3.caseManagementLocationRefData").alias("dv_dhc3_caseManagementLocationRefData"),
        col("bhc2.selectedHearingCentreRefData").alias("dv_dhc2_selectedHearingCentreRefData"),
        col("bhc3.selectedHearingCentreRefData").alias("dv_dhc3_selectedHearingCentreRefData"),
        col("map_postcode_to_hearing_centre"),
        
        when(((col('bhc.Conditions').isNull()) | (col('bhc.Conditions') == 'NO MAPPING REQUIRED')), col('bhc.hearingCentre'))
        .when((col("h.der_prevFileLocation").isin("Arnhem House","Arnhem House (Exceptions)","Loughborough","North Shields (Kings Court)","Not known at this time") | col("h.der_prevFileLocation").isNull()) ,col("map_postcode_to_hearing_centre"))
        .when(col("h.der_prevFileLocation").isin('Castle Park Storage','Field House', 'Field House (TH)','UT (IAC) Cardiff CJC','UT (IAC) Hearing in Field House','UT (IAC) Hearing in Man CJC'), lit(None)).otherwise(col("bhc2.hearingCentre")).alias("hearingCentre"),


        when(((col('bhc.Conditions').isNull()) | (col('bhc.Conditions') == 'NO MAPPING REQUIRED')), col('bhc.staffLocation'))
        .when((col("h.der_prevFileLocation").isin("Arnhem House","Arnhem House (Exceptions)","Loughborough","North Shields (Kings Court)","Not known at this time")| col("h.der_prevFileLocation").isNull()),col("bhc3.staffLocation"))
        .when(col("h.der_prevFileLocation").isin('Castle Park Storage','Field House', 'Field House (TH)','UT (IAC) Cardiff CJC','UT (IAC) Hearing in Field House','UT (IAC) Hearing in Man CJC'), lit(None)).otherwise(col("bhc2.staffLocation"))
         .alias("staffLocation"),


         when(((col('bhc.Conditions').isNull()) | (col('bhc.Conditions') == 'NO MAPPING REQUIRED')), col('bhc.caseManagementLocation'))
        .when((col("h.der_prevFileLocation").isin("Arnhem House","Arnhem House (Exceptions)","Loughborough","North Shields (Kings Court)","Not known at this time") | col("h.der_prevFileLocation").isNull()),col("bhc3.caseManagementLocation"))
        .when(col("h.der_prevFileLocation").isin('Castle Park Storage','Field House', 'Field House (TH)','UT (IAC) Cardiff CJC','UT (IAC) Hearing in Field House','UT (IAC) Hearing in Man CJC'), lit(None)).otherwise(col("bhc2.caseManagementLocation"))
        .alias("caseManagementLocation"),

        when(((col('bhc.Conditions').isNull()) | (col('bhc.Conditions') == 'NO MAPPING REQUIRED')), col('bhc.hearingCentreDynamicList'))
        .when((col("h.der_prevFileLocation").isin("Arnhem House","Arnhem House (Exceptions)","Loughborough","North Shields (Kings Court)","Not known at this time") | col("h.der_prevFileLocation").isNull()),col("bhc3.hearingCentreDynamicList"))
        .when(col("h.der_prevFileLocation").isin('Castle Park Storage','Field House', 'Field House (TH)','UT (IAC) Cardiff CJC','UT (IAC) Hearing in Field House','UT (IAC) Hearing in Man CJC'), lit(None)).otherwise(col("bhc2.hearingCentreDynamicList"))
        .alias("hearingCentreDynamicList"),

        when(((col('bhc.Conditions').isNull()) | (col('bhc.Conditions') == 'NO MAPPING REQUIRED')), col('bhc.caseManagementLocationRefData'))
        .when((col("h.der_prevFileLocation").isin("Arnhem House","Arnhem House (Exceptions)","Loughborough","North Shields (Kings Court)","Not known at this time") | col("h.der_prevFileLocation").isNull()),col("bhc3.caseManagementLocationRefData"))
        .when(col("h.der_prevFileLocation").isin('Castle Park Storage','Field House', 'Field House (TH)','UT (IAC) Cardiff CJC','UT (IAC) Hearing in Field House','UT (IAC) Hearing in Man CJC'), lit(None)).otherwise(col("bhc2.caseManagementLocationRefData"))
        .alias("caseManagementLocationRefData"),

         when(((col('bhc.Conditions').isNull()) | (col('bhc.Conditions') == 'NO MAPPING REQUIRED')), col('bhc.selectedHearingCentreRefData'))
        .when((col("h.der_prevFileLocation").isin("Arnhem House","Arnhem House (Exceptions)","Loughborough","North Shields (Kings Court)","Not known at this time") | col("h.der_prevFileLocation").isNull()),col("bhc3.selectedHearingCentreRefData"))
        .when(col("h.der_prevFileLocation").isin('Castle Park Storage','Field House', 'Field House (TH)','UT (IAC) Cardiff CJC','UT (IAC) Hearing in Field House','UT (IAC) Hearing in Man CJC'), lit(None)).otherwise(col("bhc2.selectedHearingCentreRefData"))
        .alias("selectedHearingCentreRefData")
    )

    # Filter silver_m3 to get rows with max StatusId and Outcome is not null
    # Define window partitioned by CaseNo and ordered by descending StatusId
    window_spec = Window.partitionBy("CaseNo").orderBy(col("StatusId").desc())

    # Add row_number to get the row with the highest StatusId per CaseNo
    silver_m3_ranked = silver_m3.withColumn("row_num", row_number().over(window_spec))

    # Filter the top-ranked rows where Outcome is not null
    silver_m3_filtered = silver_m3_ranked.filter(
        (col("row_num") == 1) & (col("Outcome").isNotNull())
    ).select(
        col("CaseNo"),
        lit("Yes").alias("recordedOutOfTimeDecision"), col("Outcome")
    )

    conditions = (col("dv_representation").isin('LR', 'AIP')) & (col("lu_appealType").isNotNull())

    df = silver_m1.alias("m1").join(
        silver_m3_filtered.alias("m3"),
        on="CaseNo",
        how="left"
    ).join(
        result_with_hearing_centre_df.alias("hearing"),
        on="CaseNo",
        how="left"
    ).withColumn(
        "appellantsRepresentation", when(((col("m1.dv_representation") == "LR") &  (col("lu_appealType").isNotNull())), "No").when(((col("m1.dv_representation") == "AIP") & (col("lu_appealType").isNotNull())), "Yes").otherwise(None)
    ).withColumn(
        "submissionOutOfTime", when(col("OutOfTimeIssue") == 1, lit("Yes")).otherwise(lit("No"))
    ).withColumn(
        "adminDeclaration1", lit(["hasDeclared"])
    ).withColumn(
        "appealWasNotSubmittedReason", when(((col("m1.dv_representation") == "LR") & (col("lu_appealType").isNotNull())), "This is an ARIA Migrated Case.").otherwise(None)
    ).withColumn(
        "applicationOutOfTimeExplanation", when(col("OutOfTimeIssue") == 1, "This is a migrated ARIA case. Please refer to the documents.").otherwise(None)
    ).withColumn(
        "appealSubmissionDate", date_format(col("m1.DateLodged"), "yyyy-MM-dd")
    ).withColumn(
        "appealSubmissionInternalDate", date_format(col("m1.DateLodged"), "yyyy-MM-dd")
    ).withColumn(
        "tribunalReceivedDate", date_format(col("m1.DateAppealReceived"), "yyyy-MM-dd")
    ).select(
        "CaseNo", 
        col("appellantsRepresentation"),
        when(conditions, col("submissionOutOfTime")).otherwise(None).alias("submissionOutOfTime"),
        when(conditions, col("m3.recordedOutOfTimeDecision")).otherwise(None).alias("recordedOutOfTimeDecision"),
        when(conditions, col("applicationOutOfTimeExplanation")).otherwise(None).alias("applicationOutOfTimeExplanation"), 
        when(conditions, col("hearing.hearingCentre")).otherwise(None).alias("hearingCentre"),
        when(conditions, col("hearing.staffLocation")).otherwise(None).alias("staffLocation"),
        when(conditions, col("hearing.caseManagementLocation")).otherwise(None).alias("caseManagementLocation"),
        when(conditions, col("hearing.hearingCentreDynamicList")).otherwise(None).alias("hearingCentreDynamicList"),
        when(conditions, col("hearing.caseManagementLocationRefData")).otherwise(None).alias("caseManagementLocationRefData"),
        when(conditions, col("hearing.selectedHearingCentreRefData")).otherwise(None).alias("selectedHearingCentreRefData"),
        col("appealWasNotSubmittedReason"),
        when(conditions, col("adminDeclaration1")).otherwise(None).alias("adminDeclaration1"),    
        when(conditions, col("appealSubmissionDate")).otherwise(None).alias("appealSubmissionDate"), 
        when(conditions, col("appealSubmissionInternalDate")).otherwise(None).alias("appealSubmissionInternalDate"),
        when(conditions, col("tribunalReceivedDate")).otherwise(None).alias("tribunalReceivedDate"),
        when(conditions, lit([]).cast("array<int>")).otherwise(None).alias("caseLinks"), 
        when(conditions, lit("NotSure")).otherwise(None).alias("hasOtherAppeals")
    )


    common_inputFields = [lit("dv_representation"), lit("lu_appealType")]
    common_inputValues = [col("audit.dv_representation"), col("audit.lu_appealType")]

    df_audit = silver_m1.alias("audit").join(df.alias("content"), ["CaseNo"],"left") \
                        .join(silver_m3_filtered.alias("m3"), ["CaseNo"],"left") \
                        .join(result_with_hearing_centre_df.alias("hearing"),["CaseNo"],"left").select(

    col("CaseNo"),

    #Audit appellantsRepresentation
    array(struct(*common_inputFields)).alias("appellantsRepresentation_inputFields"),
    array(struct(*common_inputValues)).alias("appellantsRepresentation_inputValues"),
    col("content.appellantsRepresentation"),
    lit("yes").alias("appellantsRepresentation_Transformation"),

    #Audit submissionOutOfTime
    array(struct(*common_inputFields,lit("OutOfTimeIssue"))).alias("submissionOutOfTime_inputFields"),
    array(struct(*common_inputValues,col("audit.OutOfTimeIssue"))).alias("submissionOutOfTime_inputValues"),
    col("content.submissionOutOfTime"),
    lit("yes").alias("submissionOutOfTime_Transformation"),

    #Audit recordedOutOfTimeDecision
    array(struct(*common_inputFields,lit("Outcome"))).alias("recordedOutOfTimeDecision_inputFields"),
    array(struct(*common_inputValues,col("m3.Outcome"))).alias("recordedOutOfTimeDecision_inputValues"),
    col("content.recordedOutOfTimeDecision"),
    lit("yes").alias("recordedOutOfTimeDecision_Transformation"),

    #Audit applicationOutOfTimeExplanation
    array(struct(*common_inputFields,lit("OutOfTimeIssue"))).alias("applicationOutOfTimeExplanation_inputFields"),
    array(struct(*common_inputValues,col("audit.OutOfTimeIssue"))).alias("applicationOutOfTimeExplanation_inputValues"),
    col("content.applicationOutOfTimeExplanation"),
    lit("yes").alias("applicationOutOfTimeExplanation_Transformation"),

    #Audit hearingCentre
    array(struct(*common_inputFields,lit("lu_hearingCentre"),lit("lu_conditions"),lit("dv_prevFileLocation"),lit("Rep_Postcode"),lit("CaseRep_Postcode"),lit("Appellant_Postcode"),lit("CentreId"),lit("dv_dhc2_hearingCentre"))).alias("hearingCentre_inputFields"),
    array(struct(*common_inputValues,col("audit.lu_hearingCentre"),col("hearing.lu_conditions"),col("hearing.dv_prevFileLocation"),col("hearing.Rep_Postcode"),col("hearing.CaseRep_Postcode"),col("hearing.Appellant_Postcode"),col("hearing.CentreId"),col("hearing.dv_dhc2_hearingCentre"))).alias("hearingCentre_inputValues"),
    col("content.hearingCentre"),
    lit("yes").alias("hearingCentre_Transformation"),



    #Audit staffLocation
    array(struct(*common_inputFields,lit("lu_staffLocation"),lit("lu_conditions"),lit("dv_prevFileLocation"),lit("Rep_Postcode"),lit("CaseRep_Postcode"),lit("Appellant_Postcode"),lit("CentreId"),lit("dv_dhc2_staffLocation"),lit("dv_dhc3_staffLocation"))).alias("staffLocation_inputFields"),
    array(struct(*common_inputValues,col("audit.lu_staffLocation"),col("hearing.lu_conditions"),col("hearing.dv_prevFileLocation"),col("hearing.Rep_Postcode"),col("hearing.CaseRep_Postcode"),col("hearing.Appellant_Postcode"),col("hearing.CentreId"),col("hearing.dv_dhc2_staffLocation"),lit("hearing.dv_dhc3_staffLocation"))).alias("staffLocation_inputValues"),
    col("content.staffLocation"),
    lit("yes").alias("staffLocation_Transformation"),

        #  when(col('bhc.Conditions').isNull(), col('bhc.staffLocation'))
    #     .when(col("h.der_prevFileLocation").isin("Arnhem House","Arnhem House (Exceptions)","Loughborough","North Shields (Kings Court)","Not known at this time"),col("bhc3.staffLocation"))
    #     .when(col("m1.CentreId").isin(77, 476, 101, 55, 296, 13, 79, 522, 406, 517, 37), col("bhc2.staffLocation"))
    #     .alias("staffLocation"),


    # col("bhc2.staffLocation").alias("dv_dhc2_staffLocation"),
    #     col("bhc3.staffLocation").alias("dv_dhc3_staffLocation"),
    #     col("bhc2.caseManagementLocation").alias("dv_dhc2_caseManagementLocation"),
    #     col("bhc3.caseManagementLocation").alias("dv_dhc3_caseManagementLocation"),
    #     col("bhc2.hearingCentreDynamicList").alias("dv_dhc2_hearingCentreDynamicList"),
    #     col("bhc3.hearingCentreDynamicList").alias("dv_dhc3_hearingCentreDynamicList"),
    #     col("bhc2.caseManagementLocationRefData").alias("dv_dhc2_caseManagementLocationRefData"),
    #     col("bhc3.caseManagementLocationRefData").alias("dv_dhc3_caseManagementLocationRefData"),
    #     col("bhc2.selectedHearingCentreRefData").alias("dv_dhc2_selectedHearingCentreRefData"),
    #     col("bhc3.selectedHearingCentreRefData").alias("dv_dhc3_selectedHearingCentreRefData"),

    #Audit caseManagementLocation
    array(struct(*common_inputFields,lit("lu_hearingCentre"),lit("lu_conditions"),lit("dv_prevFileLocation"),lit("Rep_Postcode"),lit("CaseRep_Postcode"),lit("Appellant_Postcode"),lit("CentreId"),lit("dv_dhc2_caseManagementLocation"),lit("dv_dhc3_caseManagementLocation"))).alias("caseManagementLocation_inputFields"),
    array(struct(*common_inputValues,col("audit.lu_caseManagementLocation"),col("hearing.lu_conditions"),col("hearing.dv_prevFileLocation"),col("hearing.Rep_Postcode"),col("hearing.CaseRep_Postcode"),col("hearing.Appellant_Postcode"),col("hearing.CentreId"),col("hearing.dv_dhc2_caseManagementLocation"),col("hearing.dv_dhc3_caseManagementLocation"))).alias("caseManagementLocation_inputValues"),
    col("content.caseManagementLocation"),
    lit("yes").alias("caseManagementLocation_Transformation"),

    #Audit hearingCentreDynamicList
    array(struct(*common_inputFields,lit("dv_hearingCentreDynamicList"),lit("lu_conditions"),lit("dv_prevFileLocation"),lit("Rep_Postcode"),lit("CaseRep_Postcode"),lit("Appellant_Postcode"),lit("CentreId"),lit("dv_dhc2_hearingCentreDynamicList"),lit("dv_dhc3_hearingCentreDynamicList"))).alias("hearingCentreDynamicList_inputFields"),
    array(struct(*common_inputValues,col("audit.dv_hearingCentreDynamicList"),col("hearing.lu_conditions"),col("hearing.dv_prevFileLocation"),col("hearing.Rep_Postcode"),col("hearing.CaseRep_Postcode"),col("hearing.Appellant_Postcode"),col("hearing.CentreId"),col("hearing.dv_dhc2_hearingCentreDynamicList"),col("hearing.dv_dhc3_hearingCentreDynamicList"))).alias("hearingCentreDynamicList_inputValues"),
    col("content.hearingCentreDynamicList"),
    lit("yes").alias("hearingCentreDynamicList_Transformation"),

    #Audit caseManagementLocationRefData
    array(struct(*common_inputFields,lit("dv_caseManagementLocationRefData"),lit("lu_conditions"),lit("dv_prevFileLocation"),lit("Rep_Postcode"),lit("CaseRep_Postcode"),lit("Appellant_Postcode"),lit("CentreId"),lit("dv_dhc2_caseManagementLocationRefData"),lit("dv_dhc3_caseManagementLocationRefData"))).alias("caseManagementLocationRefData_inputFields"),
    array(struct(*common_inputValues,col("audit.dv_caseManagementLocationRefData"),col("hearing.lu_conditions"),col("hearing.dv_prevFileLocation"),col("hearing.Rep_Postcode"),col("hearing.CaseRep_Postcode"),col("hearing.Appellant_Postcode"),col("hearing.CentreId"),col("hearing.dv_dhc2_caseManagementLocationRefData"),col("hearing.dv_dhc3_caseManagementLocationRefData"))).alias("caseManagementLocationRefData_inputValues"),
    col("content.caseManagementLocationRefData"),
    lit("yes").alias("caseManagementLocationRefData_Transformation"),

    #Audit selectedHearingCentreRefData
    array(struct(*common_inputFields,lit("lu_selectedHearingCentreRefData"),lit("dv_prevFileLocation"),lit("Rep_Postcode"),lit("CaseRep_Postcode"),lit("Appellant_Postcode"),lit("CentreId"),lit("dv_dhc2_caseManagementLocationRefData"),lit("dv_dhc3_caseManagementLocationRefData"))).alias("selectedHearingCentreRefData_inputFields"),
    array(struct(*common_inputValues,col("audit.lu_selectedHearingCentreRefData"),col("hearing.lu_conditions"),col("hearing.dv_prevFileLocation"),col("hearing.Rep_Postcode"),col("hearing.CaseRep_Postcode"),col("hearing.Appellant_Postcode"),col("hearing.CentreId"),col("hearing.dv_dhc2_selectedHearingCentreRefData"),col("hearing.dv_dhc3_selectedHearingCentreRefData"))).alias("selectedHearingCentreRefData_inputValues"),
    col("content.selectedHearingCentreRefData"),
    lit("yes").alias("selectedHearingCentreRefData_Transformation"),

    #Audit appealWasNotSubmittedReason
    array(struct(*common_inputFields)).alias("appealWasNotSubmittedReason_inputFields"),
    array(struct(*common_inputValues)).alias("appealWasNotSubmittedReason_inputValues"),
    col("content.appealWasNotSubmittedReason"),
    lit("yes").alias("appealWasNotSubmittedReason_Transformation"),

    # Audit adminDeclaration1
    array(struct(*common_inputFields)).alias("adminDeclaration1_inputFields"),
    array(struct(*common_inputValues)).alias("adminDeclaration1_inputValues"),
    col("content.adminDeclaration1"),
    lit("yes").alias("adminDeclaration1_Transformation"),

    # Audit appealSubmissionDate
    array(struct(*common_inputFields,lit("DateLodged"))).alias("appealSubmissionDate_inputFields"),
    array(struct(*common_inputValues,col("audit.DateLodged"))).alias("appealSubmissionDate_inputValues"),
    col("content.appealSubmissionDate"),
    lit("yes").alias("appealSubmissionDate_Transformation"),

    # Audit appealSubmissionInternalDate
    array(struct(*common_inputFields,lit("DateLodged"))).alias("appealSubmissionInternalDate_inputFields"),
    array(struct(*common_inputValues,col("audit.DateLodged"))).alias("appealSubmissionInternalDate_inputValues"),
    col("content.appealSubmissionInternalDate"),
    lit("yes").alias("appealSubmissionInternalDate_Transformation"),

    # Audit tribunalReceivedDate
    array(struct(*common_inputFields,lit("DateLodged"))).alias("tribunalReceivedDate_inputFields"),
    array(struct(*common_inputValues,col("audit.DateLodged"))).alias("tribunalReceivedDate_inputValues"),
    col("content.tribunalReceivedDate"),
    lit("yes").alias("tribunalReceivedDate_Transformation"),

    # Audit caseLinks
    array(struct(*common_inputFields)).alias("caseLinks_inputFields"),
    array(struct(*common_inputValues)).alias("caseLinks_inputValues"),
    col("content.caseLinks"),
    lit("yes").alias("caseLinks_Transformation"),

    # Audit hasOtherAppeals
    array(struct(*common_inputFields)).alias("hasOtherAppeals_inputFields"),
    array(struct(*common_inputValues)).alias("hasOtherAppeals_inputValues"),
    col("content.hasOtherAppeals"),
    lit("yes").alias("hasOtherAppeals_Transformation")

    )

    return df, df_audit

# # silver_m5 = spark.table("ariadm_active_appeals.silver_link_detail").filter(col("dv_targetState") == lit(AppealState))
# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()
# silver_m2 = spark.table("ariadm_active_appeals.silver_caseapplicant_detail").filter((col("dv_targetState") == lit(AppealState))).distinct()
# silver_m3 = spark.table("ariadm_active_appeals.silver_status_detail").filter(col("dv_targetState") == lit(AppealState))
# silver_h = spark.table("ariadm_active_appeals.silver_history_detail")
# bronze_hearing_centres = spark.table("ariadm_active_appeals.bronze_hearing_centres")
# bronze_derive_hearing_centres = spark.table("ariadm_active_appeals.bronze_derive_hearing_centres")


# df,df_audit = caseData(silver_m1,silver_m2, silver_m3,silver_h,bronze_hearing_centres,bronze_derive_hearing_centres)
# display(df)
# df.count()

In [0]:
from pyspark.sql.functions import when, lit, col, expr, struct, current_timestamp, date_format, array, collect_set, first, array_contains, size
from pyspark.sql import functions as F

## flagsLabels function
def flagsLabels(silver_m1, silver_m2, silver_c):
    ## caseFlags lookup used for conditional mapping- data comes from Data-Mapping-ARIA-CCD mapping document in the sheet 'APPENDIX-Categories'
    case_flag_lookup = {
        7:  {"name": "Urgent case", "code": "CF0007", "comment": None, "hearing": "No"},
        41: {"name": "Other", "code": "OT0001", "comment": "Expedite", "hearing": "Yes"},
        31: {"name": "Other", "code": "OT0001", "comment": "Expedite", "hearing": "Yes"},
        32: {"name": "Other", "code": "OT0001", "comment": "Expedite", "hearing": "Yes"},
        8:  {"name": "Other", "code": "OT0001", "comment": "Reclassified RFT", "hearing": "Yes"},
        24: {"name": "Other", "code": "OT0001", "comment": "EEA Family Permit", "hearing": "Yes"},
        25: {"name": "RRO (Restricted Reporting Order / Anonymisation)", "code": "CF0012", "comment": None, "hearing": "Yes"}
    }

    ## appellantLevelFlags look up used for conditional mapping- data comes from Data-Mapping-ARIA-CCD mapping document in the sheet 'APPENDIX-Categories'
    appellant_flag_lookup = {
        9:  {"name": "Unaccompanied minor", "code": "PF0013", "comment": None, "hearing": "No"},
        17: {"name": "Foreign national offender", "code": "PF0012", "comment": None, "hearing": "Yes"},
        29: {"name": "Foreign national offender", "code": "PF0012", "comment": None, "hearing": "Yes"},
        30: {"name": "Foreign national offender", "code": "PF0012", "comment": None, "hearing": "Yes"},
        39: {"name": "Foreign national offender", "code": "PF0012", "comment": None, "hearing": "Yes"},
        40: {"name": "Foreign national offender", "code": "PF0012", "comment": None, "hearing": "Yes"},
        48: {"name": "Foreign national offender", "code": "PF0012", "comment": None, "hearing": "Yes"},
        19: {"name": "Foreign national offender", "code": "PF0012", "comment": None, "hearing": "Yes"}
    }

    ## aliases for input tables- silver_m1, silver_m2, silver_c variables created in Transformation: stg_payment_pending_ccd_json_generator
    m1 = silver_m1.alias("m1")
    m2 = silver_m2.alias("m2")
    c = silver_c.alias("c")

    # Join tables needed for transformation
    joined = m1.join(c, on="CaseNo", how="left").join(m2, on="CaseNo", how="left")

    # Grouping by CaseNo to ensure one row per CaseNo, and collecting fields needed for mapping logic
    grouped = joined.groupBy("CaseNo").agg(
        # Collecting all CategoryId into an array to avoid multiples rows per CaseNo
        collect_set("CategoryId").alias("CategoryIds"),
        first("HORef", ignorenulls=True).alias("HORef"),
        first("dv_CCDAppealType", ignorenulls=True).alias("dv_CCDAppealType"),
        first("dv_representation", ignorenulls=True).alias("dv_representation"),
        first("Detained", ignorenulls=True).alias("Detained"),
        first("lu_appealType", ignorenulls=True).alias("lu_appealType")
    )

    ## Building caseFlags struct based off format shown in the mapping document in APPENDIX-Categories sheet
    def make_flag_struct(name, code, comment, hearing):
        return struct(
            lit("").alias("id"),
            struct(
                lit(name).alias("name"),
                array(struct(expr("uuid()").alias("id"), lit("Case").alias("value"))).alias("path"),
                lit("Active").alias("status"),
                lit(code).alias("flagCode"),
                lit(comment).cast("string").alias("flagComment"),
                date_format(current_timestamp(), "yyyy-MM-dd'T'HH:mm:ss'Z'").alias("dateTimeCreated"),
                lit(hearing).alias("hearingRelevant")
            ).alias("value")
        )
    
    ## Building appellantLevelFlags struct based off format shown in the mapping document in APPENDIX-Categories sheet
    def make_appellant_flag_struct(name, code, comment, hearing):
        return struct(
            lit("").alias("id"),
            struct(
                lit(name).alias("name"),
                array(struct(expr("uuid()").alias("id"), lit("Party").alias("value"))).alias("path"),
                lit("Active").alias("status"),
                lit(code).alias("flagCode"),
                lit(comment).cast("string").alias("flagComment"),
                date_format(current_timestamp(), "yyyy-MM-dd'T'HH:mm:ss'Z'").alias("dateTimeCreated"),
                lit(hearing).alias("hearingRelevant")
            ).alias("value")
        )

    ## Creating list of caseFlags for each row based on the conditions in the caseFlags lookup and the extra condition where the field 'HORef' (from silver_m1) is used
    def generate_case_flag_details(col_category_ids, col_horef):
        flags = []
        for cat_id, data in case_flag_lookup.items():
            flags.append((array_contains(col_category_ids, lit(cat_id)), make_flag_struct(**data)))
        flags.append((col_horef.isNotNull(), make_flag_struct("Other", "OT0001", "Dropped Case", "Yes")))

        exprs = [when(cond, array(flag)).otherwise(array()) for cond, flag in flags]
        return F.flatten(F.array(*exprs))
    
    ## Creating list of appellantLevelFlags for each row based on conditions in the appellantLevelFlags lookup and the extra condition where the field 'Detained' (from silver_m2) is used
    def generate_appellant_flag_details(col_category_ids, col_detained):
        flags = []
        for cat_id, data in appellant_flag_lookup.items():
            flags.append((array_contains(col_category_ids, lit(cat_id)), make_appellant_flag_struct(**data)))
        flags.append((col_detained.isin(1,2,4), make_appellant_flag_struct("Detained individual", "PF0019", None, "No")))

        exprs = [when(cond, array(flag)).otherwise(array()) for cond, flag in flags]
        return F.flatten(F.array(*exprs))

    ## Applying flag generators
    grouped = grouped.withColumn("caseFlagDetails", generate_case_flag_details(col("CategoryIds"), col("HORef")))
    grouped = grouped.withColumn("appellantFlagDetails", generate_appellant_flag_details(col("CategoryIds"), col("Detained")))
    ## Formatting caseFlags into final structure or Null/None if no conditions are met
    grouped = grouped.withColumn(
        "caseFlags",
        when(size(col("caseFlagDetails")) > 0,
            struct(
                col("caseFlagDetails").alias("details"),
                lit(None).cast("string").alias("groupId"),
                lit(None).cast("string").alias("visibility")
            )
        ).otherwise(lit(None))
    )
    ## Formatting appellantLevelFlags into final structure or Null/None if no conditions are met
    grouped = grouped.withColumn(
        "appellantLevelFlags",
        when(F.size("appellantFlagDetails") > 0,
            struct(
                col("appellantFlagDetails").alias("details"),
                lit(None).cast("string").alias("groupId"),
                lit("Functional PostDeployment").alias("partyName"),
                lit("Appellant").alias("roleOnCase"),
                lit(None).cast("string").alias("visibility")
            )
        ).otherwise(lit(None))
    )
    ## Adding extra flagsLabels fields with conditions
    grouped = grouped.withColumn("s94bStatus", lit("No"))
    ####### ARIADM-709 ticket condition here #######
    grouped = grouped.withColumn("journeyType", when(col("dv_representation") == "AIP", lit("aip")).otherwise(None))
    grouped = grouped.withColumn("isAdmin", lit("Yes"))
    ####### ARIADM-710 ticket condition here #######
    grouped = grouped.withColumn("isAriaMigratedFeeExemption", when(col("dv_CCDAppealType") == "DA", lit("Yes")).otherwise(lit("No")))
    grouped = grouped.withColumn("isEjp", lit("No"))

    ## Only returning fields where base conditions are met 
    conditions = (col("dv_representation").isin("LR", "AIP")) & (col("lu_appealType").isNotNull())
    df = grouped.select(
        col("CaseNo"),
        # col("CategoryIds"),
        # col("Detained"),
        when(conditions, col("caseFlags")).otherwise(None).alias("caseFlags"),
        when(conditions, col("appellantLevelFlags")).otherwise(None).alias("appellantLevelFlags"),
        when(conditions, col("s94bStatus")).otherwise(None).alias("s94bStatus"),
        col("journeyType"),
        when(conditions, col("isAdmin")).otherwise(None).alias("isAdmin"),
        when(conditions, col("isAriaMigratedFeeExemption")).otherwise(None).alias("isAriaMigratedFeeExemption"),
        when(conditions, col("isEjp")).otherwise(None).alias("isEjp")
    )

    common_inputFields = [lit("dv_representation"), lit("lu_appealType")]
    common_inputValues = [col("audit.dv_representation"), col("audit.lu_appealType")]

    df_audit = silver_m1.alias("audit").join(df.alias("content"), ["CaseNo"], "left") \
                        .join(silver_m2.alias("m2"), ["CaseNo"], "left") \
                        .join(grouped.alias("grp"), ["CaseNo"], "left").select(
        col("CaseNo"),

        #Audit caseFlags
        array(struct(*common_inputFields, lit("CategoryIds"), lit("HORef"))).alias("caseFlags_inputFields"),
        array(struct(*common_inputValues, col("grp.CategoryIds"), col("audit.HORef"))).alias("caseFlags_inputValues"),
        col("content.caseFlags"),
        lit("yes").alias("caseFlags_Transformation"),

        #Audit appellantLevelFlags
        array(struct(*common_inputFields, lit("CategoryIds"), lit("Detained"))).alias("appellantLevelFlags_inputFields"),
        array(struct(*common_inputValues, col("grp.CategoryIds"), col("m2.Detained"))).alias("appellantLevelFlags_inputValues"),
        col("content.appellantLevelFlags"),
        lit("yes").alias("appellantLevelFlags_Transformation"),

        #Audit s94bStatus
        array(struct(*common_inputFields)).alias("s94bStatus_inputFields"),
        array(struct(*common_inputValues)).alias("s94bStatus_inputValues"),
        col("content.s94bStatus"),
        lit("yes").alias("s94bStatus_Transformation"),

        #Audit journeyType
        array(struct(*common_inputFields)).alias("journeyType_inputFields"),
        array(struct(*common_inputValues)).alias("journeyType_inputValues"),
        col("content.journeyType"),
        lit("yes").alias("journeyType_Transformation"),

        #Audit isAdmin
        array(struct(*common_inputFields)).alias("isAdmin_inputFields"),
        array(struct(*common_inputValues)).alias("isAdmin_inputValues"),
        col("content.isAdmin"),
        lit("yes").alias("isAdmin_Transformation"),

        #Audit isAriaMigratedFeeExemption
        array(struct(*common_inputFields, lit("dv_CCDAppealType"))).alias("isAriaMigratedFeeExemption_inputFields"),
        array(struct(*common_inputValues, col("audit.dv_CCDAppealType"))).alias("isAriaMigratedFeeExemption_inputValues"),
        col("content.isAriaMigratedFeeExemption"),
        lit("yes").alias("isAriaMigratedFeeExemption_Transformation"),

        #Audit isEjp
        array(struct(*common_inputFields)).alias("isEjp_inputFields"),
        array(struct(*common_inputValues)).alias("isEjp_inputValues"),
        col("content.isEjp"),
        lit("yes").alias("isEjp_Transformation"),
    )
                        
    return df, df_audit

# AppealState = "paymentPending"
# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail") \
#     .filter(col("dv_targetState") == lit(AppealState)) \
#     .distinct()
# silver_m2 = spark.table("ariadm_active_appeals.silver_caseapplicant_detail") 
# silver_c = spark.table("ariadm_active_appeals.silver_appealcategory_detail")
# df, df_audit = flagsLabels(silver_m1, silver_m2, silver_c)
# display(df)
# display(df_audit)
# df.count()


In [0]:
import re
from pyspark.sql.functions import udf, coalesce, col
from pyspark.sql.types import StringType

# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState))

def cleanEmail(email):
    if email is None:
        return None
    
    email = re.sub(r"\s+", "", email)             # Remove all whitespace
    email = re.sub(r"\s", "", email)              # 2. Remove internal whitespace
    email = re.sub(r"^\.", "", email)             # 3. Remove leading .
    email = re.sub(r"\.$", "", email)             # 4. Remove trailing .
    email = re.sub(r"^,", "", email)              # 5. Remove leading ,
    email = re.sub(r",$", "", email)              # 6. Remove trailing ,
    email = re.sub(r"^[()]", "", email)           # 7. Remove leading parenthesis
    email = re.sub(r"[()]$", "", email)           # 8. Remove trailing parenthesis
    email = re.sub(r"^:", "", email)              # 9. Remove leading colon
    email = re.sub(r":$", "", email)              #10. Remove trailing colon
    email = re.sub(r"^\*", "", email)             #11. Remove leading asterisk
    email = re.sub(r"\*$", "", email)             #12. Remove trailing asterisk
    email = re.sub(r"^;", "", email)              #13. Remove leading semicolon
    email = re.sub(r";$", "", email)              #14. Remove trailing semicolon
    email = re.sub(r"^\?", "", email)             #15. Remove leading question
    email = re.sub(r"\?$", "", email)             #16. Remove trailing question
    email = email.strip()                         # 1. Trim spaces

    return email

##Register the UDF
cleanEmailUDF = udf(cleanEmail, StringType())


def cleanPhoneNumber(PhoneNumber):
    if PhoneNumber is None:
        return None
    
    
    PhoneNumber = re.sub(r"\s+", "", PhoneNumber)             # 1. Remove internal whitespace
    PhoneNumber = re.sub(r"\s", "", PhoneNumber)              # 2. Remove internal whitespace
    PhoneNumber = re.sub(r"^\.", "", PhoneNumber)             # 3. Remove leading .
    PhoneNumber = re.sub(r"\.$", "", PhoneNumber)             # 4. Remove trailing .
    PhoneNumber = re.sub(r"^,", "", PhoneNumber)              # 5. Remove leading ,
    PhoneNumber = re.sub(r",$", "", PhoneNumber)              # 6. Remove trailing ,
    PhoneNumber = re.sub(r"^[()]", "", PhoneNumber)           # 7. Remove leading parenthesis
    PhoneNumber = re.sub(r"[()]$", "", PhoneNumber)           # 8. Remove trailing parenthesis
    PhoneNumber = re.sub(r"^:", "", PhoneNumber)              # 9. Remove leading colon
    PhoneNumber = re.sub(r":$", "", PhoneNumber)              #10. Remove trailing colon
    PhoneNumber = re.sub(r"^\*", "", PhoneNumber)             #11. Remove leading asterisk
    PhoneNumber = re.sub(r"\*$", "", PhoneNumber)             #12. Remove trailing asterisk
    PhoneNumber = re.sub(r"^;", "", PhoneNumber)              #13. Remove leading semicolon
    PhoneNumber = re.sub(r";$", "", PhoneNumber)              #14. Remove trailing semicolon
    PhoneNumber = re.sub(r"^\?", "", PhoneNumber)             #15. Remove leading question
    PhoneNumber = re.sub(r"\?$", "", PhoneNumber)             #16. Remove trailing question
    PhoneNumber = re.sub(r"[.\-]", "", PhoneNumber)           #17. Remove internal dots and dashes
    PhoneNumber = re.sub(r"^0044", "+44", PhoneNumber)        #18. Change starting 0044 to +44
    PhoneNumber = re.sub(r"\(0\)", "", PhoneNumber)           #19. Remove (0) if +44 or 44 exists in the number
    PhoneNumber = re.sub(r"^44", "+44", PhoneNumber)          #20. Add missing + if number starts with 44 (but not already +)
    PhoneNumber = PhoneNumber.strip()                         #21. Trim spaces

    return PhoneNumber

##Register the UDF
phoneNumberUDF = udf(cleanPhoneNumber, StringType())

# df = silver_m1.withColumn(
#     "legalRepEmail",
#     cleanEmailUDF(
#         coalesce(col("Rep_Email"), col("CaseRep_Email"), col("CaseRep_FileSpecific_Email"))
    
#     )).withColumn(
#         "sponsorMobileNumberAdminJ",
#         phoneNumberUDF("Sponsor_Telephone")

# ).select(
#     col("CaseNo"),
#     "legalRepEmail",
#     "sponsorMobileNumberAdminJ"
# )

# display(df)

In [0]:
from pyspark.sql.functions import col, udf, concat_ws
from pyspark.sql.types import StringType
import pycountry
import re, string
from uk_postcodes_parsing import fix, postcode_utils

###############UDF1 - Based on Bella's function to determine if an address is in the UK

# Step 1: list of countries
all_countries = pycountry.countries
lower_countries = [country.name.lower() for country in all_countries]

translator = str.maketrans('', '', string.punctuation)

# Step 2: UDF to combine address fields
def makeFullAddress(
    a1, a2, a3, a4, a5, postcode
):
    return ', '.join(map(str, [a1, a2, a3, a4, a5, postcode]))

makeFullAddressUDF = udf(makeFullAddress, StringType())

# Step 3: UDF to check if postcode is UK (simplified)
def getUkPostcode(postcode):
    try:
        if postcode is None or str(postcode).strip() == "":
            return "False"
        
        # Step 1: Check if postcode is valid
        if postcode_utils.is_valid(postcode):
            return "True"
        
        # Step 2: Try fixing the postcode if not valid
        clean_postcode = fix(postcode)
        if postcode_utils.is_valid(clean_postcode):
            return "True"

    except Exception as e:
        print(f"Error processing {postcode}: {e}")
        pass

    return "False"

getUkPostcodeUDF = udf(getUkPostcode, StringType())

# Step 4: UDF to find a country from address

def getCountryFromAddress(address):
    countries_matched = []
    low_address = address.lower()

    translator = str.maketrans('', '', string.punctuation)

    split_address = low_address.split(' ')
    for word in reversed(split_address):
        if word.strip() in lower_countries:
            countries_matched.append(word.strip().capitalize())
            break

    if len(countries_matched) > 0:
        return ', '.join(countries_matched)

    for word in reversed(split_address):
        clean_word = word.translate(translator)
        if clean_word.lower().strip() in lower_countries:
            countries_matched.append(clean_word.strip().capitalize())
            break

    if len(countries_matched) > 0:
        return ', '.join(countries_matched)

    split_address = low_address.split(' ')
    for word in reversed(split_address):
        clean_word = word.translate(translator)
        if clean_word.strip() in lower_countries:
            countries_matched.append(clean_word.strip().capitalize())
            break

    if len(countries_matched) > 0:
        return ', '.join(countries_matched)

    split_address = re.split(r'[,\.\s]+', low_address)
    for word in split_address:
        clean_word = word.translate(translator)
        if clean_word.strip() in lower_countries:
            countries_matched.append(clean_word.strip().capitalize())
            break

    return ', '.join(countries_matched)

getCountryFromAddressUDF = udf(getCountryFromAddress, StringType())

# Step 5: UDF to decide if LR address is UK
def legalRepHasAddress(repId, clean_postcode, full_address):
    if repId > 0:
        return "Yes"

    if repId == 0:
        if clean_postcode == "True":
            return "Yes"
        elif clean_postcode == "False":
            country = getCountryFromAddress(full_address)
            if country == "United Kingdom":
                return "Yes"
            elif country != "United Kingdom":
                return "No"
            elif country == '':
                return "Yes"
    return "No"

legalRepHasAddressUDF = udf(legalRepHasAddress, StringType())

###################### UDF function to replace get_country_LR ######################

def getCountryLR(full_address):

    if full_address is None:
        return ''
    
    address_clean = full_address.lower().translate(translator)
    
    for idx, lower_country in enumerate(lower_countries):
        if lower_country in address_clean:
            return lower_country.title()
    
    return ''

getCountryLRUDF = udf(getCountryLR, StringType())

In [0]:
from pyspark.sql.functions import col, udf, concat_ws
from pyspark.sql.types import StringType
# import postcodes_uk
import pycountry
import re
import string

##Create legalRepDetails fields

def legalRepDetails(silver_m1):
    conditions_legalRepDetails = (col("dv_representation") == 'LR') & (col("lu_appealType").isNotNull())

    df = silver_m1.alias("m1").filter(conditions_legalRepDetails).withColumn(
        "legalRepGivenName",
        coalesce(col("Contact"), col("Rep_Name"), col("CaseRep_Name"))   #If contact is null use RepName, if RepName null use CaseRepName
    ).withColumn(
        "legalRepFamilyNamePaperJ",
        coalesce(col("Rep_Name"), col("CaseRep_Name"))                     #If RepName is null use CaseRepName
    ).withColumn(
        "legalRepCompanyPaperJ",
        coalesce(col("Rep_Name"), col("CaseRep_Name"))                     #If RepName is null use CaseRepName
    ).withColumn(
        "localAuthorityPolicy",
        lit("""{
            "Organisation": {},
            "OrgPolicyCaseAssignedRole": "[LEGALREPRESENTATIVE]"
        }""")
    ).withColumn(
    "legalRepEmail",
    cleanEmailUDF(
        coalesce(col("Rep_Email"), col("CaseRep_Email"), col("CaseRep_FileSpecific_Email")))
    ).withColumn(
        "legalRepAddressUK",               
        when(col("RepresentativeId") == 0, concat_ws(" ",      #If Representiative (solicitor) lives outside UK
        col("CaseRep_Address1"), col("CaseRep_Address2"),         #Concat non-null fields by space
        col("CaseRep_Address3"), col("CaseRep_Address4"),
        col("CaseRep_Address5"), col("CaseRep_Postcode")
        
        )).otherwise(concat_ws(" ",                              #If representative lives inside UK
        col("Rep_Address1"), col("Rep_Address2"),                   #Concat non-null fields by space
        col("Rep_Address3"), col("Rep_Address4"),
        col("Rep_Address5"), col("Rep_Postcode")
    ))

    ## Apply UDFs to determine if the legalRep has a UK address - 1. pull postcode. UK postcode? 
    ).withColumn(
    "ukPostcode",
    getUkPostcodeUDF(col("CaseRep_Postcode"))

    ## Apply UDFs to determine if the legalRep has a UK address - 2. Pull country. Is the country United Kingdom?
    ).withColumn(
    "countryFromAddress",
    getCountryFromAddressUDF(col("legalRepAddressUK"))

    ## Apply UDFs to determine if the legalRep has a UK address - 3. Is the Postcode or country from the UK?
    ).withColumn(
    "legalRepHasAddress",
    legalRepHasAddressUDF(
        col("RepresentativeId"),
        col("ukPostcode"),
        "countryFromAddress"
    )
                                                                             
    ).withColumn("legalRepAddressUK", col("legalRepAddressUK")

    ).withColumn("oocAddressLine1",                                            #If Address1 is null use 2. If 1,2 null use 3
                when((col("CaseRep_Address1").isNotNull()), col("CaseRep_Address1")
                      ).otherwise(coalesce(col("CaseRep_Address2"),
                          col("CaseRep_Address3"), col("CaseRep_Address4"),
                          col("CaseRep_Address5"))
                          )
                 
    ).withColumn("oocAddressLine2", 
                when((col("CaseRep_Address2").isNotNull()), col("CaseRep_Address2")
                      ).otherwise(coalesce(col("CaseRep_Address3"),
                          col("CaseRep_Address4"), col("CaseRep_Address5"))
                )
                 
    ).withColumn("oocAddressLine3",
                when((col("CaseRep_Address3").isNotNull()), concat(col("CaseRep_Address3"), lit(", "), col("CaseRep_Address4"))
                      ).otherwise(col("CaseRep_Address4")
                )
    
    ).withColumn("oocAddressLine4",
                when((col("CaseRep_Address5").isNotNull()), concat(col("CaseRep_Address5"), lit(", "), col("CaseRep_Postcode"))
                      ).otherwise(col("CaseRep_Postcode")
                )
                 
    ).withColumn(
        "oocLrCountryGovUkAdminJ",
        getCountryLRUDF(col("legalRepAddressUK"))     #Use UDF getCountryLR(). Sample data indicates function works.
                 
    ).select(
        col("CaseNo"),
        "legalRepGivenName",
        "legalRepFamilyNamePaperJ",
        "legalRepCompanyPaperJ",
        "localAuthorityPolicy",
        "legalRepEmail",
        "legalRepAddressUK",         
        "legalRepHasAddress",
        "oocAddressLine1",
        "oocAddressLine2",
        "oocAddressLine3",
        "oocAddressLine4",
        "oocLrCountryGovUkAdminJ"
    )

    common_inputFields = [lit("dv_representation"), lit("lu_appealType")]
    common_inputValues = [col("m1_audit.dv_representation"), col("m1_audit.lu_appealType")]

    df_audit = silver_m1.alias("m1_audit").join(df.alias("content"), on = ["CaseNo"], how = "left").filter(conditions_legalRepDetails).select(
        col("CaseNo"),

        ## legalRepGivenName - ARIADM-767
        array(struct(*common_inputFields ,lit("m1_audit.Contact"), lit("m1_audit.Rep_Name"), lit("m1_audit.CaseRep_Name"))).alias("legalRepGivenName_inputFields"),
        array(struct(*common_inputValues ,col("m1_audit.Contact"), col("m1_audit.Rep_Name"), col("m1_audit.CaseRep_Name"))).alias("legalRepGivenName_inputValues"),
        col("content.legalRepGivenName"),
        lit("yes").alias("legalRepGivenName_Transformed"),

        # ## legalRepFamilyNamePaperJ - ARIADM-767
        array(struct(*common_inputFields ,lit("m1_audit.Rep_Name"), lit("m1_audit.CaseRep_Name"))).alias("legalRepFamilyNamePaperJ_inputFields"),
        array(struct(*common_inputValues ,col("m1_audit.Rep_Name"), col("m1_audit.CaseRep_Name"))).alias("legalRepFamilyNamePaperJ_inputValues"),
        col("content.legalRepFamilyNamePaperJ"),
        lit("yes").alias("legalRepFamilyNamePaperJ_Transformed"),

        # ## legalRepCompanyPaperJ - ARIADM-767
        array(struct(*common_inputFields ,lit("m1_audit.Rep_Name"), lit("m1_audit.CaseRep_Name"))).alias("legalRepCompanyPaperJ_inputFields"),
        array(struct(*common_inputValues ,col("m1_audit.Rep_Name"), col("m1_audit.CaseRep_Name"))).alias("legalRepCompanyPaperJ_inputValues"),
        col("content.legalRepCompanyPaperJ"),
        lit("yes").alias("legalRepCompanyPaperJ_Transformed"),

        # ## localAuthorityPolicy - ARIADM-767
        array(struct(*common_inputFields)).alias("localAuthorityPolicy_inputFields"),
        array(struct(*common_inputValues)).alias("localAuthorityPolicy_inputValues"),
        col("content.localAuthorityPolicy"),
        lit("no").alias("localAuthorityPolicy_Transformed"),

        # ## legalRepEmail - ARIADM-771
        array(struct(*common_inputFields, lit("m1_audit.Rep_Email"), lit("m1_audit.CaseRep_Email"), lit("m1_audit.CaseRep_FileSpecific_Email"))).alias("legalRepEmail_inputFields"),
        array(struct(*common_inputValues, col("m1_audit.Rep_Email"), col("m1_audit.CaseRep_Email"), col("m1_audit.CaseRep_FileSpecific_Email"))).alias("legalRepEmail_inputValues"),
        col("content.legalRepEmail"), 
        lit("yes").alias("legalRepEmail_Transformed"),

        ## legalRepAddressUK - ARIADM-769 - Check both CaseRepAddress and RepAddress
        array(struct(*common_inputFields, lit("m1_audit.Rep_Address1"), lit("m1_audit.Rep_Address2"), lit("m1_audit.Rep_Address3"),lit("m1_audit.Rep_Address4"),lit("m1_audit.Rep_Address5"),lit("m1_audit.Rep_Postcode"),lit("m1_audit.CaseRep_Address1"), lit("m1_audit.CaseRep_Address2"), lit("m1_audit.CaseRep_Address3"),lit("m1_audit.CaseRep_Address4"),lit("m1_audit.CaseRep_Address5"),lit("m1_audit.CaseRep_Postcode") )).alias("legalRepAddressUK_inputFields"),
        array(struct(*common_inputValues, col("m1_audit.Rep_Address1"), col("m1_audit.Rep_Address2"), col("m1_audit.Rep_Address3"),col("m1_audit.Rep_Address4"),col("m1_audit.Rep_Address5"),col("m1_audit.Rep_Postcode"), col("m1_audit.CaseRep_Address1"), col("m1_audit.CaseRep_Address2"), col("m1_audit.CaseRep_Address3"),col("m1_audit.CaseRep_Address4"),col("m1_audit.CaseRep_Address5"),col("m1_audit.CaseRep_Postcode"))).alias("legalRepAddressUK_inputValues"),
        col("content.legalRepAddressUK"), 
        lit("yes").alias("legalRepAddressUK_Transformed"),

        ## legalRepHasAddress - ARIADM-769
        array(struct(*common_inputFields, lit("m1_audit.CaseRep_Postcode"), lit("content.legalRepAddressUK"), lit("m1_audit.RepresentativeId"))).alias("legalRepHasAddress_inputFields"),
        array(struct(*common_inputValues, col("m1_audit.CaseRep_Postcode"), col("content.legalRepAddressUK"), col("m1_audit.RepresentativeId"))).alias("legalRepHasAddress_inputValues"),
        col("content.legalRepHasAddress"), 
        lit("yes").alias("legalRepHasAddress_Transformed"),

        ## oocAddressLine1 - ARIADM-769
        array(struct(*common_inputFields, lit("m1_audit.CaseRep_Address1"), lit("m1_audit.CaseRep_Address2"), lit("m1_audit.CaseRep_Address3"),lit("m1_audit.CaseRep_Address4"),lit("m1_audit.CaseRep_Address5"),lit("m1_audit.CaseRep_Postcode"))).alias("oocAddressLine1_inputFields"),
        array(struct(*common_inputValues, col("m1_audit.CaseRep_Address1"), col("m1_audit.CaseRep_Address2"), col("m1_audit.CaseRep_Address3"),col("m1_audit.CaseRep_Address4"),col("m1_audit.CaseRep_Address5"),col("m1_audit.CaseRep_Postcode"))).alias("oocAddressLine1_inputValues"),
        col("content.oocAddressLine1"), 
        lit("yes").alias("oocAddressLine1_Transformed"),

        ## oocAddressLine2 - ARIADM-769
        array(struct(*common_inputFields, lit("m1_audit.CaseRep_Address2"), lit("m1_audit.CaseRep_Address3"),lit("m1_audit.CaseRep_Address4"),lit("m1_audit.CaseRep_Address5"),lit("m1_audit.CaseRep_Postcode"))).alias("oocAddressLine2_inputFields"),
        array(struct(*common_inputValues, col("m1_audit.CaseRep_Address2"), col("m1_audit.CaseRep_Address3"),col("m1_audit.CaseRep_Address4"),col("m1_audit.CaseRep_Address5"),col("m1_audit.CaseRep_Postcode"))).alias("oocAddressLine2_inputValues"),
        col("content.oocAddressLine2"), 
        lit("yes").alias("oocAddressLine2_Transformed"),

        ## oocAddressLine3 - ARIADM-769
        array(struct(*common_inputFields, lit("m1_audit.CaseRep_Address3"),lit("m1_audit.CaseRep_Address4"),lit("m1_audit.CaseRep_Address5"),lit("m1_audit.CaseRep_Postcode"))).alias("oocAddressLine3_inputFields"),
        array(struct(*common_inputValues, col("m1_audit.CaseRep_Address3"),col("m1_audit.CaseRep_Address4"),col("m1_audit.CaseRep_Address5"),col("m1_audit.CaseRep_Postcode"))).alias("oocAddressLine3_inputValues"),
        col("content.oocAddressLine3"), 
        lit("yes").alias("oocAddressLine3_Transformed"),

        ## oocAddressLine4 - ARIADM-769
        array(struct(*common_inputFields, lit("m1_audit.CaseRep_Address4"),lit("m1_audit.CaseRep_Address5"),lit("m1_audit.CaseRep_Postcode"))).alias("oocAddressLine4_inputFields"),
        array(struct(*common_inputValues, col("m1_audit.CaseRep_Address4"),col("m1_audit.CaseRep_Address5"),col("m1_audit.CaseRep_Postcode"))).alias("oocAddressLine4_inputValues"),
        col("content.oocAddressLine4"), 
        lit("yes").alias("oocAddressLine4_Transformed"),

        ## oocLrCountryGovUkAdminJ - ARIADM-769
        array(struct(*common_inputFields, lit("content.oocAddressLine1"),lit("content.oocAddressLine2"),
        lit("content.oocAddressLine3"),lit("content.oocAddressLine4"), lit("m1_audit.CaseRep_Postcode"))).alias("oocLrCountryGovUkAdminJ_inputFields"),
        array(struct(*common_inputValues, col("content.oocAddressLine1"),col("content.oocAddressLine2"),
        col("content.oocAddressLine3"),col("content.oocAddressLine4"), col("m1_audit.CaseRep_Postcode"))).alias("oocLrCountryGovUkAdminJ_inputValues"),
        col("content.oocLrCountryGovUkAdminJ"), 
        lit("yes").alias("oocLrCountryGovUkAdminJ_Transformed"),
    )

    return df, df_audit

# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState))
# df, df_audit = legalRepDetails(silver_m1)

# display(df)
# display(df_audit)
# display(silver_m1)

In [0]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, year, when, lit

# def test_is_appellant_minor():
#     # spark = SparkSession.builder.master("local[1]").appName("MinorCheckTest").getOrCreate()

#     # Sample input data
#     data = [
#         ("2007-01-01", "2025-07-01"),  # 18 years old – NOT minor
#         ("2010-01-01", "2025-07-01"),  # 15 years old – minor
#         ("2000-01-01", "2025-07-01"),  # 25 years old – NOT minor
#         (None, "2025-07-01"),          # Missing birthdate – should return None
#         ("2010-01-01", None),          # Missing DateLodged – should return None
#     ]

#     df = spark.createDataFrame(data, ["BirthDate", "DateLodged"])

#     # Cast to date
#     df = df.withColumn("BirthDate", col("BirthDate").cast("date"))
#     df = df.withColumn("DateLodged", col("DateLodged").cast("date"))

#     # Mock conditions (e.g., always true for test)
#     conditions = col("BirthDate").isNotNull() & col("DateLodged").isNotNull()

#     # Apply isAppellantMinor logic
#     df = df.withColumn("isAppellantMinor", when(
#         conditions & (year(col("BirthDate")) > (year(col("DateLodged")) - 18)),
#         lit("Yes")
#     ).when(conditions, lit("No")).otherwise(None))

#     # Show the result
#     df.show(truncate=False)

#     # spark.stop()


In [0]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import pandas as pd

def getCountryApp(country, ukPostcodeAppellant, appellantFullAddress, Appellant_Postcode):
    countryFromAddress = []
    try:
        country = country
        uk_postcode = ukPostcodeAppellant
        full_address = appellantFullAddress

        if (country is not None and country != 0) or (country is not None and pd.notna(country)):
            countryFromAddress.append(str(country))
            return ', '.join(countryFromAddress)
        else:
            if uk_postcode is True or uk_postcode == "True":
                countryFromAddress.append('GB')
                return ', '.join(countryFromAddress)
            if uk_postcode is False or uk_postcode == "False":
                searched_country = getCountryFromAddress(full_address)
                countryFromAddress.append(searched_country)
                return ', '.join(countryFromAddress)
        return ', '.join(countryFromAddress)
    except Exception:
        return None

getCountryApp_udf = udf(getCountryApp, StringType())

# df = spark.table("ariadm_active_appeals.silver_caseapplicant_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()

# df = df.withColumn(
#     "appellantFullAddress",
#     makeFullAddressUDF(
#         col("Appellant_Address1"),
#         col("Appellant_Address2"),
#         col("Appellant_Address3"),
#         col("Appellant_Address4"),
#         col("Appellant_Address5"),
#         col("Appellant_Postcode")
#     )
# ).withColumn(
#     "ukPostcodeAppellant",
#     getUkPostcodeUDF(col("Appellant_Postcode"))
# ).withColumn(
#     "countryGovUkOocAdminJ",
#     getCountryApp_udf(
#         col("lu_countryGovUkOocAdminJ").alias("country"),
#         col("ukPostcodeAppellant"),
#         col("appellantFullAddress"),
#         col("Appellant_Postcode")
#     )
# ).select("CaseNo", col("countryGovUkOocAdminJ"),col("appellantFullAddress"),col("ukPostcodeAppellant"))

# bronze_countries_countryFromAddress = spark.table("ariadm_active_appeals.bronze_countries_countryFromAddress").withColumn("lu_cfa_countryGovUkOocAdminJ",col("countryGovUkOocAdminJ")).withColumn("lu_cfa_contryFromAddress", col("countryFromAddress"))

# df = df.alias('main').join(bronze_countries_countryFromAddress.alias('cfa'), col("main.countryGovUkOocAdminJ") == col("cfa.lu_cfa_contryFromAddress"), "left").select("main.CaseNo", "appellantFullAddress", "ukPostcodeAppellant", col("cfa.lu_cfa_contryFromAddress"), col("cfa.lu_cfa_countryGovUkOocAdminJ"), col("main.countryGovUkOocAdminJ").alias("countryGovUkOocAdminJ"))

# # display(df)

# df.select("CaseNo", "appellantFullAddress", "ukPostcodeAppellant", 
#           when( col("lu_cfa_contryFromAddress").isNotNull(),col("lu_cfa_countryGovUkOocAdminJ"))
#           .otherwise(col("countryGovUkOocAdminJ")).alias("countryGovUkOocAdminJ")
#           ).display()

# AppealType grouping
def appellantDetails(silver_m1, silver_m2, silver_c,bronze_countryFromAddress,bronze_HORef_cleansing):
    conditions = (col("dv_representation").isin('LR', 'AIP')) & (col("lu_appealType").isNotNull())

    # Create DataFrame with CaseNo and list of CategoryId
    silver_c_grouped = silver_c.groupBy("CaseNo").agg(collect_list(col("CategoryId")).alias("CategoryIdList"))

    # Create DataFrame with CaseNo and list of lu_countryCode from silver_m1
    silver_m1_country_grouped = silver_m1.groupBy("CaseNo").agg(collect_list(col("lu_countryCode")).alias("lu_countryCodeList"))

    # isAppellantMinor: BirthDate > (DateLodged - 18 years) using year subtraction
    is_minor_expr = when(
        conditions & ((datediff(col("DateLodged"), col("BirthDate")) / 365.25) < 18),
        lit("Yes")
    ).when(conditions, lit("No")).otherwise(None)

    # DeportationOrderOptions logic
    deportation_category_ids = [17, 19, 29, 30, 31, 32, 41, 48]
    deportation_condition = (
        (col("CategoryIdList").isNotNull() & (expr(f"size(array_intersect(CategoryIdList, array({','.join(map(str, deportation_category_ids))})) )") > 0)) |
        (col("dv_CCDAppealType") == "DA") |
        (col("DeportationDate").isNotNull()) |
        (col("RemovalDate").isNotNull())
    )

    # appellantInUk logic
    appellant_in_uk_expr = when(
        conditions & (expr("array_contains(CategoryIdList, 37)")), lit("Yes")
    ).when(
        conditions & (expr("array_contains(CategoryIdList, 38)")), lit("No")
    ).otherwise(None)

    # appealOutOfCountry logic
    appeal_out_of_country_expr = when(
        conditions & (expr("array_contains(CategoryIdList, 38)")), lit("Yes")
    ).otherwise(None)

    # appellantStateless logic
    appellant_stateless_expr = when(
        conditions & (col("AppellantCountryId") == 211), lit("isStateless")
    ).when(
        conditions, lit("hasNationality")
    ).otherwise(None)
    
    # appellantHasFixedAddress logic
    appellant_has_fixed_address_expr = when(
        conditions & expr("array_contains(CategoryIdList, 37)"), lit("Yes")
    ).otherwise(None)

    # appellantHasFixedAddressAdminJ logic: Only include if CategoryIdList contains 38
    appellant_has_fixed_address_adminj_expr = when(conditions &
        expr("array_contains(CategoryIdList, 38)"), lit("Yes")
    ).otherwise(None) 

    # appellantAddress logic
    # Only include if CategoryIdList contains 37 and conditions
    include_appellant_address = (conditions & expr("array_contains(CategoryIdList, 37)") & 
                                    (coalesce(col("Appellant_Address1"), col("Appellant_Address2"), col("Appellant_Address3"), col("Appellant_Address4"), col("Appellant_Address5"), col("Appellant_Postcode")).isNotNull()))
    appellant_address_struct = when(
        include_appellant_address,
        struct(
            # AddressLine1 is mandatory in CCD, fallback logic
            coalesce(
                col("Appellant_Address1"),
                col("Appellant_Address2"),
                col("Appellant_Address3"),
                col("Appellant_Address4"),
                col("Appellant_Address5")
            ).alias("AddressLine1"),
            col("Appellant_Address2").alias("AddressLine2"),
            lit("").alias("AddressLine3"),
            col("Appellant_Address3").alias("PostTown"),
            col("Appellant_Address4").alias("County"),
            col("Appellant_Address5").alias("Country"),
            col("Appellant_Postcode").alias("PostCode")
        )
    ).otherwise(None)

    # addressLine1AdminJ: mandatory for OOC, fallback through address fields
    address_line1_adminj_expr = when(
        conditions & expr("array_contains(CategoryIdList, 38)"),
        coalesce(
            col("Appellant_Address1"),
            col("Appellant_Address2"),
            col("Appellant_Address3"),
            col("Appellant_Address4"),
            col("Appellant_Address5"),
            col("Appellant_Postcode")
        )
    ).otherwise(None)

    # addressLine2AdminJ: mandatory for OOC, fallback through address fields (skip the one used for addressLine1AdminJ)
    address_line2_adminj_expr = when(
        conditions & expr("array_contains(CategoryIdList, 38)"),
        coalesce(
            # Exclude the value used for addressLine1AdminJ
            when(col("Appellant_Address1").isNull(), None).otherwise(
                coalesce(
                    when(col("Appellant_Address2") != col("Appellant_Address1"), col("Appellant_Address2")),
                    when(col("Appellant_Address3") != col("Appellant_Address1"), col("Appellant_Address3")),
                    when(col("Appellant_Address4") != col("Appellant_Address1"), col("Appellant_Address4")),
                    when(col("Appellant_Address5") != col("Appellant_Address1"), col("Appellant_Address5")),
                    when(col("Appellant_Postcode") != col("Appellant_Address1"), col("Appellant_Postcode"))
                )
            ),
            when(col("Appellant_Address2").isNull() & col("Appellant_Address1").isNull(), None).otherwise(
                coalesce(
                    when(col("Appellant_Address3") != col("Appellant_Address2"), col("Appellant_Address3")),
                    when(col("Appellant_Address4") != col("Appellant_Address2"), col("Appellant_Address4")),
                    when(col("Appellant_Address5") != col("Appellant_Address2"), col("Appellant_Address5")),
                    when(col("Appellant_Postcode") != col("Appellant_Address2"), col("Appellant_Postcode"))
                )
            ),
            when(col("Appellant_Address3").isNull() & col("Appellant_Address2").isNull() & col("Appellant_Address1").isNull(), None).otherwise(
                coalesce(
                    when(col("Appellant_Address4") != col("Appellant_Address3"), col("Appellant_Address4")),
                    when(col("Appellant_Address5") != col("Appellant_Address3"), col("Appellant_Address5")),
                    when(col("Appellant_Postcode") != col("Appellant_Address3"), col("Appellant_Postcode"))
                )
            ),
            when(col("Appellant_Address4").isNull() & col("Appellant_Address3").isNull() & col("Appellant_Address2").isNull() & col("Appellant_Address1").isNull(), None).otherwise(
                coalesce(
                    when(col("Appellant_Address5") != col("Appellant_Address4"), col("Appellant_Address5")),
                    when(col("Appellant_Postcode") != col("Appellant_Address4"), col("Appellant_Postcode"))
                )
            ),
            when(col("Appellant_Address5").isNull() & col("Appellant_Address4").isNull() & col("Appellant_Address3").isNull() & col("Appellant_Address2").isNull() & col("Appellant_Address1").isNull(), None).otherwise(
                coalesce(
                    when(col("Appellant_Postcode") != col("Appellant_Address5"), col("Appellant_Postcode"))
                )
            )
        )
    ).otherwise(None)

    address_line3_adminj_expr = when(
    conditions & expr("array_contains(CategoryIdList, 38)") & (col("Appellant_Address3").isNotNull() | col("Appellant_Address4").isNotNull()),
    concat_ws(
        ", ",
        col("Appellant_Address3"), col("Appellant_Address4")
    )).otherwise(None)

    # addressLine4AdminJ logic
    # IF CategoryId IN [38] = Include; ELSE OMIT
    # AppellantAddress5 + ', ' + AppellantPostcode
    address_line4_adminj_expr = when(
        conditions & expr("array_contains(CategoryIdList, 38)") & (col("Appellant_Address5").isNotNull() | col("Appellant_Postcode").isNotNull()),
        concat_ws(",",
            col("Appellant_Address5"), col("Appellant_Postcode")
        )
    ).otherwise(None)

    # IF CategoryId IN [38] = Include; ELSE OMIT;
    # IF CleansedHORef LIKE '%GWF%' = 'entryClearanceDecision';
    # IF M1.HORef LIKE '%GWF%' = 'entryClearanceDecision';
    # IF M2.FCONumber LIKE '%GWF%' = 'entryClearanceDecision';
    # ELSE 'none'
    
    # Join bronze_HORef_cleansing to get CleansedHORef using CaseNo and coalesce(HORef, FCONumber)
    bronze_cleansing = bronze_HORef_cleansing.select(
        col("CaseNo"),
        coalesce(col("HORef"), col("FCONumber")).alias("lu_HORef")
    )

    # oocAppealAdminJ logic
    ooc_appeal_adminj_expr = when(
        conditions & expr("array_contains(CategoryIdList, 38)"),
        when(
              (col("lu_HORef").like("%GWF%")) |
              (col("HORef").like("%GWF%")) |
              (col("FCONumber").like("%GWF%"))
        
        , lit("entryClearanceDecision")).otherwise(lit("none"))
    ).otherwise(None)

    # countryGovUkOocAdminJ logic
    # IF CategoryId IN [38] = Include; ELSE OMIT
    # If AppellantCountryId is present and not 0, use as string
    # If ukPostcodeAppellant is True, use 'United Kingdom'
    # If ukPostcodeAppellant is False, use appellantFullAddress (not implemented, fallback to empty string)
    # Final fallback: empty string
    # country_gov_uk_ooc_adminj_expr = when(
    #     conditions & expr("array_contains(CategoryIdList, 38)"),
    #     getCountryApp_udf(
    #         col("AppellantCountryId"),
    #         getUkPostcodeUDF(col("Appellant_Postcode")),
    #         makeFullAddressUDF(
    #             col("Appellant_Address1"),
    #             col("Appellant_Address2"),
    #             col("Appellant_Address3"),
    #             col("Appellant_Address4"),
    #             col("Appellant_Address5"),
    #             col("Appellant_Postcode")
    #         ),
    #         col("Appellant_Postcode")
    #     )
    # ).otherwise(None)

    silver_m2_derived = silver_m2.withColumn(
                                        "appellantFullAddress",
                                        concat_ws(",",
                                            col("Appellant_Address1"),
                                            col("Appellant_Address2"),
                                            col("Appellant_Address3"),
                                            col("Appellant_Address4"),
                                            col("Appellant_Address5"),
                                            col("Appellant_Postcode")
                                        )
                                    ).withColumn(
                                        "ukPostcodeAppellant",
                                        getUkPostcodeUDF(col("Appellant_Postcode"))
                                    ).withColumn(
                                        "dv_countryGovUkOocAdminJ",
                                        getCountryApp_udf(
                                            col("lu_countryGovUkOocAdminJ").alias("country"),
                                            col("ukPostcodeAppellant"),
                                            col("appellantFullAddress"),
                                            col("Appellant_Postcode")
                                        )
                                    )


    bronze_countries_countryFromAddress = bronze_countryFromAddress.withColumn("lu_cfa_countryGovUkOocAdminJ",col("countryGovUkOocAdminJ")).withColumn("lu_cfa_contryFromAddress", col("countryFromAddress"))

    silver_m2_derived = silver_m2_derived.alias('main').join(bronze_countries_countryFromAddress.alias('cfa'), col("main.dv_countryGovUkOocAdminJ") == col("cfa.lu_cfa_contryFromAddress"), "left").select("main.*",  when( col("lu_cfa_contryFromAddress").isNotNull(),col("lu_cfa_countryGovUkOocAdminJ"))
          .otherwise(col("dv_countryGovUkOocAdminJ")).alias("countryGovUkOocAdminJ"))
    
    country_gov_uk_ooc_adminj_expr = when(
        conditions & expr("array_contains(CategoryIdList, 38)"),
        col("countryGovUkOocAdminJ")
    ).otherwise(None)
          

    # # display(df)

    # df.select("CaseNo", "appellantFullAddress", "ukPostcodeAppellant", 
    #       when( col("lu_cfa_contryFromAddress").isNotNull(),col("lu_cfa_countryGovUkOocAdminJ"))
    #       .otherwise(col("countryGovUkOocAdminJ")).alias("countryGovUkOocAdminJ")
    #       ).display()

    # internalAppellantEmail logic
    internal_appellant_email_expr = when(
        conditions & col("Appellant_Email").isNotNull(), cleanEmailUDF(col("Appellant_Email"))
    ).otherwise(None)

    # internalAppellantMobileNumber logic
    internal_appellant_mobile_number_expr = when(
        conditions & col("Appellant_Telephone").isNotNull(), phoneNumberUDF(col("Appellant_Telephone"))
    ).otherwise(None)

    # mobileNumber logic (same as internalAppellantMobileNumber)
    mobile_number_expr = internal_appellant_mobile_number_expr

    # email logic (same as internalAppellantEmail)
    email_expr = internal_appellant_email_expr

    df = silver_m1.alias("silver_m1") \
        .join(silver_m2_derived, ["CaseNo"], "left") \
        .join(silver_c_grouped, ["CaseNo"], "left") \
        .join(silver_m1_country_grouped, ["CaseNo"], "left") \
        .join(bronze_cleansing, ["CaseNo"], "left") \
        .select(
            col("CaseNo"),
            when(
                conditions,
                col("Appellant_Name")
            ).otherwise(None).alias("appellantFamilyName"),
            when(
                conditions,
                col("Appellant_Forenames")
            ).otherwise(None).alias("appellantGivenNames"),
            when(conditions, 
                 concat(col("Appellant_Forenames"), lit(" "), col("Appellant_Name"))
            ).otherwise(None).alias("appellantNameForDisplay"),
            when(conditions,
                 date_format(col("BirthDate"),"yyyy-MM-dd")
            ).otherwise(None).alias("appellantDateOfBirth"),
            is_minor_expr.alias("isAppellantMinor"),
            when(conditions,
                 concat(col("Appellant_Forenames"), lit(" "), col("Appellant_Name"))
            ).otherwise(None).alias("caseNameHmctsInternal"),
            when(conditions,
                 concat(col("Appellant_Forenames"), lit(" "), col("Appellant_Name"))
            ).otherwise(None).alias("hmctsCaseNameInternal"),
            internal_appellant_email_expr.alias("internalAppellantEmail"),
            email_expr.alias("email"),
            internal_appellant_mobile_number_expr.alias("internalAppellantMobileNumber"),
            mobile_number_expr.alias("mobileNumber"),
            appellant_in_uk_expr.alias("appellantInUk"),
            appeal_out_of_country_expr.alias("appealOutOfCountry"),
            ooc_appeal_adminj_expr.alias("oocAppealAdminJ"),
            appellant_has_fixed_address_expr.alias("appellantHasFixedAddress"),
            appellant_has_fixed_address_adminj_expr.alias("appellantHasFixedAddressAdminJ"),
            appellant_address_struct.alias("appellantAddress"),
            address_line1_adminj_expr.alias("addressLine1AdminJ"),
            address_line2_adminj_expr.alias("addressLine2AdminJ"),
            address_line3_adminj_expr.alias("addressLine3AdminJ"),
            address_line4_adminj_expr.alias("addressLine4AdminJ"),
            country_gov_uk_ooc_adminj_expr.alias("countryGovUkOocAdminJ"),
            appellant_stateless_expr.alias("appellantStateless"),
            when(conditions, col("lu_countryCodeList")).otherwise(None).alias("appellantNationalities"),
            when(conditions, col("lu_appellantNationalitiesDescription")).otherwise(None).alias("appellantNationalitiesDescription"),
            when(conditions & deportation_condition ,
                lit("Yes")
            ).when(conditions, lit("No")).otherwise(None).alias("deportationOrderOptions"),
            col("CategoryIdList")
            # internalAppellantEmail: IF AppellantEmail IS NULL = OMIT; ELSE Data from ARIA
            # internalAppellantMobileNumber: IF AppellantTelephone IS NULL = OMIT; ELSE Data from ARIA
            
            # mobileNumber: same as internalAppellantMobileNumber, with conditions
            # email: same as internalAppellantEmail, with conditions
        ).distinct()

    # new_order = ["appellantFamilyName", "appellantGivenNames","appellantNameForDisplay", "appellantDateOfBirth", "isAppellantMinor", "caseNameHmctsInternal","hmctsCaseNameInternal", "internalAppellantEmail", "email", "internalAppellantMobileNumber", "mobileNumber", "appellantInUk", "appealOutOfCountry", "oocAppealAdminJ", "appellantHasFixedAddress", "appellantHasFixedAddressAdminJ", "appellantAddress", "addressLine1AdminJ", "addressLine2AdminJ", "addressLine3AdminJ", "addressLine4AdminJ", "countryGovUkOocAdminJ", "appellantStateless", "appellantNationalities", "appellantNationalitiesDescription", "deportationOrderOptions"]

    # df = df.select(*new_order)

    common_inputFields = [lit("dv_representation"), lit("lu_appealType")]
    common_inputValues = [col("audit.dv_representation"), col("audit.lu_appealType")]

    df_audit = silver_m1.join(silver_m2_derived, ["CaseNo"], "left") \
        .alias("audit") \
        .join(silver_c_grouped, ["CaseNo"], "left") \
        .join(silver_m1_country_grouped, ["CaseNo"], "left") \
        .join(df.alias("content"), ["CaseNo"],"left") \
        .join(bronze_cleansing.alias("content_c"), ["CaseNo"],"left") \
        .select(
            col("CaseNo"),

            #Audit appellantFamilyName
            array(struct(*common_inputFields, lit("Appellant_Name"))).alias("appellantFamilyName_inputFields"),
            array(struct(*common_inputValues, col("Appellant_Name"))).alias("appellantFamilyName_inputValues"),
            col("content.appellantFamilyName"),
            lit("no").alias("appellantFamilyName_Transformation"),

            #Audit appellantGivenNames
            array(struct(*common_inputFields, lit("Appellant_Forenames"))).alias("appellantGivenNames_inputfields"),
            array(struct(*common_inputValues, col("Appellant_Forenames"))).alias("appellantGivenNames_inputvalues"),
            col("content.appellantGivenNames"),
            lit("no").alias("appellantGivenNames_Transformation"),

            #Audit appellantNameForDisplay
            array(struct(*common_inputFields,lit("Appellant_Name"), lit("Appellant_Forenames"))).alias("appellantNameForDisplay_inputfields"),
            array(struct(*common_inputValues,col("Appellant_Name"), col("Appellant_Forenames"))).alias("appellantNameForDisplay_inputvalues"),
            col("content.appellantNameForDisplay"),
            lit("no").alias("appellantNameForDisplay_Transformation"),

            #Audit appellantDateOfBirth
            array(struct(*common_inputFields,lit("BirthDate"))).alias("appellantDateOfBirth_inputfields"),
            array(struct(*common_inputValues,col("BirthDate"))).alias("appellantDateOfBirth_inputvalues"),
            col("content.appellantDateOfBirth"),
            lit("no").alias("appellantDateOfBirth_Transformation"),

            #Audit caseNameHmctsInternal
            array(struct(*common_inputFields,lit("Appellant_Name"), lit("Appellant_Forenames"))).alias("caseNameHmctsInternal_inputfields"),
            array(struct(*common_inputValues,col("Appellant_Name"), col("Appellant_Forenames"))).alias("caseNameHmctsInternal_inputvalues"),
            col("content.caseNameHmctsInternal"),
            lit("no").alias("caseNameHmctsInternal_Transformation"),

            #Audit hmctsCaseNameInternal
            array(struct(*common_inputFields,lit("Appellant_Name"), lit("Appellant_Forenames"))).alias("hmctsCaseNameInternal_inputfields"),
            array(struct(*common_inputValues,col("Appellant_Name"), col("Appellant_Forenames"))).alias("hmctsCaseNameInternal_inputvalues"),
            col("content.hmctsCaseNameInternal"),
            lit("no").alias("hmctsCaseNameInternal_Transformation"),

            # Audit deportationOrderOptions
            array(struct(*common_inputFields, lit("CategoryId"), lit("dv_CCDAppealType"), lit("DeportationDate"), lit("RemovalDate"))).alias("deportationOrderOptions_inputfields"),
            array(struct(*common_inputValues, col("content.CategoryIdList"), col("dv_CCDAppealType"), col("DeportationDate"), col("RemovalDate"))).alias("deportationOrderOptions_inputvalues"),
            col("content.deportationOrderOptions"),
            lit("yes").alias("deportationOrderOptions_Transformation"),

            # Audit appellantInUk
            array(struct(*common_inputFields, lit("CategoryId"))).alias("appellantInUk_inputfields"),
            array(struct(*common_inputValues, col("content.CategoryIdList"))).alias("appellantInUk_inputvalues"),
            col("content.appellantInUk"),
            lit("yes").alias("appellantInUk_Transformation"),

            # Audit appealOutOfCountry
            array(struct(*common_inputFields, lit("CategoryId"))).alias("appealOutOfCountry_inputfields"),
            array(struct(*common_inputValues, col("content.CategoryIdList"))).alias("appealOutOfCountry_inputvalues"),
            col("content.appealOutOfCountry"),
            lit("yes").alias("appealOutOfCountry_Transformation"),

            # Audit isAppellantMinor
            array(struct(*common_inputFields, lit("BirthDate"), lit("DateLodged"))).alias("isAppellantMinor_inputfields"),
            array(struct(*common_inputValues, col("BirthDate"), col("DateLodged"))).alias("isAppellantMinor_inputvalues"),
            col("content.isAppellantMinor"),
            lit("yes").alias("isAppellantMinor_Transformation"),

            # Audit appellantStateless
            array(struct(*common_inputFields, lit("AppellantCountryId"))).alias("appellantStateless_inputfields"),
            array(struct(*common_inputValues, col("AppellantCountryId"))).alias("appellantStateless_inputvalues"),
            col("content.appellantStateless"),
            lit("yes").alias("appellantStateless_Transformation"),

            # Audit appellantNationalities
            array(struct(*common_inputFields, lit("lu_countryCodeList"))).alias("appellantNationalities_inputfields"),
            array(struct(*common_inputValues, col("lu_countryCodeList"))).alias("appellantNationalities_inputvalues"),
            col("content.appellantNationalities"),
            lit("yes").alias("appellantNationalities_Transformation"),

            # Audit appellantNationalitiesDescription
            array(struct(*common_inputFields, lit("lu_appellantNationalitiesDescription"))).alias("lu_appellantNationalitiesDescription_inputfields"),
            array(struct(*common_inputValues, col("appellantNationalitiesDescription"))).alias("appellantNationalitiesDescription_inputvalues"),
            col("content.appellantNationalitiesDescription"),
            lit("yes").alias("appellantNationalitiesDescription_Transformation"),

            # Audit appellantHasFixedAddress
            array(struct(*common_inputFields, lit("content.CategoryIdList"))).alias("appellantHasFixedAddress_inputfields"),
            array(struct(*common_inputValues, col("content.CategoryIdList"))).alias("appellantHasFixedAddress_inputvalues"),
            col("content.appellantHasFixedAddress"),
            lit("yes").alias("appellantHasFixedAddress_Transformation"),

            # Audit appellantAddress
            array(struct(*common_inputFields, lit("Appellant_Address1"), lit("Appellant_Address2"), lit("Appellant_Address3"), lit("Appellant_Address4"), lit("Appellant_Address5"), lit("Appellant_Postcode"), lit("content.CategoryIdList"))).alias("appellantAddress_inputfields"),
            array(struct(*common_inputValues, col("Appellant_Address1"), col("Appellant_Address2"), col("Appellant_Address3"), col("Appellant_Address4"), col("Appellant_Address5"), col("Appellant_Postcode"), col("content.CategoryIdList"))).alias("appellantAddress_inputvalues"),
            col("content.appellantAddress"),
            lit("yes").alias("appellantAddress_Transformation"),



            # Audit oocAppealAdminJ
            array(struct(*common_inputFields, lit("content.CategoryIdList"), lit("HORef"),lit("FCONumber"),lit("lu_HORef"))).alias("oocAppealAdminJ_inputfields"),
            array(struct(*common_inputValues, col("content.CategoryIdList"), col("HORef"),col("FCONumber"),col("content_c.lu_HORef"))).alias("oocAppealAdminJ_inputvalues"),
            col("content.oocAppealAdminJ"),
            lit("yes").alias("oocAppealAdminJ_Transformation"),

            # Audit appellantHasFixedAddressAdminJ
            array(struct(*common_inputFields,lit("content.CategoryIdList"))).alias("appellantHasFixedAddressAdminJ_inputfields"),
            array(struct(*common_inputValues, col("content.CategoryIdList"))).alias("appellantHasFixedAddressAdminJ_inputvalues"),
            col("content.appellantHasFixedAddressAdminJ"),
            lit("yes").alias("appellantHasFixedAddressAdminJ_Transformation"),

            # Audit addressLine1AdminJ
            array(struct(*common_inputFields, lit("Appellant_Address1"),lit("Appellant_Address2"), lit("Appellant_Address3"), lit("Appellant_Address4"), lit("Appellant_Address5"), lit("Appellant_Postcode"), lit("content.CategoryIdList"))).alias("addressLine1AdminJ_inputfields"),
            array(struct(*common_inputValues, col("Appellant_Address1"),col("Appellant_Address2"), col("Appellant_Address3"), col("Appellant_Address4"), col("Appellant_Address5"), col("Appellant_Postcode"), col("content.CategoryIdList"))).alias("addressLine1AdminJ_inputvalues"),
            col("content.addressLine1AdminJ"),
            lit("yes").alias("addressLine1AdminJ_Transformation"),

            # Audit addressLine2AdminJ
            array(struct(*common_inputFields, lit("Appellant_Address2"), lit("Appellant_Address3"), lit("Appellant_Address4"), lit("Appellant_Address5"), lit("Appellant_Postcode"), lit("content.CategoryIdList"))).alias("addressLine2AdminJ_inputfields"),
            array(struct(*common_inputValues, col("Appellant_Address2"), col("Appellant_Address3"), col("Appellant_Address4"), col("Appellant_Address5"), col("Appellant_Postcode"), col("content.CategoryIdList"))).alias("addressLine2AdminJ_inputvalues"),
            col("content.addressLine2AdminJ"),
            lit("yes").alias("addressLine2AdminJ_Transformation"),

            # Audit addressLine3AdminJ
            array(struct(*common_inputFields, lit("Appellant_Address3"), lit("Appellant_Address4"), lit("content.CategoryIdList"))).alias("addressLine3AdminJ_inputfields"),
            array(struct(*common_inputValues, col("Appellant_Address3"), col("Appellant_Address4"), col("content.CategoryIdList"))).alias("addressLine3AdminJ_inputvalues"),
            col("content.addressLine3AdminJ"),
            lit("yes").alias("addressLine3AdminJ_Transformation"),

            # Audit addressLine4AdminJ
            array(struct(*common_inputFields, lit("Appellant_Address5"), lit("Appellant_Postcode"), lit("content.CategoryIdList"))).alias("addressLine4AdminJ_inputfields"),
            array(struct(*common_inputValues, col("Appellant_Address5"), col("Appellant_Postcode"), col("content.CategoryIdList"))).alias("addressLine4AdminJ_inputvalues"),
            col("content.addressLine4AdminJ"),
            lit("yes").alias("addressLine4AdminJ_Transformation"),

            # Audit countryGovUkOocAdminJ
            array(struct(*common_inputFields, lit("AppellantCountryId"), lit("ukPostcodeAppellant"), lit("appellantFullAddress"), lit("content.CategoryIdList"))).alias("countryGovUkOocAdminJ_inputfields"),
            array(struct(*common_inputValues, col("AppellantCountryId"), col("ukPostcodeAppellant"), col("appellantFullAddress"), col("content.CategoryIdList"))).alias("countryGovUkOocAdminJ_inputvalues"),
            col("content.countryGovUkOocAdminJ"),
            lit("yes").alias("countryGovUkOocAdminJ_Transformation"),

             # Audit internalAppellantEmail
            array(struct(*common_inputFields, lit("Appellant_Email"))).alias("internalAppellantEmail_inputfields"),
            array(struct(*common_inputValues, col("Appellant_Email"))).alias("internalAppellantEmail_inputvalues"),
            col("content.internalAppellantEmail"),
            lit("yes").alias("internalAppellantEmail_Transformation"),

            # Audit internalAppellantMobileNumber
            array(struct(*common_inputFields, lit("Appellant_Telephone"))).alias("internalAppellantMobileNumber_inputfields"),
            array(struct(*common_inputValues, col("Appellant_Telephone"))).alias("internalAppellantMobileNumber_inputvalues"),
            col("content.internalAppellantMobileNumber"),
            lit("yes").alias("internalAppellantMobileNumber_Transformation"),

            # Audit mobileNumber (same as internalAppellantMobileNumber)
            array(struct(*common_inputFields, lit("Appellant_Telephone"))).alias("mobileNumber_inputfields"),
            array(struct(*common_inputValues, col("Appellant_Telephone"))).alias("mobileNumber_inputvalues"),
            col("content.mobileNumber"),
            lit("yes").alias("mobileNumber_Transformation"),

            # Audit email (same as internalAppellantEmail)
            array(struct(*common_inputFields, lit("Appellant_Email"))).alias("email_inputfields"),
            array(struct(*common_inputValues, col("Appellant_Email"))).alias("email_inputvalues"),
            col("content.email"),
            lit("yes").alias("email_Transformation"),
        ).distinct()
    
    return df, df_audit

# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()
# silver_m2 =  spark.table("ariadm_active_appeals.silver_caseapplicant_detail").filter(col("dv_targetState") == lit(AppealState))   
# silver_c = spark.read.table("ariadm_active_appeals.silver_appealcategory_detail").filter(col("dv_targetState") == lit(AppealState))
# bronze_countryFromAddress = spark.table("ariadm_active_appeals.bronze_countries_countryFromAddress").withColumn("lu_countryGovUkOocAdminJ",col("countryGovUkOocAdminJ"))

# bronze_HORef_cleansing = spark.table("ariadm_active_appeals.bronze_HORef_cleansing")

# df, df_audit = appellantDetails(silver_m1,silver_m2,silver_c,bronze_countryFromAddress,bronze_HORef_cleansing)
# display(df)
# display(df_audit)
# df.printSchema()

In [0]:
from pyspark.sql.functions import col, when, lit, array, struct, collect_list, expr, first, date_format, coalesce
import re
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

# Function to clean reference numbers (HORef, FCONumber) according to business rules
def cleanReferenceNumber(ref):
    """
    Cleans a reference number string according to the following rules:
    - If ref is None, return None.
    - If ref starts with 'GWF', return as is (no cleaning).
    - Remove all spaces and special characters except '/'.
    - If ref contains '/', split into prefix and suffix:
        - If prefix contains no digits, ignore it.
        - Extract digits from prefix and suffix.
        - If suffix has <=3 digits, remove leading zeros.
        - Concatenate prefix digits and suffix digits.
    - If ref does not contain '/', extract all digits.
    - If the cleaned result is 8 digits, pad with leading zero to make 9 digits.
    - If the cleaned result is not 9 digits, return None.
    - Only return if cleaned result is exactly 9 digits.
    """
    if ref is None:
        return None
    ref = ref.strip()
    if ref.startswith("GWF"):
        return ref
    # Remove all spaces and special chars except /
    ref = re.sub(r"[^A-Za-z0-9/]", "", ref)
    # Split on /
    parts = ref.split("/")
    if len(parts) == 2:
        prefix, suffix = parts
        # If prefix is not all digits, ignore it (for ISLAMABAD/123456789, NEWDELHI/12345678, etc.)
        if not re.search(r"\d", prefix):
            prefix_digits = ""
        else:
            prefix_digits = re.sub(r"\D", "", prefix)
            if not prefix_digits and prefix:
                prefix_digits = prefix
        suffix_digits = re.sub(r"\D", "", suffix)
        if not suffix_digits:
            return None
        # Remove leading zeros for suffix if <=3 digits
        if len(suffix_digits) <= 3:
            try:
                suffix_digits = str(int(suffix_digits))
            except:
                return None
        cleaned = prefix_digits + suffix_digits
    elif len(parts) == 1:
        digits = re.sub(r"\D", "", ref)
        cleaned = digits
    else:
        return None
    # If cleaned is 8 digits, pad to 9
    if len(cleaned) == 8:
        cleaned = "0" + cleaned
    elif len(cleaned) != 9:
        return None
    if re.fullmatch(r"\d{9}", cleaned):
        return cleaned
    return None

# Register the cleaning function as a Spark UDF
cleanReferenceNumberUDF = udf(cleanReferenceNumber, StringType())

# # Example usage: Clean HORef in silver_m1 and display results where HORef is not null and length > 9
# silver_m1.withColumn("CleanedHORef", cleanReferenceNumberUDF(col("HORef"))).filter(col("HORef").isNotNull() & (F.length(col("HORef")) > 9)).select("HORef", "CleanedHORef").display()

# # Example usage: Clean FCONumber in silver_m2 and display results where FCONumber is not null and length > 9
# silver_m2.withColumn("CleanedFCONumber", cleanReferenceNumberUDF(col("FCONumber"))).filter(col("FCONumber").isNotNull() & (F.length(col("FCONumber")) > 9)).select("FCONumber", "CleanedFCONumber").display()

# # after len(HORef) > 9 tHEN NULL
# # IF starts with GWF the no cleaning eg GWF064069891
# # remove charters and special char and if after / we like 001 its 1 and 022 its 22 and if its till = 9 the no more clening and if 8 then add leading 0 else null

# # eg - A1222233/3(as 012222333) A14222233/3 (as 142222333 ) T1175081/2 (as 011750812)
# # eg - ISLAMABAD/123456789 as (123456789) NEW DELHI/12345678 (as 012345678 )
# # M1699430/002 (as 016994302 )
# # B1985113/2 (as 019851132 )
# # M0104326/010 (as 010432610 )
# # F3009964/2 (M0104326/010)
# # UKLPA/146205 (INVALID make it null)
# # TN2/5197349 ( AS 025197349 )
# # SHEFF1/4732574 (014732574)
# # B1977303/002 (019773032)

# bronze_HORef_cleansing = spark.table("ariadm_active_appeals.bronze_HORef_cleansing")

# # Join bronze_HORef_cleansing to get CleansedHORef using CaseNo and coalesce(HORef, FCONumber)
# bronze_cleansing = bronze_HORef_cleansing.select(
#     col("CaseNo"),
#     coalesce(col("HORef"), col("FCONumber")).alias("CleansedHORef"),
#     col("HORef"),
#     col("FCONumber"),
#     cleanReferenceNumberUDF(col("HORef")).alias("CleanedHORef"),
#     cleanReferenceNumberUDF(col("FCONumber")).alias("CleanedFCONumber")
# )

# display(bronze_cleansing.filter((~col("HORef").contains("GWF")) & (col("HORef").isNotNull()) & (F.length(col("HORef")) > 9)).select("HORef", "CleanedHORef"))
# display(bronze_cleansing.filter((~col("FCONumber").contains("GWF")) & (col("FCONumber").isNotNull()) & (F.length(col("FCONumber")) > 9)).select("FCONumber", "CleanedFCONumber"))


def homeOfficeDetails(silver_m1, silver_m2, silver_c, bronze_HORef_cleansing):
    conditions = (
        col("dv_representation").isin('LR', 'AIP')
    ) & (
        col("lu_appealType").isNotNull()
    )

    # Create DataFrame with CaseNo and list of CategoryId
    silver_c_grouped = silver_c.groupBy("CaseNo").agg(
        collect_list(col("CategoryId")).alias("CategoryIdList"),
        first("CategoryId", ignorenulls=True).alias("CategoryId")
    )

    # Join bronze_HORef_cleansing to get CleansedHORef using CaseNo and coalesce(HORef, FCONumber)
    bronze_cleansing = bronze_HORef_cleansing.select(
        col("CaseNo"),
        coalesce(col("HORef"), col("FCONumber")).alias("lu_HORef")
    )

    # homeOfficeDecisionDate
    home_office_decision_date_expr = when(
        expr("array_contains(CategoryIdList, 37)") & col("DateOfApplicationDecision").isNotNull(),
        date_format(col("DateOfApplicationDecision"), "yyyy-MM-dd")
    )
    # IF CategoryId IN [38] AND  IF CleansedHORef & M1.HORef & M2.FCONumber NOT LIKE '%GWF%' = Include; ELSE OMIT | ISO 8601 Standard
    # decisionLetterReceivedDate
    decision_letter_received_date_expr = when(
        (expr("array_contains(CategoryIdList, 38)")) &
        ((col("lu_HORef").like("%GWF%")) | (col("HORef").like("%GWF%")) |(col("FCONumber").like("%GWF%"))) &
        (col("DateOfApplicationDecision").isNotNull()),
        date_format(col("DateOfApplicationDecision"), "yyyy-MM-dd")
    ).otherwise(None)


    # IF CategoryId IN [38] AND  IF lu_HORef & M1.HORef & M2.FCONumber LIKE '%GWF%' = Include;; ELSE OMIT | ISO 8601 Standard
    # dateEntryClearanceDecision
    date_entry_clearance_decision_expr = when(
        (expr("array_contains(CategoryIdList, 38)")) &
        ((col("lu_HORef").like("%GWF%")) | (col("HORef").like("%GWF%")) |(col("FCONumber").like("%GWF%"))) &
        (col("DateOfApplicationDecision").isNotNull()),
        date_format(col("DateOfApplicationDecision"), "yyyy-MM-dd")
    ).otherwise(None)

    # IF CategoryId IN [38] AND MainRespondentId IS 4 = OMIT; ELSE Include
    # IF CleansedHORef IS NULL USE HORef; IF HORef IS NULL USE FCONumber
    
    # homeOfficeReferenceNumber
    # IF CategoryId IN [38] AND IF lu_HORef & M1.HORef & M2.FCONumber LIKE '%GWF%' = OMIT; ELSE Include
    # IF CleansedHORef IS NULL USE HORef; IF HORef IS NULL USE FCONumber
    # homeOfficeReferenceNumber logic
    home_office_reference_number_expr = when(
        expr("array_contains(CategoryIdList, 38)") &
        (
            col("lu_HORef").like("%GWF%") |
            col("HORef").like("%GWF%") |
            col("FCONumber").like("%GWF%")
        ),
        lit(None)
    ).otherwise(
        coalesce(
            cleanReferenceNumberUDF(col("lu_HORef")),
            cleanReferenceNumberUDF(col("HORef")),
            cleanReferenceNumberUDF(col("FCONumber"))
        )
    )

    # IF CategoryId IN [38] AND  IF CleansedHORef & M1.HORef & M2.FCONumber LIKE '%GWF%' = Include; ELSE OMIT
    # IF CleansedHORef IS NULL USE HORef; IF HORef IS NULL USE FCONumber

    # gwfReferenceNumber logic
    gwf_reference_number_expr = when(
        expr("array_contains(CategoryIdList, 38)") 
        & ( col("lu_HORef").like("%GWF%") |
            col("HORef").like("%GWF%") |
            col("FCONumber").like("%GWF%")),
        coalesce(
            cleanReferenceNumberUDF(col("lu_HORef")),
            cleanReferenceNumberUDF(col("HORef")),
            cleanReferenceNumberUDF(col("FCONumber"))
        )
    ).otherwise(
        lit(None)
    )

    df = (
        silver_m1
        .join(silver_c_grouped, ["CaseNo"], "left")
        .join(silver_m2, ["CaseNo"], "left")
        .join(bronze_cleansing, ["CaseNo"], "left")
        .filter(conditions)
        .select(
            col("CaseNo"),
            # col("CategoryIdList"),
            home_office_decision_date_expr.alias("homeOfficeDecisionDate"),
            decision_letter_received_date_expr.alias("decisionLetterReceivedDate"),
            date_entry_clearance_decision_expr.alias("dateEntryClearanceDecision"),
            home_office_reference_number_expr.alias("homeOfficeReferenceNumber"),
            gwf_reference_number_expr.alias("gwfReferenceNumber"),
            lit("Yes").alias("isHomeOfficeIntegrationEnabled"),
            lit("Yes").alias("homeOfficeNotificationsEligible")
            # col("HORef"),
            # col("FCONumber"),
            # col("lu_HORef")
        )
    )

    common_inputFields = [lit("lu_appealType"), lit("dv_representation")]
    common_inputValues = [col("audit.lu_appealType"), col("audit.dv_representation")]

    df_audit = (
        silver_m1.alias("audit")
        .join(silver_m2.alias("audit_m2"), ["CaseNo"], "left")
        .join(df.alias("content"), ["CaseNo"], "left")
        .join(silver_c_grouped.alias("audit_c"), ["CaseNo"], "left")
        .join(bronze_cleansing.alias("content_c"), ["CaseNo"], "left")
        .select(
            col("CaseNo"),
            array(struct(*common_inputFields, lit("audit_c.CategoryIdList"))).alias("homeOfficeDecisionDate_inputFields"),
            array(struct(*common_inputValues, col("audit_c.CategoryIdList"))).alias("homeOfficeDecisionDate_inputValues"),
            col("content.homeOfficeDecisionDate"),
            lit("yes").alias("homeOfficeDecisionDate_Transformed"),

            array(struct(*common_inputFields, lit("audit_c.CategoryIdList"),  lit("content_c.lu_HORef"), lit("audit.HORef"), lit("audit_m2.FCONumber"))).alias("decisionLetterReceivedDate_inputFields"),
            array(struct(*common_inputValues, col("audit_c.CategoryIdList"), col("content_c.lu_HORef"), col("audit.HORef"), col("audit_m2.FCONumber"))).alias("decisionLetterReceivedDate_inputValues"),
            col("content.decisionLetterReceivedDate"),
            lit("yes").alias("decisionLetterReceivedDate_Transformed"),

            array(struct(*common_inputFields, lit("audit_c.CategoryIdList"), lit("content_c.lu_HORef"), lit("audit.HORef"), lit("audit_m2.FCONumber"))).alias("dateEntryClearanceDecision_inputFields"),
            array(struct(*common_inputValues, col("audit_c.CategoryIdList"), col("content_c.lu_HORef"), col("audit.HORef"), col("audit_m2.FCONumber"))).alias("dateEntryClearanceDecision_inputValues"),
            col("content.dateEntryClearanceDecision"),
            lit("yes").alias("dateEntryClearanceDecision_Transformed"),

            array(struct(*common_inputFields, lit("audit_c.CategoryIdList"), lit("content_c.lu_HORef"), lit("audit.HORef"), lit("audit_m2.FCONumber"))).alias("homeOfficeReferenceNumber_inputFields"),
            array(struct(*common_inputValues, col("audit_c.CategoryIdList"), col("content_c.lu_HORef"), col("audit.HORef"), col("audit_m2.FCONumber"))).alias("homeOfficeReferenceNumber_inputValues"),
            col("content.homeOfficeReferenceNumber"),
            lit("yes").alias("homeOfficeReferenceNumber_Transformed"),

            array(struct(*common_inputFields, lit("audit_c.CategoryIdList"), lit("content_c.lu_HORef"), lit("audit.HORef"), lit("audit_m2.FCONumber"))).alias("gwfReferenceNumber_inputFields"),
            array(struct(*common_inputValues, col("audit_c.CategoryIdList"), col("content_c.lu_HORef"), col("audit.HORef"), col("audit_m2.FCONumber"))).alias("gwfReferenceNumber_inputValues"),
            col("content.gwfReferenceNumber"),
            lit("yes").alias("gwfReferenceNumber_Transformed"),
        
            #isHomeOfficeIntegrationEnabled - ARIADM-797
            array(struct(*common_inputFields, lit("isHomeOfficeIntegrationEnabled"))).alias("isHomeOfficeIntegrationEnabled_inputFields"),
            array(struct(*common_inputValues, lit("null"))).alias("isHomeOfficeIntegrationEnabled_inputValues"),
            col("content.isHomeOfficeIntegrationEnabled"),
            lit("no").alias("isHomeOfficeIntegrationEnabled_Transformation"),

            #homeOfficeNotificationsEligible - ARIADM-797
            array(struct(*common_inputFields, lit("homeOfficeNotificationsEligible"))).alias("homeOfficeNotificationsEligible_inputFields"),
            array(struct(*common_inputValues, lit("null"))).alias("homeOfficeNotificationsEligible_inputValues"),
            col("content.homeOfficeNotificationsEligible"),
            lit("no").alias("homeOfficeNotificationsEligible_Transformation")
        )
    )

    return df, df_audit

# silver_m1 = (spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct())

# silver_m2 = (spark.table("ariadm_active_appeals.silver_caseapplicant_detail").filter(col("dv_targetState") == lit(AppealState)))

# silver_c = (spark.table("ariadm_active_appeals.silver_appealcategory_detail").filter(col("dv_targetState") == lit(AppealState)))

# bronze_HORef_cleansing = spark.table("ariadm_active_appeals.bronze_HORef_cleansing")

# df, df_audit = homeOfficeDetails(silver_m1, silver_m2, silver_c, bronze_HORef_cleansing)
# display(df)
# display(df_audit)

In [0]:
from pyspark.sql.functions import col, when, lit, array, struct

def paymentType(silver_m1):
    conditions_all = col("dv_CCDAppealType").isin(["EA", "EU", "HU", "PA"])

    payment_content = silver_m1.filter(conditions_all).select(
        col("CaseNo"),
        when(col("VisitVisatype") == 1, 8000)
            .when(col("VisitVisatype") == 2, 14000)
            .otherwise("unknown").alias("feeAmountGbp"),
        when(col("VisitVisatype") == 1, "Notice of Appeal - appellant consents without hearing A")
            .when(col("VisitVisatype") == 2, "Appeal determined with a hearing")
            .otherwise("unknown").alias("feeDescription"),
        when(col("VisitVisatype") == 1, None)
            .when(col("VisitVisatype") == 2, 140)
            .otherwise("unknown").alias("feeWithHearing"),
        when(col("VisitVisatype") == 1, 80)
            .when(col("VisitVisatype") == 2, None)
            .otherwise("unknown").alias("feeWithoutHearing"),
        when(col("VisitVisatype") == 1, "Appeal determined without a hearing")
            .when(col("VisitVisatype") == 2, "Appeal determined with a hearing")
            .otherwise("unknown").alias("paymentDescription"),
        lit("Yes").alias("feePaymentAppealType"),
        lit("Payment Pending").alias("paymentStatus"),
        lit(2).alias("feeVersion"),
        when(col("VisitVisatype") == 1, "decisionWithoutHearing")
            .when(col("VisitVisatype") == 2, "decisionWithHearing")
            .otherwise("unknown").alias("decisionHearingFeeOption"),
        lit("No").alias("hasServiceRequestAlready")
    )

    payment_common_inputFields = [lit("VisitVisatype")]
    payment_common_inputValues = [col("VisitVisatype")]

    payment_audit = silver_m1.filter(conditions_all).join(payment_content, on="CaseNo").filter(conditions_all).select(
        col("CaseNo"),

        # feeAmountGbp
        array(struct(*payment_common_inputFields)).alias("feeAmountGbp_inputFields"),
        array(struct(*payment_common_inputValues)).alias("feeAmountGbp_inputValues"),
        col("feeAmountGbp"),
        lit("yes").alias("feeAmountGbp_Transformation"),

        # feeDescription
        array(struct(*payment_common_inputFields)).alias("feeDescription_inputFields"),
        array(struct(*payment_common_inputValues)).alias("feeDescription_inputValues"),
        col("feeDescription"),
        lit("yes").alias("feeDescription_Transformation"),

        # feeWithHearing
        array(struct(*payment_common_inputFields)).alias("feeWithHearing_inputFields"),
        array(struct(*payment_common_inputValues)).alias("feeWithHearing_inputValues"),
        col("feeWithHearing"),
        lit("yes").alias("feeWithHearing_Transformation"),

        # feeWithoutHearing
        array(struct(*payment_common_inputFields)).alias("feeWithoutHearing_inputFields"),
        array(struct(*payment_common_inputValues)).alias("feeWithoutHearing_inputValues"),
        col("feeWithoutHearing"),
        lit("yes").alias("feeWithoutHearing_Transformation"),

        # paymentDescription
        array(struct(*payment_common_inputFields)).alias("paymentDescription_inputFields"),
        array(struct(*payment_common_inputValues)).alias("paymentDescription_inputValues"),
        col("paymentDescription"),
        lit("yes").alias("paymentDescription_Transformation"),

        # feePaymentAppealType (literal, so empty arrays)
        array().cast("array<struct<dummy:string>>").alias("feePaymentAppealType_inputFields"),
        array().cast("array<struct<dummy:string>>").alias("feePaymentAppealType_inputValues"),
        col("feePaymentAppealType"),
        lit("yes").alias("feePaymentAppealType_Transformation"),

        # paymentStatus (literal, so empty arrays)
        array().cast("array<struct<dummy:string>>").alias("paymentStatus_inputFields"),
        array().cast("array<struct<dummy:string>>").alias("paymentStatus_inputValues"),
        col("paymentStatus"),
        lit("yes").alias("paymentStatus_Transformation"),

        # feeVersion (literal, so empty arrays)
        array().cast("array<struct<dummy:string>>").alias("feeVersion_inputFields"),
        array().cast("array<struct<dummy:string>>").alias("feeVersion_inputValues"),
        col("feeVersion"),
        lit("yes").alias("feeVersion_Transformation"),

        # decisionHearingFeeOption
        array(struct(*payment_common_inputFields)).alias("decisionHearingFeeOption_inputFields"),
        array(struct(*payment_common_inputValues)).alias("decisionHearingFeeOption_inputValues"),
        col("decisionHearingFeeOption"),
        lit("yes").alias("decisionHearingFeeOption_Transformation"),

        # hasServiceRequestAlready (literal, so empty arrays)
        array().cast("array<struct<dummy:string>>").alias("hasServiceRequestAlready_inputFields"),
        array().cast("array<struct<dummy:string>>").alias("hasServiceRequestAlready_inputValues"),
        col("hasServiceRequestAlready"),
        lit("yes").alias("hasServiceRequestAlready_Transformation")
    )

    return payment_content, payment_audit


# Example usage:
# df, a_df = paymentType(silver_m1)
# display(df)
# display(a_df)

In [0]:
from pyspark.sql.functions import udf, when, col, lit, expr, collect_list
from pyspark.sql.types import StringType
import uuid

# Create UDF to generate UUID for relevant field as a string type
uuid4_udf = udf(lambda: str(uuid.uuid4()), StringType())

def partyID(silver_m1, silver_m3,silver_c):
    conditions_all = (col("lu_appealType").isNotNull())
    window_spec = Window.partitionBy("CaseNo").orderBy(col("StatusId").desc())

    # Add row_number to get the row with the highest StatusId per CaseNo
    silver_m3_ranked = silver_m3.withColumn("row_num", row_number().over(window_spec))

    # Filter the top-ranked rows where Outcome is not null
    silver_m3_filtered = silver_m3_ranked.filter(
        (col("row_num") == 1) & (col("Outcome").isNotNull())
    ).select(col("CaseNo"))

    # Create DataFrame with CaseNo and list of CategoryId
    silver_c_grouped = silver_c.groupBy("CaseNo").agg(collect_list(col("categoryId")).alias("CategoryIdList"))

    df = silver_m1.alias("m1").join(silver_m3_filtered.alias("m3"), on="CaseNo", how="left").join(silver_c_grouped.alias("c"), ["CaseNo"], "left"
    ).withColumn(
        "appellantsRepresentation", when(((col("m1.dv_representation") == "LR") &  (col("lu_appealType").isNotNull())), "No").when(((col("m1.dv_representation") == "AIP") & (col("lu_appealType").isNotNull())), "Yes").otherwise(None)
    ).withColumn(
        "appellantPartyId",
        when(conditions_all, expr("uuid()")).otherwise(expr("uuid()"))  #use expr() to ensure a unique UUID per row
    ).withColumn(
        "legalRepIndividualPartyId",
        when((col("appellantsRepresentation") == 'No'), expr("uuid()")).otherwise(None) #If appelRep = LR (no) then valid
    ).withColumn(
        "legalRepOrganisationPartyId",
        when((col("appellantsRepresentation") == 'No'), expr("uuid()")).otherwise(None) #If appelRep = LR (no) then valid
    ).withColumn(
        "sponsorPartyId",
        when(conditions_all & col("m1.Sponsor_Name").isNotNull() & (array_contains(col("CategoryIdList"), 38)), expr("uuid()")).otherwise(None) 
    ).select(
        col("m1.CaseNo"),
        col("appellantPartyId"),
        col("legalRepIndividualPartyId"),
        col("legalRepOrganisationPartyId"),
        col("sponsorPartyId")
    ).distinct()

    common_inputFields = [lit("dv_representation"), lit("lu_appealType")]
    common_inputValues = [col("audit.dv_representation"), col("audit.lu_appealType")]

    df_audit = silver_m1.join(silver_c, ["CaseNo"], "left").alias("audit").join(df.alias("content"), ["CaseNo"],"left").select(
        col("CaseNo"),

        #Audit appellantPartyId - ARIADM-779
        array(struct(*common_inputFields, lit("content.appellantPartyId"))).alias("appellantPartyId_inputFields"),
        array(struct(*common_inputValues, col("content.appellantPartyId"))).alias("appellantPartyId_inputValues"),
        col("content.appellantPartyId"),
        lit("yes").alias("appellantPartyId_Transformation"),

        #Audit legalRepIndividualPartyId - ARIADM-779
        array(struct(*common_inputFields, lit("content.legalRepIndividualPartyId"))).alias("legalRepIndividualPartyId_inputFields"),
        array(struct(*common_inputValues, col("content.legalRepIndividualPartyId"))).alias("legalRepIndividualPartyId_inputValues"),
        col("content.legalRepIndividualPartyId"),
        lit("yes").alias("legalRepIndividualPartyId_Transformation"),

        #Audit legalRepOrganisationPartyId - ARIADM-779
        array(struct(*common_inputFields, lit("content.legalRepOrganisationPartyId"))).alias("legalRepOrganisationPartyId_inputFields"),
        array(struct(*common_inputValues, col("content.legalRepOrganisationPartyId"))).alias("legalRepOrganisationPartyId_inputValues"),
        col("content.legalRepOrganisationPartyId"),
        lit("yes").alias("legalRepOrganisationPartyId_Transformation"),

        #Audit sponsorPartyId - ARIADM-779
        array(struct(*common_inputFields, lit("content.sponsorPartyId"))).alias("sponsorPartyId_inputFields"),
        array(struct(*common_inputValues, col("content.sponsorPartyId"))).alias("sponsorPartyId_inputValues"),
        col("content.sponsorPartyId"),
        lit("yes").alias("sponsorPartyId_Transformation")
    ).distinct()

    return df, df_audit

silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()
silver_c = spark.table("ariadm_active_appeals.silver_appealcategory_detail").filter(col("dv_targetState") == lit(AppealState))
silver_m3 = spark.table("ariadm_active_appeals.silver_status_detail").filter(col("dv_targetState") == lit(AppealState))
df_final,df_audit = partyID(silver_m1, silver_m3, silver_c)
display(df_final)
# display(df_audit)

In [0]:
from pyspark.sql.functions import col, when, lit

def remissionTypes(silver_m1, bronze_remission_lookup_df):

    #Including conditions specified in mapping document (column H)
    conditions_remissionTypes = col("dv_CCDAppealType").isin("EA", "EU", "HU", "PA")
    conditions = (col("dv_representation").isin('LR', 'AIP')) & (col("lu_appealType").isNotNull())

    # No need for all logic in ticket. The remission table exists in bronze. Left join on the unique field. Rename "Omit" to null. Results are correct from the samples I can see below.
    df = silver_m1.alias("m1").filter(conditions_remissionTypes & conditions).join(bronze_remission_lookup_df, on=["PaymentRemissionReason","PaymentRemissionRequested"], how="left"
        ).withColumn(
        "remissionType",
        col("remissionType")
    ).withColumn(
        "remissionClaim",
        when(col("remissionClaim") == lit("OMIT"), None).otherwise(col("remissionClaim"))
    ).withColumn(
        "feeRemissionType",
        when(col("feeRemissionType") == lit("OMIT"), None).otherwise(col("feeRemissionType"))
    ).withColumn(
        "exceptionalCircumstances",
        when(col("exceptionalCircumstances") == lit("OMIT"), None).otherwise(col("feeRemissionType"))

    ).withColumn("ReasonDescription", col("ReasonDescription")

    ).withColumn(
        "legalAidAccountNumber",
        when(col("legalAidAccountNumber") == lit("OMIT"), None                               #When set to OMIT, replace with NULL
        ).when(col("legalAidAccountNumber") == lit("M1.LSCReference; ELSE IF NULL 'Unknown'"),    #If record matches the string
        when(col("m1.LSCReference").isNotNull(), col("m1.LSCReference")).otherwise(lit("Unknown")) #Perform logic in the string
    ).otherwise(col("legalAidAccountNumber"))

    ).withColumn(
        "asylumSupportReference",
        when(col("asylumSupportReference") == lit("OMIT"), None                                  #When set to OMIT, replace with NULL
        ).when(col("asylumSupportReference") == lit("M1.ASFReferenceNo ELSE IF NULL 'Unknown'"),      #If record matches the string
        when(col("m1.ASFReferenceNo").isNotNull(), col("m1.ASFReferenceNo")).otherwise(lit("Unknown")) #Perform logic in the string
    ).otherwise(col("asylumSupportReference"))
        
    ).withColumn(
        "helpWithFeesReferenceNumber",
        when(col("helpWithFeesReferenceNumber") == lit("OMIT"), None                             #As above
        ).when(col("helpWithFeesReferenceNumber") == lit("M1.PaymentRemissionReasonNote; ELSE IF NULL 'Unknown'"),
        when(col("m1.PaymentRemissionReasonNote").isNotNull(), col("m1.PaymentRemissionReasonNote")).otherwise(lit("Unknown"))
        ).otherwise(col("helpWithFeesReferenceNumber"))

    ).select(
        col("CaseNo"),
        "remissionType",
        "remissionClaim",
        "feeRemissionType",
        "exceptionalCircumstances",
        "legalAidAccountNumber",
        "asylumSupportReference",
        "helpWithFeesReferenceNumber"
    )

    common_inputFields = [lit("dv_representation"), lit("lu_appealType")]
    common_inputValues = [col("m1_audit.dv_representation"), col("m1_audit.lu_appealType")]


    df_audit = silver_m1.alias("m1_audit").filter(conditions_remissionTypes).join(df.alias("content"), on = ["CaseNo"], how = "left").select(
        col("CaseNo"),

        ## remissionType - ARIADM-784
        array(struct(*common_inputFields ,lit("content.remissionType"))).alias("remissionType_inputFields"),
        array(struct(*common_inputValues ,col("content.remissionType"))).alias("remissionType_inputValues"),
        col("content.remissionType"),
        lit("yes").alias("remissionType_Transformed"),

        ## remissionClaim - ARIADM-784
        array(struct(*common_inputFields ,lit("content.remissionClaim"))).alias("remissionClaim_inputFields"),
        array(struct(*common_inputValues ,col("content.remissionClaim"))).alias("remissionClaim_inputValues"),
        col("content.remissionClaim"),
        lit("yes").alias("remissionClaim_Transformed"),

        ## feeRemissionType - ARIADM-784
        array(struct(*common_inputFields ,lit("content.feeRemissionType"))).alias("feeRemissionType_inputFields"),
        array(struct(*common_inputValues ,col("content.feeRemissionType"))).alias("feeRemissionType_inputValues"),
        col("content.feeRemissionType"),
        lit("yes").alias("feeRemissionType_Transformed"),

        ## exceptionalCircumstances - ARIADM-786
        array(struct(*common_inputFields ,lit("content.exceptionalCircumstances"))).alias("exceptionalCircumstances_inputFields"),
        array(struct(*common_inputValues ,col("content.exceptionalCircumstances"))).alias("exceptionalCircumstances_inputValues"),
        col("content.exceptionalCircumstances"),
        lit("yes").alias("exceptionalCircumstances_Transformed"),

        ## legalAidAccountNumber - ARIADM-786
        array(struct(*common_inputFields ,lit("content.legalAidAccountNumber"), lit("LSCReference"))).alias("legalAidAccountNumber_inputFields"),
        array(struct(*common_inputValues ,col("content.legalAidAccountNumber"), col("LSCReference"))).alias("legalAidAccountNumber_inputValues"),
        col("content.legalAidAccountNumber"),
        lit("yes").alias("legalAidAccountNumber_Transformed"),

        ## asylumSupportReference - ARIADM-786
        array(struct(*common_inputFields ,lit("content.asylumSupportReference"), lit("ASFReferenceNo"))).alias("asylumSupportReference_inputFields"),
        array(struct(*common_inputValues ,col("content.asylumSupportReference"), col("ASFReferenceNo"))).alias("asylumSupportReference_inputValues"),
        col("content.asylumSupportReference"),
        lit("yes").alias("asylumSupportReference_Transformed"),

        ## helpWithFeesReferenceNumber - ARIADM-786
        array(struct(*common_inputFields ,lit("content.helpWithFeesReferenceNumber"), lit("PaymentRemissionReasonNote"))).alias("helpWithFeesReferenceNumber_inputFields"),
        array(struct(*common_inputValues ,col("content.helpWithFeesReferenceNumber"), col("PaymentRemissionReasonNote"))).alias("helpWithFeesReferenceNumber_inputValues"),
        col("content.helpWithFeesReferenceNumber"),
        lit("yes").alias("helpWithFeesReferenceNumber_Transformed")
    )

    return df, df_audit

silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()
bronze_remission_lookup_df = spark.table("ariadm_active_appeals.bronze_remissions").distinct()
display(bronze_remission_lookup_df)
df_final, df_audit = remissionTypes(silver_m1, bronze_remission_lookup_df)
display(df_final)
# display(df_audit)
# display(bronze_remission_lookup_df)
# display(silver_m1)

In [0]:
from pyspark.sql.functions import when, col, first, lit, concat_ws, trim, collect_list, array_contains

def sponsorDetails(silver_m1, silver_c):
    m1 = silver_m1.alias("m1")
    c = silver_c.alias("c")

    joined = m1.join(c, on='CaseNo', how="left")

    grouped = joined.groupBy("CaseNo").agg(
        collect_list("CategoryId").alias("CategoryIdList"),
        first("Sponsor_Name", ignorenulls=True).alias("Sponsor_Name"),
        first("Sponsor_Forenames", ignorenulls=True).alias("Sponsor_Forenames"),
        first("Sponsor_Address1", ignorenulls=True).alias("Sponsor_Address1"),
        first("Sponsor_Address2", ignorenulls=True).alias("Sponsor_Address2"),
        first("Sponsor_Address3", ignorenulls=True).alias("Sponsor_Address3"),
        first("Sponsor_Address4", ignorenulls=True).alias("Sponsor_Address4"),
        first("Sponsor_Address5", ignorenulls=True).alias("Sponsor_Address5"),
        first("Sponsor_Postcode", ignorenulls=True).alias("Sponsor_Postcode"),
        first("Sponsor_Authorisation", ignorenulls=True).alias("Sponsor_Authorisation"),
        first("Sponsor_Email", ignorenulls=True).alias("Sponsor_Email"),
        first("Sponsor_Telephone", ignorenulls=True).alias("Sponsor_Telephone")
    )

    sponsor_condition = array_contains(col("CategoryIdList"), 38)

    grouped = grouped.withColumn(
        "hasSponsor",
        when(sponsor_condition, when(col("Sponsor_Name").isNotNull(), lit("Yes")).otherwise(lit("No")))
    ).withColumn(
        "sponsorGivenNames",
        when(sponsor_condition & (col("hasSponsor") == "Yes"), col("Sponsor_Forenames"))
    ).withColumn(
        "sponsorFamilyName",
        when(sponsor_condition & (col("hasSponsor") == "Yes"), col("Sponsor_Name"))
    ).withColumn(
        "sponsorAuthorisation",
        when(
            sponsor_condition & (col("hasSponsor") == "Yes"),
            when(col("Sponsor_Authorisation") == 1, lit("Yes")).otherwise(lit("No"))
        )
    ).withColumn(
        "sponsorAddress",
        when(
            sponsor_condition & (col("hasSponsor") == "Yes"),
            trim(
                concat_ws(
                    " ",
                    col("Sponsor_Address1"),
                    col("Sponsor_Address2"),
                    col("Sponsor_Address3"),
                    col("Sponsor_Address4"),
                    col("Sponsor_Address5"),
                    col("Sponsor_Postcode")
                )
            )
        )
    ).withColumn(
        "sponsorEmailAdminJ",
        when(sponsor_condition & (col("hasSponsor") == "Yes"),
             cleanEmailUDF(col("Sponsor_Email")))
    ).withColumn(
        "sponsorMobileNumberAdminJ",
        when(sponsor_condition & (col("hasSponsor") == "Yes"),
             phoneNumberUDF(col("Sponsor_Telephone")))
    )

    df = grouped.select(
        "CaseNo",
        # "CategoryIdList",
        "hasSponsor",
        "sponsorGivenNames",
        "sponsorFamilyName",
        "sponsorAddress",
        "sponsorEmailAdminJ",
        "sponsorMobileNumberAdminJ",
        "sponsorAuthorisation"
    )

    common_inputFields = [lit("dv_representation"), lit("lu_appealType")]
    common_inputValues = [col("audit.dv_representation"), col("audit.lu_appealType")]

    df_audit = silver_m1.alias("audit").join(df.alias("content"), ["CaseNo"], "left"
            ).join(grouped.alias("grp"), ["CaseNo"], "left").select(
        col("CaseNo"),
        
        #audit hasSponsor
        array(struct(*common_inputFields, lit("audit.Sponsor_Name"), lit("grp.CategoryIdList"))).alias("hasSponsor_inputFields"),
        array(struct(*common_inputValues, col("audit.Sponsor_Name"), col("grp.CategoryIdList"))).alias("hasSponsor_inputValues"),
        col("content.hasSponsor"),
        lit("yes").alias("hasSponsor_Transformation"),

        #audit sponsorGivenNames
        array(struct(*common_inputFields, lit("audit.Sponsor_Forenames"), lit("grp.CategoryIdList"))).alias("sponsorGivenName_inputFields"),
        array(struct(*common_inputValues, col("audit.Sponsor_Forenames"), col("grp.CategoryIdList"))).alias("sponsorGivenName_inputValues"),
        col("content.sponsorGivenNames"),
        lit("yes").alias("sponsorGivenNames_Transformation"),

        #audit sponsorFamilyName
        array(struct(*common_inputFields, lit("audit.Sponsor_Name"), lit("grp.CategoryIdList"))).alias("sponsorFamilyName_inputFields"),
        array(struct(*common_inputValues, col("audit.Sponsor_Name"), col("grp.CategoryIdList"))).alias("sponsorFamilyName_inputValues"),
        col("content.sponsorFamilyName"),
        lit("yes").alias("sponsorFamilyName_Transformation"),

        #audit sponsorAddress
        array(struct(*common_inputFields, lit("audit.Sponsor_Address1"), lit("audit.Sponsor_Address2"), lit("audit.Sponsor_Address3"), lit("audit.Sponsor_Address4"), lit("audit.Sponsor_Address5"), lit("audit.Sponsor_Postcode"), lit("grp.CategoryIdList"))).alias("sponsorAddress.inputFields"),
        array(struct(*common_inputValues, col("audit.Sponsor_Address1"), col("audit.Sponsor_Address2"), col("audit.Sponsor_Address3"), col("audit.Sponsor_Address4"), col("audit.Sponsor_Address5"), col("audit.Sponsor_Postcode"), col("grp.CategoryIdList"))).alias("sponsorAddress.inputValues"),
        col("content.sponsorAddress"),
        lit("yes").alias("sponsorAddress_Transformation"),

        #audit sponsorAuthorisation
        array(struct(*common_inputFields, lit("audit.Sponsor_Authorisation"), lit("grp.CategoryIdList"))).alias("sponsorAuthorisation_inputFields"),
        array(struct(*common_inputValues, col("audit.Sponsor_Authorisation"), col("grp.CategoryIdList"))).alias("sponsorAuthorisation_inputValues"),
        col("content.sponsorAuthorisation"),
        lit("yes").alias("sponsorAuthorisation_Transformation"),

        #audit sponsorEmailAdminJ
        array(struct(*common_inputFields, lit("audit.Sponsor_Email"), lit("grp.CategoryIdList"))).alias("sponsorEmailAdminJ_inputFields"),
        array(struct(*common_inputValues, col("audit.Sponsor_Email"), col("grp.CategoryIdList"))).alias("sponsorEmailAdminJ.inputValues"),
        col("content.sponsorEmailAdminJ"),
        lit("yes").alias("sponsorEmailAdminJ_Transformation"),

        #audit sponsorMobileNumberAdminJ
        array(struct(*common_inputFields, lit("audit.Sponsor_Telephone"), lit("grp.CategoryIdList"))).alias("sponsorMobileNumberAdminJ_inputFields"),
        array(struct(*common_inputValues, col("audit.Sponsor_Telephone"), col("grp.CategoryIdList"))).alias("sponsorMobileNumberAdminJ.inputValues"),
        col("content.sponsorMobileNumberAdminJ"),
        lit("yes").alias("sponsorMobileNumberAdminJ_Transformation")
    )

    return df, df_audit


# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail") \
#     .filter(col("dv_targetState") == lit(AppealState)) \
#     .distinct()

# silver_c = spark.table("ariadm_active_appeals.silver_appealcategory_detail")

# df, df_audit = sponsorDetails(silver_m1, silver_c)
# display(df)
# display(df_audit)

In [0]:
def general(silver_m1):
    conditions_general = (col("dv_representation").isin('LR','AIP')) & (col("lu_appealType").isNotNull())

    df = silver_m1.filter(conditions_general).withColumn(
        "isServiceRequestTabVisibleConsideringRemissions",
        when(
            (col("PaymentRemissionRequested").isNull()) | (col("PaymentRemissionRequested") == lit('2')),
            "Yes"
        ).otherwise("No")
    ).select(
        col("CaseNo"),
        "isServiceRequestTabVisibleConsideringRemissions",
        col("lu_applicationChangeDesignatedHearingCentre"),
    )

    common_inputFields = [lit("dv_representation"), lit("lu_appealType")]
    common_inputValues = [col("m1_audit.dv_representation"), col("m1_audit.lu_appealType")]

    df_audit = silver_m1.alias("m1_audit").join(df.alias("content"), ["CaseNo"], "left").select(
        col("CaseNo"),

        ## isServiceRequestTabVisibleConsideringRemissions - ARIADM-767
        array(struct(*common_inputFields ,lit("m1_audit.PaymentRemissionRequested"), lit("content.isServiceRequestTabVisibleConsideringRemissions"))).alias("isServiceRequestTabVisibleConsideringRemissions_inputFields"),
        array(struct(*common_inputValues ,col("m1_audit.PaymentRemissionRequested"), col("content.isServiceRequestTabVisibleConsideringRemissions"))).alias("isServiceRequestTabVisibleConsideringRemissions_inputValues"),
        col("content.isServiceRequestTabVisibleConsideringRemissions"),
        lit("yes").alias("isServiceRequestTabVisibleConsideringRemissions_Transformed"),

        # ## lu_applicationChangeDesignatedHearingCentre - ARIADM-767
       array(struct(*common_inputFields ,lit("m1_audit.lu_applicationChangeDesignatedHearingCentre"))).alias("lu_applicationChangeDesignatedHearingCentre_inputFields"),
        array(struct(*common_inputValues ,col("m1_audit.lu_applicationChangeDesignatedHearingCentre"))).alias("lu_applicationChangeDesignatedHearingCentre_inputValues"),
        col("content.lu_applicationChangeDesignatedHearingCentre"),
        lit("yes").alias("lu_applicationChangeDesignatedHearingCentre_Transformed")
    )

    return df, df_audit

# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState))
# df, df_audit = general(silver_m1)

# display(df)
# display(df_audit)
# display(silver_m1)

In [0]:

def generalDefault(silver_m1):

    conditions_generalDefault = (col("dv_representation").isin('LR','AIP')) & (col("lu_appealType").isNotNull())

    df = silver_m1.filter(conditions_generalDefault
                ).withColumn("notificationsSent", lit([]).cast("array<string>")
                ).withColumn("submitNotificationStatus", lit("")
                ).withColumn("isFeePaymentEnabled", lit("Yes")
                ).withColumn("isRemissionsEnabled", lit("Yes")
                ).withColumn("isOutOfCountryEnabled", lit("Yes")
                ).withColumn("isIntegrated", lit("No")
                ).withColumn("isNabaEnabled", lit("No")
                ).withColumn("isNabaAdaEnabled", lit("Yes")
                ).withColumn("isNabaEnabledOoc", lit("No")
                ).withColumn("isCaseUsingLocationRefData", lit("Yes")
                ).withColumn("hasAddedLegalRepDetails", lit("Yes")
                ).withColumn("autoHearingRequestEnabled", lit("No")
                ).withColumn("isDlrmFeeRemissionEnabled", lit("Yes")
                ).withColumn("isDlrmFeeRefundEnabled", lit("Yes")
                ).withColumn("sendDirectionActionAvailable", lit("Yes")
                ).withColumn("changeDirectionDueDateActionAvailable", lit("No")
                ).withColumn("markEvidenceAsReviewedActionAvailable", lit("No")
                ).withColumn("uploadAddendumEvidenceActionAvailable", lit("No")
                ).withColumn("uploadAdditionalEvidenceActionAvailable", lit("No")
                ).withColumn("displayMarkAsPaidEventForPartialRemission", lit("No")
                ).withColumn("haveHearingAttendeesAndDurationBeenRecorded", lit("No")
                ).withColumn("markAddendumEvidenceAsReviewedActionAvailable", lit("No")
                ).withColumn("uploadAddendumEvidenceLegalRepActionAvailable", lit("No")
                ).withColumn("uploadAddendumEvidenceHomeOfficeActionAvailable", lit("No")
                ).withColumn("uploadAddendumEvidenceAdminOfficerActionAvailable", lit("No")
                ).withColumn("uploadAdditionalEvidenceHomeOfficeActionAvailable", lit("No")                
    ).select(
        col("CaseNo"),
        # lit("").alias("ccdReferenceNumberForDisplay"), Done as part of AppealType 
        # lit("No").alias("hasOtherAppeals"), Done as part of payment pending
        # array(lit("hasDeclared")).alias("adminDeclaration1"), Done as part of caseData
        # lit("No").alias("s94bStatus"), Done as part of payment pending
        # lit("Yes").alias("isAdmin"), Done as part of payment pending
        # lit("No").alias("isEjp"), Done as part of payment pending
        # "isHomeOfficeIntegrationEnabled", Done as part of home office
        # "homeOfficeNotificationsEligible", done as part of home office
        # "ariaDesiredState", done as part of case data
        # "ariaMigrationTaskDueDays", done as part of case data
        "notificationsSent",
        "submitNotificationStatus",
        "isFeePaymentEnabled",
        "isRemissionsEnabled",
        "isOutOfCountryEnabled",
        "isIntegrated",
        "isNabaEnabled",
        "isNabaAdaEnabled",
        "isNabaEnabledOoc",
        "isCaseUsingLocationRefData",
        "hasAddedLegalRepDetails",
        "autoHearingRequestEnabled",
        "isDlrmFeeRemissionEnabled",
        "isDlrmFeeRefundEnabled",
        "sendDirectionActionAvailable",
        "changeDirectionDueDateActionAvailable",
        "markEvidenceAsReviewedActionAvailable",
        "uploadAddendumEvidenceActionAvailable",
        "uploadAdditionalEvidenceActionAvailable",
        "displayMarkAsPaidEventForPartialRemission",
        "haveHearingAttendeesAndDurationBeenRecorded",
        "markAddendumEvidenceAsReviewedActionAvailable",
        "uploadAddendumEvidenceLegalRepActionAvailable",
        "uploadAddendumEvidenceHomeOfficeActionAvailable",
        "uploadAddendumEvidenceAdminOfficerActionAvailable",
        "uploadAdditionalEvidenceHomeOfficeActionAvailable"
    )

    return df


# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState))
# df= generalDefault(silver_m1)

# display(df)
# display(df_audit)
# display(silver_m1)

In [0]:
def documents(silver_m1):

    documents_content = silver_m1.select(
        "CaseNo",
        lit([]).cast("array<string>").alias("uploadTheAppealFormDocs"),
        lit([]).cast("array<string>").alias("caseNotes"),
        lit([]).cast("array<string>").alias("tribunalDocuments"),
        lit([]).cast("array<string>").alias("legalRepresentativeDocuments")

        # array().cast("array<string>").alias("uploadTheAppealFormDocs"),  # Temp logic
        # array().cast("array<string>").alias("caseNotes"),
        # array().cast("array<string>").alias("tribunalDocuments"),        # Temp logic
        # array().cast("array<string>").alias("legalRepresentativeDocuments")  # Temp logic
    )

    common_inputFields = [lit("dv_representation"), lit("lu_appealType")]
    common_inputValues = [col("audit.dv_representation"), col("audit.lu_appealType")]

    documents_audit = silver_m1.alias("audit").join(documents_content.alias("content"), on = ["CaseNo"], how = "left").select(
        col("CaseNo"),

        #-----# uploadTheAppealFormDocs #-----# 
        array(struct(*common_inputFields, lit("uploadTheAppealFormDocs"))).alias("uploadTheAppealFormDocs_inputFields"),
        array(struct(*common_inputValues, lit("null"))).alias("uploadTheAppealFormDocs_inputValues"),
        col("uploadTheAppealFormDocs"),
        lit("no").alias("uploadTheAppealFormDocs_Transformation"),

        #-----# caseNotes #-----# 
        array(struct(*common_inputFields, lit("caseNotes"))).alias("caseNotes_inputFields"),
        array(struct(*common_inputValues, lit("null"))).alias("caseNotes_inputValues"),
        col("caseNotes"),
        lit("no").alias("caseNotes_Transformation"),

        #-----# tribunalDocuments #-----# 
        array(struct(*common_inputFields, lit("tribunalDocuments"))).alias("tribunalDocuments_inputFields"),
        array(struct(*common_inputValues, lit("null"))).alias("tribunalDocuments_inputValues"),
        col("tribunalDocuments"),
        lit("no").alias("tribunalDocuments_Transformation"),

        #-----# legalRepresentativeDocuments #-----# 
        array(struct(*common_inputFields, lit("legalRepresentativeDocuments"))).alias("legalRepresentativeDocuments_inputFields"),
        array(struct(*common_inputValues, lit("null"))).alias("legalRepresentativeDocuments_inputValues"),
        col("legalRepresentativeDocuments"),
        lit("no").alias("legalRepresentativeDocuments_Transformation")
    )

    return documents_content, documents_audit

# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState))
# display(silver_m1)
# documents_content, documents_audit = documents(silver_m1)
# display(documents_content)
# display(documents_audit)

In [0]:
def caseState(silver_m1):

    df = silver_m1.select(
        "CaseNo", 
        lit("paymentPending").alias("ariaDesiredState"),
        lit("14").alias("ariaMigrationTaskDueDays")
    )

    return df

# df, df_audit = caseState(silver_m1)
# display(df)
# display(df_audit)

In [0]:
# display(silver_m1.select("CaseNo", "lu_appealType"))

In [0]:
from pyspark.sql.types import StringType, StructType, ArrayType, MapType
from pyspark.sql.functions import col, lit, to_json, struct, concat, regexp_replace
from datetime import datetime

def mainPaymentPending(silver_m1, silver_m2, silver_m3, silver_c,silver_h, bronze_remissions, bronze_countryFromAddress, bronze_HORef_cleansing,bronze_hearing_centres,bronze_derive_hearing_centres):


    AppealState = "paymentPending"

    # Aggregate details
    AppealType_df, AppealType_df_audit = appealType(silver_m1)
    caseData_df, caseData_df_audit = caseData(silver_m1, silver_m2, silver_m3, silver_h, bronze_hearing_centres, bronze_derive_hearing_centres)
    flagsLabels_df, flagsLabels_df_audit = flagsLabels(silver_m1, silver_m2, silver_c)
    appellantDetails_df, appellantDetails_df_audit = appellantDetails(silver_m1, silver_m2, silver_c, bronze_countryFromAddress, bronze_HORef_cleansing)
    legalRepDetails_df, legalRepDetails_df_audit = legalRepDetails(silver_m1)
    partyID_df, partyID_df_audit = partyID(silver_m1, silver_m3, silver_c)
    payment_df, payment_df_audit = paymentType(silver_m1)
    homeOfficeDetails_df, homeOfficeDetails_df_audit = homeOfficeDetails(silver_m1, silver_m2, silver_c, bronze_HORef_cleansing)
    remissionTypes_df, remissionTypes_df_audit = remissionTypes(silver_m1, bronze_remissions)
    sponsorDetails_df, sponsorDetails_df_audit = sponsorDetails(silver_m1, silver_c)
    general_df, general_df_audit = general(silver_m1)
    generalDefault_df = generalDefault(silver_m1)
    documents_df, documents_df_audit = documents(silver_m1)
    caseState_df  = caseState(silver_m1)

    # Join all aggregated data with Appeal Case Details
    df_combined = (
        AppealType_df.join(caseData_df, on="CaseNo", how="left")
        .join(legalRepDetails_df, on="CaseNo", how="left")
        .join(appellantDetails_df, on="CaseNo", how="left")
        .join(flagsLabels_df, on="CaseNo", how="left")
        .join(partyID_df, on="CaseNo", how="left")
        .join(homeOfficeDetails_df, on="CaseNo", how="left")
        .join(remissionTypes_df, on="CaseNo", how="left")
        .join(sponsorDetails_df, on="CaseNo", how="left")
        .join(payment_df, on="CaseNo", how="left")
        .join(general_df, on="CaseNo", how="left")
        .join(generalDefault_df, on="CaseNo", how="left")
        .join(documents_df, on="CaseNo", how="left")
        .join(caseState_df, on = "CaseNo", how="left")
    )

    # Join all aggregated data with Appeal Case Details
    df_combined_audit = (
        AppealType_df_audit.join(caseData_df_audit, on="CaseNo", how="left")
        .join(legalRepDetails_df_audit, on="CaseNo", how="left")
        .join(appellantDetails_df_audit, on="CaseNo", how="left")
        .join(flagsLabels_df_audit, on="CaseNo", how="left")
        .join(partyID_df_audit, on="CaseNo", how="left")
        .join(homeOfficeDetails_df_audit, on="CaseNo", how="left")
        .join(remissionTypes_df_audit, on="CaseNo", how="left")
        .join(sponsorDetails_df_audit, on="CaseNo", how="left")
        .join(payment_df_audit, on="CaseNo", how="left")
        .join(general_df_audit, on="CaseNo", how="left")
        .join(documents_df_audit, on="CaseNo", how="left")
    )

    Datetime_name = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

    # Create JSON and filename and omit columns that are with null values
    df_final = df_combined.withColumn(
        "JSON_Content", to_json(struct(*df_combined.drop(col("CaseNo")).columns))
    ).withColumn(
        "JSON_File_name", concat(lit(f"{gold_outputs}/{Datetime_name}/JSON/APPEALS_"), regexp_replace(col("CaseNo"), "/", "_"), lit(".json"))
    )
    
    return df_final, df_combined_audit

# silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()
# silver_m2 =  spark.table("ariadm_active_appeals.silver_caseapplicant_detail").filter(col("dv_targetState") == lit(AppealState))
# silver_m3 = spark.table("ariadm_active_appeals.silver_status_detail").filter(col("dv_targetState") == lit(AppealState))
# silver_c = spark.table("ariadm_active_appeals.silver_appealcategory_detail").filter(col("dv_targetState") == lit(AppealState))
# silver_h = spark.table("ariadm_active_appeals.silver_history_detail")
# bronze_remission_lookup_df = spark.table("ariadm_active_appeals.bronze_remissions").distinct()

# bronze_countryFromAddress = spark.table("ariadm_active_appeals.bronze_countries_countryFromAddress").withColumn("lu_countryGovUkOocAdminJ",col("countryGovUkOocAdminJ"))

# bronze_hearing_centres = spark.table("ariadm_active_appeals.bronze_hearing_centres")
# bronze_derive_hearing_centres = spark.table("ariadm_active_appeals.bronze_derive_hearing_centres")

# bronze_HORef_cleansing = spark.table("ariadm_active_appeals.bronze_HORef_cleansing")

# df_final, df_audit = mainPaymentPending(silver_m1, silver_m2, silver_m3,silver_c,silver_h, bronze_remission_lookup_df,bronze_countryFromAddress,bronze_HORef_cleansing,bronze_hearing_centres,bronze_derive_hearing_centres)

# display(df_final)


# df,df_audit = caseData(silver_m1,silver_m2, silver_m3,silver_h,bronze_hearing_centres,bronze_derive_hearing_centres)


### Function: Upload  and Blob Client Connection Configuration

In [0]:
secret = dbutils.secrets.get(KeyVault_name, f"CURATED-{env_name}-SAS-TOKEN")

In [0]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os

# Set up the BlobServiceClient with your connection string
connection_string = secret

blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Specify the container name
container_name = "gold"
container_client = blob_service_client.get_container_client(container_name)

In [0]:
# Upload HTML to Azure Blob Storage
def upload_to_blob(file_name, file_content):
    try:
        # blob_client = container_client.get_blob_client(f"{gold_outputs}/HTML/{file_name}")
        blob_client = container_client.get_blob_client(f"{file_name}")
        blob_client.upload_blob(file_content, overwrite=True)
        return "success"
    except Exception as e:
        return f"error: {str(e)}"

# Register the upload function as a UDF
upload_udf = udf(upload_to_blob)

# df_with_upload_status = df_final.withColumn(
#     "Status", upload_udf(col("JSON_File_name"), col("JSON_Content"))
# )

# display(df_with_upload_status)


## Gold Outputs and Tracking DLT Table Creation

![DQValidation.png](./Images/DQValidation.png "DQValidation.png")

In [0]:
# Define a dictionary to hold data quality checks
checks = {}

# ##############################
# # ARIADM-669 (appealType)
# ##############################
checks["valid_appealReferenceNumber_not_null"] = "(appealReferenceNumber IS NOT NULL)"

# ##############################
# # ARIADM-671 (appealType)
# ##############################
checks["valid_appealtype_in_allowed_values"] = (
    "(AppealType IN ('refusalOfHumanRights', 'refusalOfEu', 'deprivation', 'protection', 'revocationOfProtection', 'euSettlementScheme'))"
)
checks["valid_hmctsCaseCategory_not_null"] = "(hmctsCaseCategory IS NOT NULL)"
checks["valid_appealTypeDescription_not_null"] = "(appealTypeDescription IS NOT NULL)"
# Null Values as accepted values as where Representation = AIP
checks["valid_caseManagementCategory_code_in_list_items"] = """
(
  caseManagementCategory.value.code IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(caseManagementCategory.list_items, x -> x.code),
    caseManagementCategory.value.code
  )
)
"""
checks["valid_caseManagementCategory_label_in_list_items"] = """
(
  caseManagementCategory.value.label IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(caseManagementCategory.list_items, x -> x.label),
    caseManagementCategory.value.label
  )
)
"""

# ##############################
# # ARIADM-673 (caseData)

# \d is a regular expression (regex) metacharacter that matches any single digit from 0 to 9.
# "yyyy-mm-ddTHH:mm:ssZ" r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$'" for ISO 8601 datetime format
# "yyyy-MM-dd" r'^\d{4}-\d{2}-\d{2}$' for ISO 8601 date format
##############################
checks["valid_appealSubmissionDate_format"] = (
    "(appealSubmissionDate RLIKE r'^\\d{4}-\\d{2}-\\d{2}$')"
)
checks["valid_appealSubmissionInternalDate_format"] = (
    "(appealSubmissionInternalDate RLIKE r'^\\d{4}-\\d{2}-\\d{2}$')"
)
checks["valid_tribunalReceivedDate_format"] = (
    "(tribunalReceivedDate RLIKE r'^\\d{4}-\\d{2}-\\d{2}$')"
)

# ##############################
# # ARIADM-675 (caseData)
# ##############################
checks["valid_appellantsRepresentation_yes_no"] = (
    "(appellantsRepresentation IS NOT NULL AND appellantsRepresentation IN ('Yes', 'No'))"
)
checks["valid_submissionOutOfTime_yes_no"] = (
    "(submissionOutOfTime IS NOT NULL AND submissionOutOfTime IN ('Yes', 'No'))"
)
checks["valid_recordedOutOfTimeDecision_yes_no_or_null"] = (
    "(recordedOutOfTimeDecision IS NULL OR recordedOutOfTimeDecision IN ('Yes', 'No'))"
)
checks["valid_applicationOutOfTimeExplanation_yes_no_or_null"] = (
    "(applicationOutOfTimeExplanation IS NULL OR applicationOutOfTimeExplanation IN ('Yes', 'No'))"
)

# ##############################
# # ARIADM-708 (CaseData)
# ##############################
checks["valid_hearingCentre_in_allowed_values"] = """
(
    hearingCentre IN ('taylorHouse', 'newport', 'newcastle', 'manchester', 'hattonCross', 
    'glasgow', 'bradford', 'birmingham', 'arnhemHouse', 'crownHouse', 'harmondsworth', 
    'yarlsWood', 'remoteHearing', 'decisionWithoutHearing')
)
"""
checks["valid_staffLocation_not_null"] = "(staffLocation IS NOT NULL)"
checks["valid_caseManagementLocation_region_and_baseLocation"] = """
(
  caseManagementLocation.region = '1' AND
  caseManagementLocation.baseLocation IN (
    '231596', '698118', '366559', '386417', '512401',
    '227101', '765324', '366796', '324339', '649000',
    '999971', '420587', '28837'
  )
)
"""
checks["valid_hearingCentreDynamicList_code_in_list_items"] = """
(
  hearingCentreDynamicList.value.code IS NOT NULL AND
  ARRAY_CONTAINS(
    TRANSFORM(hearingCentreDynamicList.list_items, x -> x.code),
    hearingCentreDynamicList.value.code
  )
)
"""
checks["valid_hearingCentreDynamicList_label_in_list_items"] = """
(
  hearingCentreDynamicList.value.label IS NOT NULL AND
  ARRAY_CONTAINS(
    TRANSFORM(hearingCentreDynamicList.list_items, x -> x.label),
    hearingCentreDynamicList.value.label
  )
)
"""
checks["valid_caseManagementLocationRefData_code_in_list_items"] = """
(
  caseManagementLocationRefData.baseLocation.value.code IS NOT NULL AND
  ARRAY_CONTAINS(
    TRANSFORM(caseManagementLocationRefData.baseLocation.list_items, x -> x.code),
    caseManagementLocationRefData.baseLocation.value.code
  )
)
"""
checks["valid_caseManagementLocationRefData_label_in_list_items"] = """
(
  caseManagementLocationRefData.baseLocation.value.label IS NOT NULL AND
  ARRAY_CONTAINS(
    TRANSFORM(caseManagementLocationRefData.baseLocation.list_items, x -> x.label),
    caseManagementLocationRefData.baseLocation.value.label
  )
)
"""
checks["valid_selectedHearingCentreRefData_not_null"] = "(selectedHearingCentreRefData IS NOT NULL)"


# ##############################
# # ARIADM-768 (legalRepDetails)
# # Null Values as accepted values as where Representation = AIP
# ##############################

checks["valid_legalRepGivenName_not_null"] = "((dv_representation = 'LR' AND legalRepGivenName IS NOT NULL) OR (dv_representation != 'LR' AND legalRepGivenName IS NULL))"

checks["valid_legalRepFamilyNamePaperJ_not_null"] = "((dv_representation = 'LR' AND legalRepFamilyNamePaperJ IS NOT NULL) OR (dv_representation != 'LR' AND legalRepFamilyNamePaperJ IS NULL))"

checks["valid_legalRepCompanyPaperJ_not_null"] = "((dv_representation = 'LR' AND legalRepCompanyPaperJ IS NOT NULL) OR (dv_representation != 'LR' AND legalRepCompanyPaperJ IS NULL))"


# ##############################
# # ARIADM-756 (appellantDetails)
# ##############################
checks["valid_appellantFamilyName_not_null"] = "(appellantFamilyName IS NOT NULL)"
checks["valid_appellantGivenNames_not_null"] = "(appellantGivenNames IS NOT NULL)"
checks["valid_appellantNameForDisplay_not_null"] = "(appellantNameForDisplay IS NOT NULL)"

checks["valid_appellantDateOfBirth_format"] = (
    "(appellantDateOfBirth RLIKE r'^\\d{4}-\\d{2}-\\d{2}$')"
)
checks["valid_caseNameHmctsInternal_not_null"] = "(caseNameHmctsInternal IS NOT NULL)"
checks["valid_hmctsCaseNameInternal_not_null"] = "(hmctsCaseNameInternal IS NOT NULL)"

# ##############################
# # ARIADM-771 (AppealType - legalRepDetails)
# ##############################

checks["valid_legalrepEmail_not_null"] = "((dv_representation = 'LR' AND legalRepEmail RLIKE r'^([a-zA-Z0-9_\\-\\.]+)@([a-zA-Z0-9_\\-\\.]+)\\.([a-zA-Z]{2,5})$') OR (dv_representation != 'LR' AND legalRepEmail IS NULL))"

# ##############################
# # ARIADM-758 (appellantDetails)
# ##############################

checks["valid_isAppellantMinor_yes_no"] = (
    "(isAppellantMinor IS NOT NULL AND isAppellantMinor IN ('Yes', 'No'))"
)
checks["valid_deportationOrderOptions_yes_no"] = (
    "(deportationOrderOptions IS NULL OR deportationOrderOptions IN ('Yes', 'No'))"
)
checks["valid_appellantInUk_yes_no"] = (
    "(appellantInUk IS  NULL OR appellantInUk IN ('Yes', 'No'))"
)
checks["valid_appealOutOfCountry_yes_no"] = (
    "(appealOutOfCountry IS  NULL OR appealOutOfCountry IN ('Yes', 'No'))"
)

# ##############################
# # ARIADM-769 (legalRepDetails - Address logic)CaseRepAddress5
# ##############################

checks["valid_legalRepHasAddress_yes_no"] = ( #Omit non-LR records. NLE data will fail all expectations (55) as address are non-UK
  "((dv_representation = 'LR' AND legalRepHasAddress IS NOT NULL AND legalRepHasAddress = 'Yes') OR (dv_representation != 'LR' AND legalRepHasAddress IS NULL))"
)
checks["valid_legalRepHasAddressUK"]   = ( #Omit non-LR records. All fields are null, hence all expectations will fail. (55)
  "(((dv_representation = 'LR' AND oocAddressLine1 IS NOT NULL AND LEN(oocAddressLine1) < 151) OR (dv_representation != 'LR' AND oocAddressLine1 IS NULL)"
  "OR ((dv_representation = 'LR' AND LEN(oocAddressLine2) < 51) OR (dv_representation != 'LR' AND oocAddressLine2 IS NULL))" 
  "OR ((dv_representation = 'LR' AND LEN(oocAddressLine3) < 51) OR (dv_representation != 'LR' AND oocAddressLine3 IS NULL))"
  "OR ((dv_representation = 'LR' AND LEN(oocAddressLine4) < 51) OR (dv_representation != 'LR' AND oocAddressLine4 IS NULL))"
  "OR ((dv_representation = 'LR' AND LEN(CaseRep_Address5) < 51) OR (dv_representation != 'LR' AND CaseRep_Address5 IS NULL))"
  "OR ((dv_representation = 'LR' AND LEN(CaseRep_Postcode) < 15) OR (dv_representation != 'LR' AND CaseRep_Postcode IS NULL))))"
)   
checks["valid_oocAddressLine1"] = ( #Omit non-LR records. NLE data will fail all expectations (55) as fields are null
  "((dv_representation = 'LR' AND oocAddressLine1 IS NOT NULL) OR (dv_representation != 'LR' AND oocAddressLine1 IS NULL))"
)
checks["valid_oocAddressLine2"] = ( #Omit non-LR records. NLE data will fail all expectations (55) as fields are null
  "((dv_representation = 'LR' AND oocAddressLine2 IS NOT NULL) OR (dv_representation != 'LR' AND oocAddressLine2 IS NULL))"
)
checks["valid_oocAddressLine3"] = ( #Omit non-LR records. NLE data will fail all expectations (55) as fields are null
  "((dv_representation = 'LR' AND oocAddressLine3 IS NOT NULL) OR (dv_representation != 'LR' AND oocAddressLine3 IS NULL))"
)
checks["valid_oocAddressLine4"] = ( #Omit non-LR records. NLE data will fail all expectations (55) as fields are null
  "((dv_representation = 'LR' AND oocAddressLine4 IS NOT NULL) OR (dv_representation != 'LR' AND oocAddressLine4 IS NULL))"
)
checks["valid_oocrCountryGovUkAdminJ"] = ( #Omit non-LR records. NLE data will fail all expectations (55) as fields are null
  "((dv_representation = 'LR' AND CaseRep_Address5 IS NOT NULL AND valid_countryGovUkOocAdminJ IS NOT NULL ) OR (dv_representation != 'LR' AND CaseRep_Address5 IS NULL))"
  )

# ##############################
# # ARIADM-766 (appellantStateless)
# ##############################
checks["valid_appellantStateless_values"] = ("(appellantStateless IN ('isStateless', 'hasNationality'))")

checks["valid_appellantNationalitiesDescription_not_null"] = "(appellantNationalitiesDescription IS NOT NULL)"

checks["valid_appellantNationalities_not_null"] = ("(appellantNationalities IS NOT NULL)")

##############################
# ARIADM-760 (appellantDetails) - appellantHasFixedAddress and appellantAddress
##############################

# Only include if CategoryIdList contains 37; check for 'Yes' 
checks["valid_appellantHasFixedAddress_yes_no_if_cat37"] = (
    "( (array_contains(valid_categoryIdList, 37) AND appellantHasFixedAddress IS NOT NULL AND appellantHasFixedAddress IN ('Yes')) OR (appellantHasFixedAddress IS NULL) )"
)

# Only include if array_contains(valid_categoryIdList, 37)
checks["valid_appellantAddress_AddressLine1_mandatory_and_length"] = (
    "( (array_contains(valid_categoryIdList, 37) AND appellantAddress.AddressLine1 IS NOT NULL AND LENGTH(appellantAddress.AddressLine1) <= 150) OR (appellantAddress.AddressLine1 IS NULL) )"
)
checks["valid_appellantAddress_AddressLine2_length"] = (
    "( (array_contains(valid_categoryIdList, 37) AND (appellantAddress.AddressLine2 IS NULL OR LENGTH(appellantAddress.AddressLine2) <= 50)) OR ( appellantAddress.AddressLine2 IS NULL))"
)
checks["valid_appellantAddress_AddressLine3_length"] = (
    "( (array_contains(valid_categoryIdList, 37) AND (appellantAddress.AddressLine3 IS NULL OR LENGTH(appellantAddress.AddressLine3) <= 50)) OR (appellantAddress.AddressLine3 IS NULL) )"
)
checks["valid_appellantAddress_PostTown_length"] = (
    "( (array_contains(valid_categoryIdList, 37) AND (appellantAddress.PostTown IS NULL OR LENGTH(appellantAddress.PostTown) <= 50)) OR (appellantAddress.PostTown IS NULL) )"
)
checks["valid_appellantAddress_County_length"] = (
    "( (array_contains(valid_categoryIdList, 37) AND (appellantAddress.County IS NULL OR LENGTH(appellantAddress.County) <= 50)) OR (appellantAddress.County IS NULL) )"
)
checks["valid_appellantAddress_PostCode_length"] = (
    "( (array_contains(valid_categoryIdList, 37) AND (appellantAddress.PostCode IS NULL OR LENGTH(appellantAddress.PostCode) <= 14)) OR (appellantAddress.PostCode IS NULL) )"
)
checks["valid_appellantAddress_Country_length"] = (
    "( (array_contains(valid_categoryIdList, 37) AND (appellantAddress.Country IS NULL OR LENGTH(appellantAddress.Country) <= 50)) OR (appellantAddress.Country IS NULL) )"
)


# #############################
# # ARIADM-709 (flagsLabels)
# #############################

checks["valid_journeyType_aip_orNull"] = "((dv_representation = 'AIP' AND journeyType = 'aip') OR (dv_representation != 'AIP' AND journeyType IS NULL))"

# #############################
# # ARIADM-710 (flagsLabels)
# #############################

checks["valid_isAriaMigratedFeeExemption_yes_no"] = "((dv_CCDAppealType = 'DA' AND isAriaMigratedFeeExemption = 'Yes') OR (dv_CCDAppealType != 'DA' AND isAriaMigratedFeeExemption = 'No'))"

# ##############################
# # ARIADM-712 (flagsLabel)- caseFlags
# ##############################
checks["valid_caseFlags_name_in_list"] = """
(
  caseFlags.details IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(caseFlags.details, x -> x.value.name),
    caseFlags.details[0].value.name
  )
)
"""
checks["valid_caseFlags_pathId_in_list"] = """
(
  caseFlags.details IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(caseFlags.details, x -> x.value.path[0].id),
    caseFlags.details[0].value.path[0].id
  )
)
"""
checks["valid_caseFlags_flagCode_in_list"] = """
(
  caseFlags.details IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(caseFlags.details, x -> x.value.flagCode),
    caseFlags.details[0].value.flagCode
  )
)
"""
checks["valid_caseFlags_flagComment_in_list"] = """
(
  caseFlags.details IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(caseFlags.details, x -> x.value.flagComment),
    caseFlags.details[0].value.flagComment
  )
)
"""
checks["valid_caseFlags_hearingRelevant_in_list"] = """
(
  caseFlags.details IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(caseFlags.details, x -> x.value.hearingRelevant),
    caseFlags.details[0].value.hearingRelevant
  )
)
"""

# ##############################
# # ARIADM-712 (flagsLabel)- appellantLevelFlags
# ##############################

checks["valid_appellantLevelFlags_name_in_details"] = """
(
  appellantLevelFlags.details[0].value.name IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(appellantLevelFlags.details, x -> x.value.name),
    appellantLevelFlags.details[0].value.name
  )
)
"""

checks["valid_appellantLevelFlags_path_id_in_details"] = """
(
  appellantLevelFlags.details[0].value.path[0].id IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(appellantLevelFlags.details, x -> x.value.path[0].id),
    appellantLevelFlags.details[0].value.path[0].id
  )
)
"""

checks["valid_appellantLevelFlags_flagCode_in_details"] = """
(
  appellantLevelFlags.details[0].value.flagCode IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(appellantLevelFlags.details, x -> x.value.flagCode),
    appellantLevelFlags.details[0].value.flagCode
  )
)
"""

checks["valid_appellantLevelFlags_flagComment_in_details"] = """
(
  appellantLevelFlags.details[0].value.flagComment IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(appellantLevelFlags.details, x -> x.value.flagComment),
    appellantLevelFlags.details[0].value.flagComment
  )
)
"""

checks["valid_appellantLevelFlags_hearingRelevant_in_details"] = """
(
  appellantLevelFlags.details[0].value.hearingRelevant IS NULL OR
  ARRAY_CONTAINS(
    TRANSFORM(appellantLevelFlags.details, x -> x.value.hearingRelevant),
    appellantLevelFlags.details[0].value.hearingRelevant
  )
)
"""

# ##############################
# # ARIADM-780 (PartyID)
# ##############################

checks["valid_appellantPartyId_not_null"] = (
  "(appellantPartyId IS NOT NULL)"
  )
checks["valid_legalRepIndividualPartyId_not_null"] = ( #If appellantsRep = no then appellantsRep = LR
  "((legalRepIndividualPartyId IS NOT NULL AND appellantsRepresentation = 'No') OR (legalRepIndividualPartyId IS NULL AND appellantsRepresentation = 'Yes'))"
  )
checks["validlegalRepOrganisationPartyId_not_null"] = ( #If appellantsRep = no then appellantsRep = LR
  "((legalRepOrganisationPartyId IS NOT NULL AND appellantsRepresentation = 'No') OR (legalRepOrganisationPartyId IS NULL AND appellantsRepresentation = 'Yes'))"
  )
checks["valid_sponsorPartyId_not_null"] = (
  "((sponsorPartyId IS NOT NULL AND hasSponsor = 'Yes') OR (sponsorPartyId IS NULL and hasSponsor = 'No') OR (sponsorPartyID IS NULL and hasSponsor IS NULL))"
  )

# ##############################
# # ARIADM-783 (payment)
# ##############################
checks["valid_feeAmountGbp"] = ( # fee amount is not null and is an int
    "(dv_CCDAppealType IN ('EA','EU','HU','PA') AND (feeAmountGbp IS NOT NULL) AND (TRY_CAST(feeAmountGbp AS INT) IS NOT NULL)) OR (dv_CCDAppealType NOT IN ('EA','EU','HU','PA') AND (feeAmountGbp IS NULL))"
    )

# checks["valid_feeAmountGbp"] = ( # fee amount is not null and is an int
#     "(feeAmountGbp IS NOT NULL AND TRY_CAST(feeAmountGbp AS INT) IS NOT NULL)"
# )

checks["valid_feeDescription"] = ( #feeDescription is not null
    "(dv_CCDAppealType IN ('EA','EU','HU','PA') AND (feeDescription IS NOT NULL)) OR (dv_CCDAppealType NOT IN ('EA','EU','HU','PA') AND (feeDescription IS NULL))"
)

# checks["valid_feeDescription"] = ( #feeDescription is not null
#     "(feeDescription IS NOT NULL)"
# )

checks["valid_feeWithHearing"] = ( # feeWithHearing is not null and is an int
    "(dv_CCDAppealType IN ('EA','EU','HU','PA') AND (feeWithHearing IS NOT NULL) AND (TRY_CAST(feeWithHearing AS INT) IS NOT NULL)) OR (dv_CCDAppealType NOT IN ('EA','EU','HU','PA') AND (feeWithHearing IS NULL))"
)

# checks["valid_feeWithHearing"] = ( # feeWithHearing is not null and is an int
#     "(feeWithHearing IS NOT NULL AND TRY_CAST(feeWithHearing AS INT) IS NOT NULL)"
# )

checks["valid_feeWithoutHearing"] = (# feeWithoutHearing is not null and is an int
    "(dv_CCDAppealType IN ('EA','EU','HU','PA') AND (feeWithoutHearing IS NOT NULL) AND (TRY_CAST(feeWithoutHearing AS INT) IS NOT NULL)) OR (dv_CCDAppealType NOT IN ('EA','EU','HU','PA') AND (feeWithoutHearing IS NULL))"
)

# checks["valid_feeWithoutHearing"] = (# feeWithoutHearing is not null and is an int
#     "(feeWithoutHearing IS NOT NULL AND TRY_CAST(feeWithoutHearing AS INT) IS NOT NULL)"
# )

checks["valid_paymentDescription"] = ( # paymentDescription is not null
    "(dv_CCDAppealType IN ('EA','EU','HU','PA') AND (paymentDescription IS NOT NULL)) OR (dv_CCDAppealType NOT IN ('EA','EU','HU','PA') AND (paymentDescription IS NULL))"
)

# checks["valid_paymentDescription"] = ( # paymentDescription is not null
#     "(paymentDescription IS NOT NULL)"
# )

checks["valid_paymentStatus"] = ( # paymentStatus is not null
  "(dv_CCDAppealType IN ('EA','EU','HU','PA') AND (paymentStatus IS NOT NULL)) OR (dv_CCDAppealType NOT IN ('EA','EU','HU','PA') AND (paymentStatus IS NULL))"
)

# checks["valid_paymentStatus"] = ( # paymentStatus is not null
#   "(paymentStatus IS NOT NULL)"
# )

checks["valid_feeVersion"] = ( # feeVersion is not null
  "(dv_CCDAppealType IN ('EA','EU','HU','PA') AND (feeVersion IS NOT NULL)) OR (dv_CCDAppealType NOT IN ('EA','EU','HU','PA') AND (feeVersion IS NULL))"
)

# checks["valid_feeVersion"] = ( # feeVersion is not null
#   "(feeVersion IS NOT NULL)"
# )

checks["valid_feePaymentAppealType"] = ( # feePaymentAppealType is not null
  "(dv_CCDAppealType IN ('EA','EU','HU','PA') AND (feePaymentAppealType IS NOT NULL)) OR (dv_CCDAppealType NOT IN ('EA','EU','HU','PA') AND (feePaymentAppealType IS NULL))"
)

# checks["valid_feePaymentAppealType"] = ( # feePaymentAppealType is not null
#   "(feePaymentAppealType IS NOT NULL)"
# )

# ##############################
# # ARIADM-785 (remissionTypes)
# ############################## 

checks["valid_remissionType_in_list"] = (
    "(dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionType IS NOT NULL AND remissionType IN ('noRemission', 'hoWaiverRemission', 'helpWithFees', 'exceptionalCircumstancesRemission')) OR (dv_CCDAppealType NOT IN ('EA', 'EU', 'HU', 'PA') AND remissionType IS NULL)"
)

checks["valid_remissionClaim_in_list"] = (
    "(dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionClaim IN ('asylumSupport', 'legalAid', 'section17', 'section20', 'homeOfficeWaiver')) OR (dv_CCDAppealType NOT IN ('EA', 'EU', 'HU', 'PA') AND remissionClaim IS NULL)"
) 

checks["valid_feeRemissionType_not_null"] = (
    "(dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND feeRemissionType IS NOT NULL)"
) 

# ##############################
# # ARIADM-786 (remissionTypes)
# ##############################

checks["valid_exceptionalCircumstances_not_null"] = (
    "((dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND ReasonDescription = 'Oral Hearing Direction' AND remissionType = 'exceptionalCircumstancesremission' AND exceptionalCircumstances = 'This is a migrated ARIA case. The remission reason was Oral Hearing Direction. Please see the documents for further information.') OR (dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND ReasonDescription = 'Other' AND remissionType = 'exceptionalCircumstancesremission' AND exceptionalCircumstances = 'This is a migrated ARIA case. The remission reason was Oral Hearing Direction. Please see the documents for further information.') OR (dv_CCDAppealType NOT IN ('EA', 'EU', 'HU', 'PA') AND remissionType = 'noRemission' AND exceptionalCircumstances IS NULL) OR (dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionType != 'exceptionalCircumstancesremission' AND exceptionalCircumstances IS NULL))"
) 

checks["valid_helpWithFeesReferenceNumber_not_null"] = (
    "((dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionType = 'helpWithFees' AND helpWithFeesReferenceNumber IS NOT NULL) OR (dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionType != 'helpWithFees' AND helpWithFeesReferenceNumber IS NULL) OR (dv_CCDAppealType NOT IN ('EA', 'EU', 'HU', 'PA') AND remissionType = 'noRemission' AND helpWithFeesReferenceNumber IS NULL))"
) 

checks["valid_legalAidAccountNumber_not_null"] = (
      "((dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionType = 'hoWaiverRemission' AND remissionClaim = 'legalAid' AND feeRemissionType = 'Legal Aid' AND legalAidAccountNumber IS NOT NULL) OR (dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionType = 'noRemission' AND remissionClaim IS NULL AND feeRemissionType IS NULL AND legalAidAccountNumber IS NULL) OR (dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionType = 'hoWaiverRemission' AND remissionClaim = 'asylumSupport' AND feeRemissionType = 'Asylum Support' AND asylumSupportReference IS NOT NULL))"
) 

checks["valid_asylumSupportReference_not_null"] = (
      "((dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionType = 'hoWaiverRemission' AND remissionClaim = 'asylumSupport' AND feeRemissionType = 'Asylum Support' AND asylumSupportReference IS NOT NULL) OR (dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionType = 'noRemission' AND remissionClaim IS NULL AND feeRemissionType IS NULL AND asylumSupportReference IS NULL) OR (dv_CCDAppealType IN ('EA', 'EU', 'HU', 'PA') AND remissionType = 'hoWaiverRemission' AND remissionClaim = 'legalAid' AND feeRemissionType = 'Legal Aid' AND legalAidAccountNumber IS NOT NULL))"
) 

##############################
# ARIADM-773 (SponsorDetails)
##############################

# IF CategoryId IN [38] = Include; ELSE OMIT
# IF SponsorName IS NOT NULL = Yes; ELSE No
checks["valid_hasSponsor_yes_no"] = (
    "((array_contains(valid_categoryIdList, 38) AND Sponsor_Name IS NOT NULL AND hasSponsor = 'Yes') "
    "OR (array_contains(valid_categoryIdList, 38) AND Sponsor_Name IS NULL AND hasSponsor = 'No') "
    "OR ( hasSponsor IS NULL))"
)

# IF CategoryId IN [38] = Include; ELSE OMIT
checks["valid_sponsorGivenNames_not_null"] = (
    "( (array_contains(valid_categoryIdList, 38) AND sponsorGivenNames IS NOT NULL) OR (sponsorGivenNames IS NULL) )"
)

# IF CategoryId IN [38] = Include; ELSE OMIT
checks["valid_sponsorFamilyName_not_null"] = (
    "(((array_contains(valid_categoryIdList, 38) AND sponsorFamilyName IS NOT NULL) OR (sponsorFamilyName IS NULL)))"
)

# IF CategoryId IN [38] = Include; ELSE OMIT
# IF Sponsor_Authorisation IS 1 = Yes; ELSE No 
checks["valid_sponsorAuthorisation_yes_no"] = (
"((array_contains(valid_categoryIdList, 38) AND (Sponsor_Authorisation = 1 ) AND (sponsorAuthorisation IN ('Yes', 'No'))) OR (sponsorAuthorisation IS NULL))"
)

##############################
# ARIADM-776 (SponsorDetails)
##############################
checks["valid_sponsorAddress_not_null"] = (
    "(((array_contains(valid_categoryIdList, 38) AND hasSponsor = 'Yes' AND sponsorAddress IS NOT NULL) OR (NOT array_contains(valid_categoryIdList, 38) AND hasSponsor = 'No' AND sponsorAddress IS NULL)))"
)
##############################
# ARIADM-778 (SponsorDetails)
##############################
checks["valid_sponsorEmailAdminJ"] = (
    "(((array_contains(valid_categoryIdList, 38) AND hasSponsor = 'Yes' AND sponsorEmailAdminJ IS NOT NULL) "
    "OR (NOT array_contains(valid_categoryIdList, 38) OR hasSponsor = 'No') AND sponsorEmailAdminJ IS NULL))"
)

checks["valid_sponsorMobileNumberAdminJ"] = (
    "(((array_contains(valid_categoryIdList, 38) AND hasSponsor = 'Yes' AND sponsorMobileNumberAdminJ IS NOT NULL) "
    "OR (NOT array_contains(valid_categoryIdList, 38) OR hasSponsor = 'No') AND sponsorMobileNumberAdminJ IS NULL))"
)
# ##############################
# ARIADM-760 (appellantDetails)
# ARIADM-762 (appellantDetails)
# ##############################
checks["valid_oocAppealAdminJ_values"] = (
    "( ( (array_contains(valid_categoryIdList, 38) OR MainRespondentId = 4) "
    "AND oocAppealAdminJ IN ('entryClearanceDecision', 'leaveUk', 'none') ) "
    "OR (oocAppealAdminJ IS NULL) )"
)

# Only IF CategoryId IN [38] = Include; ELSE null
checks["valid_appellantHasFixedAddressAdminJ"] = (
    "( (array_contains(valid_categoryIdList, 38) AND appellantHasFixedAddressAdminJ IN ('Yes', 'No')) "
    "OR (NOT array_contains(valid_categoryIdList, 38) AND appellantHasFixedAddressAdminJ IS NULL) )"
)

# addressLine1AdminJ: IS NOT NULL when array_contains(valid_categoryIdList, 38) AND at least one of the coalesce fields is not null; ELSE can be NULL
checks["valid_addressLine1AdminJ"] = (
    "( (array_contains(valid_categoryIdList, 38) AND "
    "(Appellant_Address1 IS NOT NULL OR Appellant_Address2 IS NOT NULL OR Appellant_Address3 IS NOT NULL OR Appellant_Address4 IS NOT NULL OR Appellant_Address5 IS NOT NULL OR Appellant_Postcode IS NOT NULL) "
    "AND addressLine1AdminJ IS NOT NULL) "
    "OR (addressLine1AdminJ IS NULL) )"
)

# addressLine2AdminJ: IS NOT NULL when array_contains(valid_categoryIdList, 38) AND dv_representation = 'LR' AND at least one of the coalesce fields is not null; ELSE can be NULL
checks["valid_addressLine2AdminJ"] = (
    "( (array_contains(valid_categoryIdList, 38) AND dv_representation = 'LR' AND "
    "(Appellant_Address2 IS NOT NULL OR Appellant_Address3 IS NOT NULL OR Appellant_Address4 IS NOT NULL OR Appellant_Address5 IS NOT NULL OR Appellant_Postcode IS NOT NULL) "
    "AND addressLine2AdminJ IS NOT NULL) "
    "OR (addressLine2AdminJ IS NULL) )"
)

# addressLine3AdminJ: IS NOT NULL when array_contains(valid_categoryIdList, 38) AND at least one of the coalesce fields is not null; ELSE can be NULL
checks["valid_addressLine3AdminJ"] = (
    "( (array_contains(valid_categoryIdList, 38) AND "
    "(Appellant_Address3 IS NOT NULL OR Appellant_Address4 IS NOT NULL) "
    "AND addressLine3AdminJ IS NOT NULL) "
    "OR ( addressLine3AdminJ IS NULL) )"
)

# addressLine4AdminJ: IS NOT NULL when array_contains(valid_categoryIdList, 38) AND at least one of the coalesce fields is not null; ELSE can be NULL
checks["valid_addressLine4AdminJ"] = (
    "( (array_contains(valid_categoryIdList, 38) AND "
    "(Appellant_Address5 IS NOT NULL OR Appellant_Postcode IS NOT NULL) "
    "AND addressLine4AdminJ IS NOT NULL) "
    "OR ( addressLine4AdminJ IS NULL) )"
)


# countryGovUkOocAdminJ: IS NOT NULL when array_contains(valid_categoryIdList, 38); ELSE can be NULL
checks["valid_countryGovUkOocAdminJ"] = ("(((array_contains(valid_categoryIdList, 38)) AND (countryGovUkOocAdminJ IS NOT NULL) AND (countryGovUkOocAdminJ IN ('AF', 'AX', 'AL', 'DZ', 'AD', 'AO', 'AI', 'AG', 'AR', 'AM', 'AW', 'AC', 'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ', 'BM', 'BT', 'BO', 'BQ', 'BA', 'BW', 'BR', 'IO', 'VG', 'BN', 'BG', 'BF', 'BI', 'KH', 'CM', 'CA', 'IC', 'CV', 'KY', 'CF', 'EA', 'TD', 'CL', 'CN', 'CX', 'CO', 'KM', 'CD', 'CG', 'CK', 'CR', 'HR', 'CU', 'CW', 'CY', 'CZ', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'EE', 'ET', 'FK', 'FO', 'FJ', 'FI', 'FR', 'GF', 'PF', 'TF', 'GA', 'GM', 'GE', 'DE', 'GH', 'GI', 'GR', 'GL', 'GD', 'GP', 'GT', 'GN', 'GW', 'GY', 'HT', 'HN', 'HK', 'HU', 'IS', 'IN', 'ID', 'IR', 'IQ', 'IE', 'IL', 'IT', 'CI', 'JM', 'JP', 'JO', 'KZ', 'KE', 'KI', 'KO', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI', 'LT', 'LU', 'MO', 'MK', 'MG', 'YT', 'MW', 'MY', 'MV', 'ML', 'MT', 'MQ', 'MR', 'MU', 'MX', 'MD', 'MN', 'ME', 'MS', 'MA', 'MZ', 'MM', 'NA', 'NR', 'NP', 'NL', 'NC', 'NZ', 'NI', 'NE', 'NG', 'NU', 'KP', 'NO', 'OM', 'PK', 'PW', 'PA', 'PG', 'PY', 'PE', 'PH', 'PN', 'PL', 'PT', 'PR', 'QA', 'RE', 'RO', 'RU', 'RW', 'SM', 'ST', 'SA', 'SN', 'RS', 'SC', 'SL', 'SG', 'SK', 'SI', 'SB', 'ZA', 'KR', 'SS', 'ES', 'LK', 'BQ', 'SH', 'KN', 'LC', 'MF', 'VC', 'SD', 'SR', 'SZ', 'SE', 'CH', 'SY', 'TW', 'TJ', 'TZ', 'TH', 'TL', 'TG', 'TK', 'TO', 'TT', 'TN', 'TR', 'TM', 'TC', 'TV', 'UG', 'UA', 'AE', 'GB', 'UY', 'US', 'UZ', 'VU', 'VA', 'VE', 'VN', 'WF', 'EH', 'WS', 'YE', 'ZM', 'ZW'))) OR (countryGovUkOocAdminJ IS NULL))")
##############################
# AARIADM-764 (appellantDetails)
##############################
# ^([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})$ 
checks["valid_internalAppellantEmail_format"] = (
    "( internalAppellantEmail RLIKE r'^([a-zA-Z0-9_\\-\\.]+)@([a-zA-Z0-9_\\-\\.]+)\\.([a-zA-Z]{2,5})$' OR internalAppellantEmail IS NULL)"
)

checks["valid_email_format"] = (
    "(email RLIKE r'^([a-zA-Z0-9_\\-\\.]+)@([a-zA-Z0-9_\\-\\.]+)\\.([a-zA-Z]{2,5})$' OR email IS NULL)"
)

checks["valid_internalAppellantMobileNumber"] = (
    "(internalAppellantMobileNumber RLIKE r'^(?=(?:\\D*\\d){7,15}\\D*$)\\+?(\\d[\\d-. ]+)?(\\([\\d-. ]+\\))?[\\d-. ]*\\d$' OR internalAppellantMobileNumber IS NULL)"
)

# ^(?=(?:\D*\d){7,15}\D*$)\+?(\d[\d-. ]+)?(\([\d-. ]+\))?[\d-. ]*\d$
checks["valid_mobileNumber"] = (
    "(mobileNumber RLIKE r'^(?=(?:\\D*\\d){7,15}\\D*$)\\+?(\\d[\\d-. ]+)?(\\([\\d-. ]+\\))?[\\d-. ]*\\d$' OR mobileNumber IS NULL)"
)
##############################
# ARIADM-778 (General)
##############################
checks["isServiceRequestTabVisibleConsideringRemissions_yes_no"] = (
    "(isServiceRequestTabVisibleConsideringRemissions IS NOT NULL AND isServiceRequestTabVisibleConsideringRemissions IN ('Yes', 'No'))"
)

checks["lu_applicationChangeDesignatedHearingCentre_fixed_list"] = (
  "(lu_applicationChangeDesignatedHearingCentre IS NOT NULL AND lu_applicationChangeDesignatedHearingCentre IN ('taylorHouse', 'newport', 'newcastle', 'manchester', 'hattonCross' ,'glasgow' ,'bradford' ,'birmingham', 'arnhemHouse', 'crownHouse', 'harmondsworth', 'yarlsWood', 'remoteHearing', 'decisionWithoutHearing'))"
)
#########################################
# ARIADM-788 and ARIADM-792 (homeOffice)
#########################################
checks["valid_homeOfficeDecisionDate_format"] = (
    "((array_contains(valid_categoryIdList, 37) AND homeOfficeDecisionDate IS NOT NULL AND homeOfficeDecisionDate RLIKE r'^\\d{4}-\\d{2}-\\d{2}$') OR (homeOfficeDecisionDate IS NULL))"
)

checks["valid_decisionLetterReceivedDate_format"] = (
    "( (decisionLetterReceivedDate IS NOT NULL AND decisionLetterReceivedDate RLIKE r'^\\d{4}-\\d{2}-\\d{2}$' "
    "AND array_contains(valid_categoryIdList, 38) "
    "AND (lu_HORef LIKE '%GWF%' OR HORef LIKE '%GWF%' OR FCONumber LIKE '%GWF%')) "
    "OR (decisionLetterReceivedDate IS NULL) )"
)

checks["valid_dateEntryClearanceDecision_format"] = (
    "( (array_contains(valid_categoryIdList, 38) "
    "AND (lu_HORef LIKE '%GWF%' OR HORef LIKE '%GWF%' OR FCONumber LIKE '%GWF%') "
    "AND dateEntryClearanceDecision IS NOT NULL "
    "AND dateEntryClearanceDecision RLIKE r'^\\d{4}-\\d{2}-\\d{2}$') "
    "OR (dateEntryClearanceDecision IS NULL) )"
)

checks["valid_homeOfficeReferenceNumber_not_null"] = (
    "( (array_contains(valid_categoryIdList, 38) "
    "AND (lu_HORef LIKE '%GWF%' OR HORef LIKE '%GWF%' OR FCONumber LIKE '%GWF%') "
    "AND homeOfficeReferenceNumber IS NOT NULL "
    "AND homeOfficeReferenceNumber RLIKE r'^\\d{4}-\\d{2}-\\d{2}$') "
    "OR (homeOfficeReferenceNumber IS NULL) )"
)

checks["valid_gwfReferenceNumber_not_null"] = (
    "( (array_contains(valid_categoryIdList, 38) "
    "AND (lu_HORef LIKE '%GWF%' OR HORef LIKE '%GWF%' OR FCONumber LIKE '%GWF%') "
    "AND gwfReferenceNumber IS NOT NULL "
    "AND gwfReferenceNumber RLIKE r'^\\d{4}-\\d{2}-\\d{2}$') "
    "OR (gwfReferenceNumber IS NULL) )"
)

#########################################
# ARIADM-799 (Documents)
#########################################

checks["valid_uploadTheAppealFormDocs"] = (
  "(uploadTheAppealFormDocs IS NOT NULL)"
)

checks["valid_caseNotes"] = (
  "(caseNotes IS NOT NULL)"
)

checks["valid_tribunalDocuments"] = (
  "(tribunalDocuments IS NOT NULL)"
)

checks["valid_legalRepresentativeDocuments"] = (
  "(legalRepresentativeDocuments IS NOT NULL)"
)

In [0]:
# # Define a dictionary to hold data quality checks
# checks = {}

# # ##############################
# # # ARIADM-669 (appealType)
# # ##############################
# checks["valid_appealReferenceNumber_not_null"] = "(appealReferenceNumber IS NOT NULL)"


In [0]:
# checks["valid_hasSponsor_yes_no"] = (
#     "((array_contains(valid_categoryIdList, 38) AND Sponsor_Name IS NOT NULL AND hasSponsor = 'Yes') "
#     "OR (array_contains(valid_categoryIdList, 38) AND Sponsor_Name IS NULL AND hasSponsor = 'No') "
#     "OR ( hasSponsor IS NULL))"
# )

In [0]:
# Combine all checks into a single string with AND conditions
# Create a validation expression to quarantine records
dq_rules = "({0})".format(" AND ".join(checks.values()))

In [0]:
# print(dq_rules)

In [0]:
# This is a placeholder, like a test
# spark.sql(f"""
# SELECT 
#   CaseNo,
#     appealReferenceNumber 
#  FROM ariadm_active_appeals.stg_main_payment_pending_validation
#  WHERE {dq_rules}
# """).display()



In [0]:
import dlt
from pyspark.sql.functions import col, lit, expr

@dlt.table(
    name="stg_main_payment_pending_validation",
    comment="DLT table running mainPaymentPending to generate a JSON_Content column for CCD validation. Applies DLT expectations on CCD, adding is_valid to flag validation results.",
    path=f"{audit_path}/stg_main_payment_pending_validation"
)
@dlt.expect_all(checks)
def stg_main_payment_pending_validation():
    try:
        silver_m1 = dlt.read("silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()
        bronze_appealtype_lookup_df = dlt.read("bronze_appealtype").distinct()
        bronze_hearing_centres_lookup_df = dlt.read("bronze_hearing_centres").distinct()
        # stg_representation = dlt.read("stg_representation").select(col("Representation").alias("valid_representation"))
        silver_m2 = dlt.read("silver_caseapplicant_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_m3 = dlt.read("silver_status_detail").filter(col("dv_targetState") == lit(AppealState))
        # silver_m4 = dlt.read("silver_transaction_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_m5 = dlt.read("silver_link_detail").filter(col("dv_targetState") == lit(AppealState))
        # silver_m6 = dlt.read("silver_adjudicator_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_c = dlt.read("silver_appealcategory_detail").filter(col("dv_targetState") == lit(AppealState))
        # silver_m8 = dlt.read("silver_documentsreceived_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_h = dlt.read("silver_history_detail").filter(col("dv_targetState") == lit(AppealState))
        bronze_countries_postal_lookup_df = dlt.read("bronze_countries_postal").distinct()
        bronze_remissions_lookup_df = dlt.read("bronze_remissions").distinct()
        bronze_countryFromAddress = dlt.read("bronze_countries_countryFromAddress")
        bronze_HORef_cleansing = dlt.read("bronze_HORef_cleansing")
        bronze_hearing_centres = dlt.read("bronze_hearing_centres")
        bronze_derive_hearing_centres = dlt.read("bronze_derive_hearing_centres")
    except:
        silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()
        bronze_appealtype_lookup_df = spark.table("ariadm_active_appeals.bronze_appealtype").distinct()
        bronze_hearing_centres_lookup_df = spark.table("ariadm_active_appeals.bronze_hearing_centres").distinct()
        # stg_representation = spark.table("ariadm_active_appeals.stg_representation").select(col("Representation").alias("valid_representation"))
        silver_m2 = spark.table("ariadm_active_appeals.silver_caseapplicant_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_m3 = spark.table("ariadm_active_appeals.silver_status_detail").filter(col("dv_targetState") == lit(AppealState))
        # silver_m4 = spark.table("ariadm_active_appeals.silver_transaction_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_m5 = spark.table("ariadm_active_appeals.silver_link_detail").filter(col("dv_targetState") == lit(AppealState))
        # silver_m6 = spark.table("ariadm_active_appeals.silver_adjudicator_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_c = spark.table("ariadm_active_appeals.silver_appealcategory_detail").filter(col("dv_targetState") == lit(AppealState))
        # silver_m8 = spark.table("ariadm_active_appeals.silver_documentsreceived_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_h = spark.table("ariadm_active_appeals.silver_history_detail").filter(col("dv_targetState") == lit(AppealState))
        bronze_countries_postal_lookup_df = spark.table("ariadm_active_appeals.bronze_countries_postal").distinct() 
        bronze_remissions_lookup_df = spark.table("ariadm_active_appeals.bronze_remissions").distinct()
        bronze_countryFromAddress = spark.table("ariadm_active_appeals.bronze_countries_countryFromAddress")
        bronze_HORef_cleansing = spark.table("ariadm_active_appeals.bronze_HORef_cleansing")
        bronze_hearing_centres = spark.table("ariadm_active_appeals.bronze_hearing_centres")
        bronze_derive_hearing_centres = spark.table("ariadm_active_appeals.bronze_derive_hearing_centres")
        valid_HORef_cleansing = bronze_HORef_cleansing.select(
            col("CaseNo"),
            coalesce(col("HORef"), col("FCONumber")).alias("lu_HORef")
        )

 
    df_final,df_audit = mainPaymentPending(silver_m1, silver_m2, silver_m3, silver_c,silver_h, bronze_remissions_lookup_df, bronze_countryFromAddress, bronze_HORef_cleansing,bronze_hearing_centres,bronze_derive_hearing_centres)

 
    valid_representation = silver_m1.select(col("CaseNo"), col("dv_representation"),col("dv_CCDAppealType"),col("CaseRep_Address5"), col("CaseRep_Postcode"),col("MainRespondentId"), col("lu_appealType"), col("HORef"),col("Sponsor_Authorisation"),col("Sponsor_Name")) 
    valid_appealant_address = silver_m2.select(col("CaseNo"), col("Appellant_Address1"), col("Appellant_Address2"),col("Appellant_Address3"),("Appellant_Address4"), col("Appellant_Address5"), col("Appellant_Postcode"),col("Appellant_Email"),col("Appellant_Telephone"), col("FCONumber"))
    valid_country_list = bronze_countries_postal_lookup_df.select(col("countryGovUkOocAdminJ").alias("valid_countryGovUkOocAdminJ")).distinct()
    valid_catagoryid_list = silver_c.groupBy("CaseNo").agg(F.collect_list("CategoryId").alias("valid_categoryIdList"))
    valid_HORef_cleansing = bronze_HORef_cleansing.select( col("CaseNo"),coalesce(col("HORef"), col("FCONumber")).alias("lu_HORef"))
    valid_reasonDescription = silver_m1.alias("m1").join(bronze_remission_lookup_df, on=["PaymentRemissionReason"], how="left").select("CaseNo", "ReasonDescription")
 
    df_final = df_final.join(valid_representation, on="CaseNo", how="left"
                            ).join(valid_country_list, on=col("CaseRep_Address5") == col("valid_countryGovUkOocAdminJ"), how="left"
                            ).join(valid_catagoryid_list, on="CaseNo", how="left"
                            ).join(valid_appealant_address, on="CaseNo", how="left"
                            ).join(valid_HORef_cleansing, on="CaseNo", how="left"
                            ).join(valid_reasonDescription, on="CaseNo", how="left")

 
    df_final = df_final.withColumn("is_valid", expr(dq_rules))

    # df_final = df_final.drop(col("dv_representation"), col("CaseRepAddress5"), col("CaseRepPostcode"), col("valid_countryGovUkOocAdminJ"))

    # columns_to_drop = ["dv_representation", "CaseRepAddress5", "CaseRepPostcode"]

    # if all(col in df_final.columns for col in columns_to_drop): #If the columns exist - remove 
    #     df_final = df_final.drop(*columns_to_drop)

    return df_final

In [0]:
@dlt.table(
    name="stg_valid_payment_pending_records",
    comment="Delta Live Gold Table with JSON content.",
    path=f"{audit_path}/stg_valid_payment_pending_records"
)
def stg_valid_payment_pending_records():
    """
    Delta Live Table for creating and uploading JSON content for Appeals.
    """
    # Load source data
    df = dlt.read("stg_main_payment_pending_validation")

    df_filtered = df.filter(
        (col("is_valid") == True)
    )

    # Repartition to optimize parallelism
    repartitioned_df = df_filtered.repartition(64)

    df_with_upload_status = repartitioned_df.filter(~col("JSON_content").like("Error%")).withColumn(
            "Status", upload_udf(col("JSON_File_Name"), col("JSON_content"))
        )

    # Return the DataFrame for DLT table creation
    return df_with_upload_status.select("CaseNo", "JSON_content",col("JSON_File_Name").alias("File_Name"),"Status")


In [0]:
@dlt.table(
    name="stg_invalid_payment_pending_quarantine_records",
    comment="Quarantined records that failed data quality checks or JSON generation.",
    path=f"{audit_path}/stg_invalid_payment_pending_quarantine_records"
)
def stg_invalid_payment_pending_quarantine_records():

    df = dlt.read("stg_main_payment_pending_validation")

    df_filtered = df.filter(
        (col("is_valid") != True)
    ).withColumn("JSON_File_Name", regexp_replace(col("JSON_File_Name"), "/JSON/", "/INVALID_JSON/"))

    # Repartition to optimize parallelism
    repartitioned_df = df_filtered.repartition(64)

    df_with_upload_status = repartitioned_df.filter(~col("JSON_content").like("Error%")).withColumn(
            "Status", upload_udf(col("JSON_File_Name"), col("JSON_content"))
        )

    return df_with_upload_status.select("CaseNo", "JSON_content",col("JSON_File_Name").alias("File_Name"),"Status") 


In [0]:
import dlt
from pyspark.sql.functions import col, lit, expr

@dlt.table(
    name="apl_active_payment_pending_cr_audit_table",
    comment="DLT table Covers 4.2 Silver layer LLD requirements: Audits CCD attributes, input field values, derived values, and all columns for validation and traceability.",
    path=f"{audit_path}/apl_active_payment_pending_cr_audit_table"
)
def apl_active_payment_pending_cr_audit_table():
    try:
        silver_m1 = dlt.read("silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()
        silver_m2 = dlt.read("silver_caseapplicant_detail").filter(col("dv_targetState") == lit(AppealState))
        bronze_appealtype_lookup_df = dlt.read("bronze_appealtype").distinct()
        bronze_hearing_centres_lookup_df = dlt.read("bronze_hearing_centres").distinct()
        silver_m3 = dlt.read("silver_status_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_c = dlt.read("ariadm_active_appeals.silver_appealcategory_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_h = dlt.read("silver_history_detail").filter(col("dv_targetState") == lit(AppealState))
        bronze_remission_lookup_df = dlt.read("bronze_remissions").distinct()
        bronze_remissions_lookup_df = dlt.read("bronze_remissions").distinct()
        bronze_countryFromAddress = dlt.read("bronze_countries_countryFromAddress")
        bronze_HORef_cleansing = dlt.read("bronze_HORef_cleansing")
        bronze_hearing_centres = dlt.read("bronze_hearing_centres")
        bronze_derive_hearing_centres = dlt.read("bronze_derive_hearing_centres")
      
    except:
        silver_m1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").filter(col("dv_targetState") == lit(AppealState)).distinct()
        silver_m2 = spark.table("ariadm_active_appeals.silver_caseapplicant_detail").filter(col("dv_targetState") == lit(AppealState))
        bronze_appealtype_lookup_df = spark.table("ariadm_active_appeals.bronze_appealtype").distinct()
        bronze_hearing_centres_lookup_df = spark.table("ariadm_active_appeals.bronze_hearing_centres").distinct()
        silver_m3 = spark.table("ariadm_active_appeals.silver_status_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_c = spark.table("ariadm_active_appeals.silver_appealcategory_detail").filter(col("dv_targetState") == lit(AppealState))
        silver_h = spark.table("ariadm_active_appeals.silver_history_detail").filter(col("dv_targetState") == lit(AppealState))
        bronze_remissions_lookup_df = spark.table("ariadm_active_appeals.bronze_remissions").distinct()
        bronze_countryFromAddress = spark.table("ariadm_active_appeals.bronze_countries_countryFromAddress")
        bronze_HORef_cleansing = spark.table("ariadm_active_appeals.bronze_HORef_cleansing")
        bronze_hearing_centres = spark.table("ariadm_active_appeals.bronze_hearing_centres")
        bronze_derive_hearing_centres = spark.table("ariadm_active_appeals.bronze_derive_hearing_centres")

 
    df_final,df_audit = mainPaymentPending(silver_m1, silver_m2, silver_m3, silver_c,silver_h, bronze_remissions_lookup_df, bronze_countryFromAddress, bronze_HORef_cleansing,bronze_hearing_centres,bronze_derive_hearing_centres)

    return df_audit

In [0]:
dbutils.notebook.exit("Notebook completed successfully")

## Appendix

### Analysis

In [0]:
# %sql
# drop schema ariadm_active_appeals cascade

In [0]:
# import json

# first_row = df_final.filter(df_final["CaseNo"] == "HU/00035/2017").select("JSON_Content").first()
# json_content = first_row["JSON_Content"]
# parsed_json = json.loads(json_content)
# display(parsed_json)

In [0]:
# from pyspark.sql.functions import col, count

# # Reading tables into DataFrames and labeling as M1 to M9
# M1 = spark.table("ariadm_active_appeals.silver_appealcase_detail").distinct()
# M2 = spark.table("ariadm_active_appeals.silver_caseapplicant_detail")
# M3 = spark.table("ariadm_active_appeals.silver_status_detail")
# M4 = spark.table("ariadm_active_appeals.silver_transaction_detail")
# M5 = spark.table("ariadm_active_appeals.silver_link_detail")
# M6 = spark.table("ariadm_active_appeals.silver_adjudicator_detail")
# M7 = spark.table("ariadm_active_appeals.silver_appealcategory_detail")
# M8 = spark.table("ariadm_active_appeals.silver_documentsreceived_detail")
# M9 = spark.table("ariadm_active_appeals.silver_history_detail")

# # Function to check for duplicates
# def check_duplicates(df, table_name):
#     duplicates = df.groupBy("caseno").agg(count("*").alias("count")).filter(col("count") > 1)
#     if duplicates.count() > 0:
#         displayHTML(f"<span style='color:red;'>&#x274C; Table {table_name} has duplicates.</span>")
#     else:
#         displayHTML(f"<span style='color:green;'>&#x2705; Table {table_name} has no duplicates.</span>")

# # Check for duplicates in each table
# check_duplicates(M1, "silver_appealcase_detail")
# check_duplicates(M2, "silver_caseapplicant_detail")
# check_duplicates(M3, "silver_status_detail")
# check_duplicates(M4, "silver_transaction_detail")
# check_duplicates(M5, "silver_link_detail")
# check_duplicates(M6, "silver_adjudicator_detail")
# check_duplicates(M7, "silver_appealcategory_detail")
# check_duplicates(M8, "silver_documentsreceived_detail")
# check_duplicates(M9, "silver_history_detail")